## Importing necessary libraries

In [1]:
import pandas as pd
import numpy as np
from collections import defaultdict

## Location Data Collection - TaskRabbit
 Extracted service locations from official website for TaskRabbit,  standardizing city/state formats and removing duplicates.

In [2]:
taskrabbit_cities = [
    # Arizona
    "Phoenix, AZ",
    "Tucson, AZ",

    # California
    "Alameda, CA",
    "Berkeley, CA",
    "Burbank, CA",
    "Fresno, CA",
    "Hillsborough, CA",
    "Los Angeles, CA",
    "Menlo Park, CA",
    "Mountain View, CA",
    "Newport Beach, CA",
    "Oakland, CA",
    "Orange County, CA",
    "Palo Alto, CA",
    "Pasadena, CA",
    "Piedmont, CA",
    "Redwood City, CA",
    "Sacramento, CA",
    "San Diego, CA",
    "San Francisco, CA",
    "San Jose, CA",
    "Santa Monica, CA",
    "Venice Beach, CA",
    "Walnut Creek, CA",
    "West Hollywood, CA",

    # Colorado
    "Denver, CO",

    # Connecticut
    "Bridgeport, CT",
    "Fairfield, CT",

    # Florida
    "Jacksonville, FL",
    "Miami, FL",
    "Orlando, FL",

    # Georgia
    "Atlanta, GA",

    # Hawaii
    "Honolulu, HI",

    # Illinois
    "Arlington Heights, IL",
    "Aurora, IL",
    "Bolingbrook, IL",
    "Chicago, IL",
    "Cicero, IL",
    "Evanston, IL",
    "Hoffman Estates, IL",
    "Naperville, IL",
    "Oak Lawn, IL",
    "Orland Park, IL",
    "Schaumberg, IL",

    # Indiana
    "Indianapolis, IN",

    # Iowa
    "Des Moines, IA",

    # Kentucky
    "Louisville, KY",

    # Louisiana
    "New Orleans, LA",

    # Maryland
    "Baltimore, MD",

    # Massachusetts
    "Boston, MA",

    # Michigan
    "Ann Arbor, MI",

    # Missouri
    "Kansas City, MO",
    "St. Louis, MO",

    # Nevada
    "Las Vegas, NV",

    # New Jersey
    "Jersey City, NJ",
    "Montclair, NJ",
    "Parsippany- Troy Hills, NJ",
    "Princeton, NJ",

    # New York
    "Bronx, NY",
    "Brooklyn, NY",
    "Hempstead, NY",
    "Huntington, NY",
    "Manhattan, NY",
    "New York, NY",
    "Queens, NY",
    "Staten Island, NY",
    "White Plains, NY",
    "Yonkers, NY",

    # North Carolina
    "Durham, NC",
    "Raleigh, NC",

    # Ohio
    "Cincinnati, OH",
    "Cleveland, OH",

    # Oklahoma
    "Oklahoma City, OK",

    # Oregon
    "Portland, OR",

    # Pennsylvania
    "Philadelphia, PA",
    "Pittsburgh, PA",

    # Rhode Island
    "Providence, RI",

    # South Carolina
    "Charleston, SC",

    # Tennessee
    "Memphis, TN",
    "Nashville, TN",

    # Texas
    "Austin, TX",
    "Carrollton, TX",
    "Dallas, TX",
    "Denton, TX",
    "El Paso, TX",
    "Fort Worth, TX",
    "Frisco, TX",
    "Houston, TX",
    "Irving, TX",
    "McKinney, TX",
    "Plano, TX",
    "Round Rock, TX",
    "San Antonio, TX",
    "Sugar Land, TX",
    "The Woodlands, TX",
    "Waco, TX",

    # Virginia
    "Norfolk, VA",
    "Richmond, VA",

    # Washington
    "Seattle, WA",

    # Wisconsin
    "Milwaukee, WI"
]

## MSA Mapping Process - TaskRabbit
Mapped each location to OMB-defined Metropolitan Statistical Areas using 2020 delineations and Census population data for market sizing analysis.

In [3]:
# TaskRabbit Cities mapped to Official OMB Metropolitan Statistical Areas (MSAs)
# Corrected to match official OMB naming conventions from Census data

city_to_msa_mapping = {
    # Arizona
    "Phoenix, AZ": "Phoenix-Mesa-Chandler, AZ Metro Area",
    "Tucson, AZ": "Tucson, AZ Metro Area",

    # California
    "Alameda, CA": "San Francisco-Oakland-Berkeley, CA Metro Area",
    "Berkeley, CA": "San Francisco-Oakland-Berkeley, CA Metro Area",
    "Burbank, CA": "Los Angeles-Long Beach-Anaheim, CA Metro Area",
    "Fresno, CA": "Fresno, CA Metro Area",
    "Hillsborough, CA": "San Francisco-Oakland-Berkeley, CA Metro Area",
    "Los Angeles, CA": "Los Angeles-Long Beach-Anaheim, CA Metro Area",
    "Menlo Park, CA": "San Francisco-Oakland-Berkeley, CA Metro Area",
    "Mountain View, CA": "San Jose-Sunnyvale-Santa Clara, CA Metro Area",
    "Newport Beach, CA": "Los Angeles-Long Beach-Anaheim, CA Metro Area",
    "Oakland, CA": "San Francisco-Oakland-Berkeley, CA Metro Area",
    "Orange County, CA": "Los Angeles-Long Beach-Anaheim, CA Metro Area",
    "Palo Alto, CA": "San Jose-Sunnyvale-Santa Clara, CA Metro Area",
    "Pasadena, CA": "Los Angeles-Long Beach-Anaheim, CA Metro Area",
    "Piedmont, CA": "San Francisco-Oakland-Berkeley, CA Metro Area",
    "Redwood City, CA": "San Francisco-Oakland-Berkeley, CA Metro Area",
    "Sacramento, CA": "Sacramento-Roseville-Folsom, CA Metro Area",
    "San Diego, CA": "San Diego-Chula Vista-Carlsbad, CA Metro Area",
    "San Francisco, CA": "San Francisco-Oakland-Berkeley, CA Metro Area",
    "San Jose, CA": "San Jose-Sunnyvale-Santa Clara, CA Metro Area",
    "Santa Monica, CA": "Los Angeles-Long Beach-Anaheim, CA Metro Area",
    "Venice Beach, CA": "Los Angeles-Long Beach-Anaheim, CA Metro Area",
    "Walnut Creek, CA": "San Francisco-Oakland-Berkeley, CA Metro Area",
    "West Hollywood, CA": "Los Angeles-Long Beach-Anaheim, CA Metro Area",

    # Colorado
    "Denver, CO": "Denver-Aurora-Lakewood, CO Metro Area",

    # Connecticut
    "Bridgeport, CT": "Bridgeport-Stamford-Norwalk, CT Metro Area",
    "Fairfield, CT": "Bridgeport-Stamford-Norwalk, CT Metro Area",

    # Florida
    "Jacksonville, FL": "Jacksonville, FL Metro Area",
    "Miami, FL": "Miami-Fort Lauderdale-Pompano Beach, FL Metro Area",
    "Orlando, FL": "Orlando-Kissimmee-Sanford, FL Metro Area",

    # Georgia
    "Atlanta, GA": "Atlanta-Sandy Springs-Alpharetta, GA Metro Area",

    # Hawaii
    "Honolulu, HI": "Urban Honolulu, HI Metro Area",

    # Illinois
    "Arlington Heights, IL": "Chicago-Naperville-Elgin, IL-IN-WI Metro Area",
    "Aurora, IL": "Chicago-Naperville-Elgin, IL-IN-WI Metro Area",
    "Bolingbrook, IL": "Chicago-Naperville-Elgin, IL-IN-WI Metro Area",
    "Chicago, IL": "Chicago-Naperville-Elgin, IL-IN-WI Metro Area",
    "Cicero, IL": "Chicago-Naperville-Elgin, IL-IN-WI Metro Area",
    "Evanston, IL": "Chicago-Naperville-Elgin, IL-IN-WI Metro Area",
    "Hoffman Estates, IL": "Chicago-Naperville-Elgin, IL-IN-WI Metro Area",
    "Naperville, IL": "Chicago-Naperville-Elgin, IL-IN-WI Metro Area",
    "Oak Lawn, IL": "Chicago-Naperville-Elgin, IL-IN-WI Metro Area",
    "Orland Park, IL": "Chicago-Naperville-Elgin, IL-IN-WI Metro Area",
    "Schaumberg, IL": "Chicago-Naperville-Elgin, IL-IN-WI Metro Area",

    # Indiana
    "Indianapolis, IN": "Indianapolis-Carmel-Anderson, IN Metro Area",

    # Iowa
    "Des Moines, IA": "Des Moines-West Des Moines, IA Metro Area",

    # Kentucky
    "Louisville, KY": "Louisville/Jefferson County, KY-IN Metro Area",

    # Louisiana
    "New Orleans, LA": "New Orleans-Metairie, LA Metro Area",

    # Maryland
    "Baltimore, MD": "Baltimore-Columbia-Towson, MD Metro Area",

    # Massachusetts
    "Boston, MA": "Boston-Cambridge-Newton, MA-NH Metro Area",

    # Michigan
    "Ann Arbor, MI": "Ann Arbor, MI Metro Area",

    # Missouri
    "Kansas City, MO": "Kansas City, MO-KS Metro Area",
    "St. Louis, MO": "St. Louis, MO-IL Metro Area",

    # Nevada
    "Las Vegas, NV": "Las Vegas-Henderson-Paradise, NV Metro Area",

    # New Jersey
    "Jersey City, NJ": "New York-Newark-Jersey City, NY-NJ-PA Metro Area",
    "Montclair, NJ": "New York-Newark-Jersey City, NY-NJ-PA Metro Area",
    "Parsippany- Troy Hills, NJ": "New York-Newark-Jersey City, NY-NJ-PA Metro Area",
    "Princeton, NJ": "Trenton-Princeton, NJ Metro Area",

    # New York
    "Bronx, NY": "New York-Newark-Jersey City, NY-NJ-PA Metro Area",
    "Brooklyn, NY": "New York-Newark-Jersey City, NY-NJ-PA Metro Area",
    "Hempstead, NY": "New York-Newark-Jersey City, NY-NJ-PA Metro Area",
    "Huntington, NY": "New York-Newark-Jersey City, NY-NJ-PA Metro Area",
    "Manhattan, NY": "New York-Newark-Jersey City, NY-NJ-PA Metro Area",
    "New York, NY": "New York-Newark-Jersey City, NY-NJ-PA Metro Area",
    "Queens, NY": "New York-Newark-Jersey City, NY-NJ-PA Metro Area",
    "Staten Island, NY": "New York-Newark-Jersey City, NY-NJ-PA Metro Area",
    "White Plains, NY": "New York-Newark-Jersey City, NY-NJ-PA Metro Area",
    "Yonkers, NY": "New York-Newark-Jersey City, NY-NJ-PA Metro Area",

    # North Carolina
    "Durham, NC": "Durham-Chapel Hill, NC Metro Area",
    "Raleigh, NC": "Raleigh-Cary, NC Metro Area",

    # Ohio
    "Cincinnati, OH": "Cincinnati, OH-KY-IN Metro Area",
    "Cleveland, OH": "Cleveland-Elyria, OH Metro Area",

    # Oklahoma
    "Oklahoma City, OK": "Oklahoma City, OK Metro Area",

    # Oregon
    "Portland, OR": "Portland-Vancouver-Hillsboro, OR-WA Metro Area",

    # Pennsylvania
    "Philadelphia, PA": "Philadelphia-Camden-Wilmington, PA-NJ-DE-MD Metro Area",
    "Pittsburgh, PA": "Pittsburgh, PA Metro Area",

    # Rhode Island
    "Providence, RI": "Providence-Warwick, RI-MA Metro Area",

    # South Carolina
    "Charleston, SC": "Charleston-North Charleston, SC Metro Area",

    # Tennessee
    "Memphis, TN": "Memphis, TN-MS-AR Metro Area",
    "Nashville, TN": "Nashville-Davidson--Murfreesboro--Franklin, TN Metro Area",

    # Texas
    "Austin, TX": "Austin-Round Rock-Georgetown, TX Metro Area",
    "Carrollton, TX": "Dallas-Fort Worth-Arlington, TX Metro Area",
    "Dallas, TX": "Dallas-Fort Worth-Arlington, TX Metro Area",
    "Denton, TX": "Dallas-Fort Worth-Arlington, TX Metro Area",
    "El Paso, TX": "El Paso, TX Metro Area",
    "Fort Worth, TX": "Dallas-Fort Worth-Arlington, TX Metro Area",
    "Frisco, TX": "Dallas-Fort Worth-Arlington, TX Metro Area",
    "Houston, TX": "Houston-The Woodlands-Sugar Land, TX Metro Area",
    "Irving, TX": "Dallas-Fort Worth-Arlington, TX Metro Area",
    "McKinney, TX": "Dallas-Fort Worth-Arlington, TX Metro Area",
    "Plano, TX": "Dallas-Fort Worth-Arlington, TX Metro Area",
    "Round Rock, TX": "Austin-Round Rock-Georgetown, TX Metro Area",
    "San Antonio, TX": "San Antonio-New Braunfels, TX Metro Area",
    "Sugar Land, TX": "Houston-The Woodlands-Sugar Land, TX Metro Area",
    "The Woodlands, TX": "Houston-The Woodlands-Sugar Land, TX Metro Area",
    "Waco, TX": "Waco, TX Metro Area",

    # Virginia
    "Norfolk, VA": "Virginia Beach-Norfolk-Newport News, VA-NC Metro Area",
    "Richmond, VA": "Richmond, VA Metro Area",

    # Washington
    "Seattle, WA": "Seattle-Tacoma-Bellevue, WA Metro Area",

    # Wisconsin
    "Milwaukee, WI": "Milwaukee-Waukesha, WI Metro Area"
}

# Add Minneapolis to TaskRabbit cities if not already there
taskrabbit_cities_to_add = ["Omaha", "Minneapolis", "St. Paul"]

for city in taskrabbit_cities_to_add:
    if city not in taskrabbit_cities:
        taskrabbit_cities.append(city)

# Update MSA mappings
city_to_msa_mapping["Omaha"] = "Omaha-Council Bluffs, NE-IA Metro Area"
city_to_msa_mapping["Minneapolis"] = "Minneapolis-St. Paul-Bloomington, MN-WI Metro Area"
city_to_msa_mapping["St. Paul"] = "Minneapolis-St. Paul-Bloomington, MN-WI Metro Area"

# Create a summary by MSA
msa_summary = {}
for city, msa in city_to_msa_mapping.items():
    if msa not in msa_summary:
        msa_summary[msa] = []
    msa_summary[msa].append(city)

# Print summary
print(f"Total TaskRabbit cities: {len(city_to_msa_mapping)}")
print(f"Total unique MSAs covered: {len(msa_summary)}")

# Show MSAs with multiple cities
print("\nMSAs with multiple TaskRabbit cities:")
for msa, cities in msa_summary.items():
    if len(cities) > 1:
        print(f"{msa}: {len(cities)} cities")
        for city in cities:
            print(f"  - {city}")
        print()

Total TaskRabbit cities: 103
Total unique MSAs covered: 52

MSAs with multiple TaskRabbit cities:
San Francisco-Oakland-Berkeley, CA Metro Area: 9 cities
  - Alameda, CA
  - Berkeley, CA
  - Hillsborough, CA
  - Menlo Park, CA
  - Oakland, CA
  - Piedmont, CA
  - Redwood City, CA
  - San Francisco, CA
  - Walnut Creek, CA

Los Angeles-Long Beach-Anaheim, CA Metro Area: 8 cities
  - Burbank, CA
  - Los Angeles, CA
  - Newport Beach, CA
  - Orange County, CA
  - Pasadena, CA
  - Santa Monica, CA
  - Venice Beach, CA
  - West Hollywood, CA

San Jose-Sunnyvale-Santa Clara, CA Metro Area: 3 cities
  - Mountain View, CA
  - Palo Alto, CA
  - San Jose, CA

Bridgeport-Stamford-Norwalk, CT Metro Area: 2 cities
  - Bridgeport, CT
  - Fairfield, CT

Chicago-Naperville-Elgin, IL-IN-WI Metro Area: 11 cities
  - Arlington Heights, IL
  - Aurora, IL
  - Bolingbrook, IL
  - Chicago, IL
  - Cicero, IL
  - Evanston, IL
  - Hoffman Estates, IL
  - Naperville, IL
  - Oak Lawn, IL
  - Orland Park, IL
  - S

## Location Data Collection - Dolly
 Extracted service locations from official website for Dolly,  standardizing city/state formats and removing duplicates.

In [4]:
dolly_cities = [
"Atlanta, GA",
"Austin, TX",
"Baltimore, MD",
"Boston, MA",
"Charlotte, NC",
"Chicago, IL",
"Cincinnati, OH",
"Cleveland, OH",
"Columbus, OH",
"Dallas, TX",
"Denver, CO",
"Detroit, MI",
"Fort Lauderdale, FL",
"Hartford, CT",
"Houston, TX",
"Indianapolis, IN",
"Kansas City, MO",
"Las Vegas, NV",
"Los Angeles, CA",
"Miami, FL",
"Milwaukee, WI",
"Minneapolis, MN",
"Nashville, TN",
"New Haven, CT",
"New York, NY",
"Orange County, CA",
"Orlando, FL",
"Philadelphia, PA",
"Phoenix, AZ",
"Pittsburgh, PA",
"Portland, OR",
"Providence, RI",
"Raleigh-Durham, NC",
"Richmond, VA",
"Sacramento, CA",
"Salt Lake City, UT",
"San Antonio, TX",
"San Diego, CA",
"San Francisco, CA",
"San Jose, CA",
"St. Louis, MO",
"Tampa, FL",
"Washington, DC",
"Wilmington, DE"
]

## MSA Mapping Process - Dolly
Mapped each location to OMB-defined Metropolitan Statistical Areas using 2020 delineations and Census population data for market sizing analysis.

In [5]:
# Dolly Cities mapped to Official OMB Metropolitan Statistical Areas (MSAs)
# Corrected to match official OMB naming conventions from Census data

dolly_city_to_msa_mapping = {
    "Atlanta, GA": "Atlanta-Sandy Springs-Alpharetta, GA Metro Area",
    "Austin, TX": "Austin-Round Rock-Georgetown, TX Metro Area",
    "Baltimore, MD": "Baltimore-Columbia-Towson, MD Metro Area",
    "Boston, MA": "Boston-Cambridge-Newton, MA-NH Metro Area",
    "Charlotte, NC": "Charlotte-Concord-Gastonia, NC-SC Metro Area",
    "Chicago, IL": "Chicago-Naperville-Elgin, IL-IN-WI Metro Area",
    "Cincinnati, OH": "Cincinnati, OH-KY-IN Metro Area",
    "Cleveland, OH": "Cleveland-Elyria, OH Metro Area",
    "Columbus, OH": "Columbus, OH Metro Area",
    "Dallas, TX": "Dallas-Fort Worth-Arlington, TX Metro Area",
    "Denver, CO": "Denver-Aurora-Lakewood, CO Metro Area",
    "Detroit, MI": "Detroit-Warren-Dearborn, MI Metro Area",
    "Fort Lauderdale, FL": "Miami-Fort Lauderdale-Pompano Beach, FL Metro Area",
    "Hartford, CT": "Hartford-East Hartford-Middletown, CT Metro Area",
    "Houston, TX": "Houston-The Woodlands-Sugar Land, TX Metro Area",
    "Indianapolis, IN": "Indianapolis-Carmel-Anderson, IN Metro Area",
    "Kansas City, MO": "Kansas City, MO-KS Metro Area",
    "Las Vegas, NV": "Las Vegas-Henderson-Paradise, NV Metro Area",
    "Los Angeles, CA": "Los Angeles-Long Beach-Anaheim, CA Metro Area",
    "Miami, FL": "Miami-Fort Lauderdale-Pompano Beach, FL Metro Area",
    "Milwaukee, WI": "Milwaukee-Waukesha, WI Metro Area",
    "Minneapolis, MN": "Minneapolis-St. Paul-Bloomington, MN-WI Metro Area",
    "Nashville, TN": "Nashville-Davidson--Murfreesboro--Franklin, TN Metro Area",
    "New Haven, CT": "New Haven-Milford, CT Metro Area",
    "New York, NY": "New York-Newark-Jersey City, NY-NJ-PA Metro Area",
    "Orange County, CA": "Los Angeles-Long Beach-Anaheim, CA Metro Area",
    "Orlando, FL": "Orlando-Kissimmee-Sanford, FL Metro Area",
    "Philadelphia, PA": "Philadelphia-Camden-Wilmington, PA-NJ-DE-MD Metro Area",
    "Phoenix, AZ": "Phoenix-Mesa-Chandler, AZ Metro Area",
    "Pittsburgh, PA": "Pittsburgh, PA Metro Area",
    "Portland, OR": "Portland-Vancouver-Hillsboro, OR-WA Metro Area",
    "Providence, RI": "Providence-Warwick, RI-MA Metro Area",
    "Raleigh-Durham, NC": "Raleigh-Cary, NC Metro Area",  # Note: Durham is separate MSA (Durham-Chapel Hill, NC Metro Area)
    "Richmond, VA": "Richmond, VA Metro Area",
    "Sacramento, CA": "Sacramento-Roseville-Folsom, CA Metro Area",
    "Salt Lake City, UT": "Salt Lake City, UT Metro Area",
    "San Antonio, TX": "San Antonio-New Braunfels, TX Metro Area",
    "San Diego, CA": "San Diego-Chula Vista-Carlsbad, CA Metro Area",
    "San Francisco, CA": "San Francisco-Oakland-Berkeley, CA Metro Area",
    "San Jose, CA": "San Jose-Sunnyvale-Santa Clara, CA Metro Area",
    "Seattle, WA": "Seattle-Tacoma-Bellevue, WA Metro Area",  # Note: Dolly lists this as covered
    "St. Louis, MO": "St. Louis, MO-IL Metro Area",
    "Tampa, FL": "Tampa-St. Petersburg-Clearwater, FL Metro Area",
    "Washington, DC": "Washington-Arlington-Alexandria, DC-VA-MD-WV Metro Area",
    "Wilmington, DE": "Philadelphia-Camden-Wilmington, PA-NJ-DE-MD Metro Area"
}

# Create a summary by MSA
msa_summary = {}
for city, msa in dolly_city_to_msa_mapping.items():
    if msa not in msa_summary:
        msa_summary[msa] = []
    msa_summary[msa].append(city)

# Print summary
print(f"Total Dolly cities: {len(dolly_city_to_msa_mapping)}")
print(f"Total unique MSAs covered: {len(msa_summary)}")

# Show MSAs with multiple cities
print("\nMSAs with multiple Dolly cities:")
for msa, cities in msa_summary.items():
    if len(cities) > 1:
        print(f"{msa}: {len(cities)} cities")
        for city in cities:
            print(f"  - {city}")
        print()

Total Dolly cities: 45
Total unique MSAs covered: 42

MSAs with multiple Dolly cities:
Miami-Fort Lauderdale-Pompano Beach, FL Metro Area: 2 cities
  - Fort Lauderdale, FL
  - Miami, FL

Los Angeles-Long Beach-Anaheim, CA Metro Area: 2 cities
  - Los Angeles, CA
  - Orange County, CA

Philadelphia-Camden-Wilmington, PA-NJ-DE-MD Metro Area: 2 cities
  - Philadelphia, PA
  - Wilmington, DE



## Location Data Collection - Goshare
 Extracted service locations from official website for Goshare,  standardizing city/state formats and removing duplicates.

In [6]:
goshare_cities = [
    # Alabama
    "Birmingham, AL",
    "Alabaster, AL",
    "Homewood, AL",
    "Hoover, AL",
    "Trussville, AL",
    "Vestavia Hills, AL",
    "Huntsville, AL",
    "Athens, AL",
    "Decatur, AL",
    "Hartselle, AL",
    "Harvest, AL",
    "Madison, AL",
    "Mobile, AL",
    "Daphne, AL",
    "Fairhope, AL",
    "Foley, AL",
    "Prichard, AL",
    "Saraland, AL",
    "Montgomery, AL",

    # Arkansas
    "Little Rock, AR",
    "Benton, AR",
    "Conway, AR",
    "Jacksonville, AR",
    "North Little Rock, AR",
    "Sherwood, AR",

    # Arizona
    "Phoenix, AZ",
    "Chandler, AZ",
    "Gilbert, AZ",
    "Glendale, AZ",
    "Goodyear, AZ",
    "Mesa, AZ",
    "Peoria, AZ",
    "San Tan Valley, AZ",
    "Scottsdale, AZ",
    "Surprise, AZ",
    "Tempe, AZ",
    "Sierra Vista, AZ",
    "Tucson, AZ",
    "Casas Adobes, AZ",
    "Catalina Foothills, AZ",
    "Marana, AZ",
    "Oro Valley, AZ",
    "Sahuarita, AZ",
    "Yuma, AZ",

    # California
    "Anaheim, CA",
    "Costa Mesa, CA",
    "Fullerton, CA",
    "Garden Grove, CA",
    "Huntington Beach, CA",
    "Irvine, CA",
    "Lake Forest, CA",
    "Mission Viejo, CA",
    "Orange, CA",
    "Santa Ana, CA",
    "Westminster, CA",
    "Bakersfield, CA",
    "Chico, CA",
    "El Centro, CA",
    "Fresno, CA",
    "Clovis, CA",
    "Fowler, CA",
    "Madera, CA",
    "Sanger, CA",
    "Selma, CA",
    "Los Angeles, CA",
    "Downey, CA",
    "Glendale, CA",
    "Long Beach, CA",
    "Oxnard, CA",
    "Pasadena, CA",
    "Pomona, CA",
    "Santa Clarita, CA",
    "Simi Valley, CA",
    "Thousand Oaks, CA",
    "Torrance, CA",
    "Ventura, CA",
    "Monterey, CA",
    "Palm Springs, CA",
    "Redding, CA",
    "Sacramento, CA",
    "Arden-Arcade, CA",
    "Carmichael, CA",
    "Citrus Heights, CA",
    "Davis, CA",
    "Elk Grove, CA",
    "Folsom, CA",
    "Rancho Cordova, CA",
    "Rocklin, CA",
    "Roseville, CA",
    "Woodland, CA",
    "San Bernardino, CA",
    "Corona, CA",
    "Fontana, CA",
    "Jurupa Valley, CA",
    "Menifee, CA",
    "Moreno Valley, CA",
    "Murrieta, CA",
    "Ontario, CA",
    "Rancho Cucamonga, CA",
    "Riverside, CA",
    "Temecula, CA",
    "San Diego, CA",
    "Carlsbad, CA",
    "Chula Vista, CA",
    "El Cajon, CA",
    "Encinitas, CA",
    "Escondido, CA",
    "La Mesa, CA",
    "Oceanside, CA",
    "San Marcos, CA",
    "Santee, CA",
    "Vista, CA",
    "San Francisco, CA",
    "Antioch, CA",
    "Berkeley, CA",
    "Concord, CA",
    "Fremont, CA",
    "Hayward, CA",
    "Oakland, CA",
    "Richmond, CA",
    "San Jose, CA",
    "Santa Clara, CA",
    "Sunnyvale, CA",
    "San Luis Obispo, CA",
    "Santa Barbara, CA",
    "Santa Rosa, CA",
    "Stockton, CA",
    "Lodi, CA",
    "Manteca, CA",
    "Modesto, CA",
    "Tracy, CA",
    "Turlock, CA",
    "Vallejo, CA",
    "Visalia, CA",

    # Colorado
    "Colorado Springs, CO",
    "Denver, CO",
    "Arvada, CO",
    "Aurora, CO",
    "Boulder, CO",
    "Broomfield, CO",
    "Centennial, CO",
    "Highlands Ranch, CO",
    "Lakewood, CO",
    "Longmont, CO",
    "Thornton, CO",
    "Westminster, CO",

    # Connecticut
    "Bridgeport, CT",
    "Danbury, CT",
    "Fairfield, CT",
    "Greenwich, CT",
    "New Haven, CT",
    "Norwalk, CT",
    "Stamford, CT",
    "Hartford, CT",
    "Bristol, CT",
    "East Hartford, CT",
    "Enfield, CT",
    "Manchester, CT",
    "Meriden, CT",
    "Middletown, CT",
    "New Britain, CT",
    "Southington, CT",
    "Waterbury, CT",
    "West Hartford, CT",

    # District of Columbia
    "Washington, DC",

    # Delaware
    "Wilmington, DE",
    "Georgetown, DE",
    "Milford, DE",
    "Millsboro, DE",
    "Seaford, DE",

    # Florida
    "Crestview, FL",
    "Fort Myers, FL",
    "Bonita Springs, FL",
    "Cape Coral, FL",
    "Estero, FL",
    "Lehigh Acres, FL",
    "Naples, FL",
    "Gainesville, FL",
    "Jacksonville, FL",
    "Fleming Island, FL",
    "Fruit Cove, FL",
    "Jacksonville Beach, FL",
    "Lakeside, FL",
    "Oakleaf Plantation, FL",
    "Lake City, FL",
    "Marianna, FL",
    "Miami, FL",
    "Coral Springs, FL",
    "Davie, FL",
    "Fort Lauderdale, FL",
    "Hialeah, FL",
    "Hollywood, FL",
    "Miami Gardens, FL",
    "Miramar, FL",
    "Pembroke Pines, FL",
    "Pompano Beach, FL",
    "West Palm Beach, FL",
    "North Port, FL",
    "Orlando, FL",
    "Alafaya, FL",
    "Altamonte Springs, FL",
    "Apopka, FL",
    "Kissimmee, FL",
    "Ocoee, FL",
    "Pine Hills, FL",
    "Poinciana, FL",
    "Sanford, FL",
    "St. Cloud, FL",
    "Winter Garden, FL",
    "Panama City, FL",
    "Pensacola, FL",
    "Tallahassee, FL",
    "Tampa, FL",
    "Brandon, FL",
    "Clearwater, FL",
    "Largo, FL",
    "Palm Harbor, FL",
    "Pinellas Park, FL",
    "Riverview, FL",
    "Spring Hill, FL",
    "St. Petersburg, FL",
    "Town 'n' Country, FL",
    "Wesley Chapel, FL",

    # Georgia
    "Atlanta, GA",
    "Alpharetta, GA",
    "Brookhaven, GA",
    "Dunwoody, GA",
    "Johns Creek, GA",
    "Marietta, GA",
    "Roswell, GA",
    "Sandy Springs, GA",
    "Smyrna, GA",
    "South Fulton, GA",
    "Stonecrest, GA",
    "Dalton, GA",

    # Idaho
    "Boise, ID",

    # Illinois
    "Chicago, IL",
    "Arlington Heights, IL",
    "Aurora, IL",
    "Cicero, IL",
    "Elgin, IL",
    "Evanston, IL",
    "Joliet, IL",
    "Naperville, IL",
    "Schaumburg, IL",
    "Waukegan, IL",
    "Peoria, IL",
    "Belleville, IL",
    "Edwardsville, IL",

    # Indiana
    "Hammond, IN",
    "Indianapolis, IN",
    "Carmel, IN",
    "Fishers, IN",
    "Greenwood, IN",
    "Lawrence, IN",
    "Noblesville, IN",
    "Clarksville, IN",
    "Jeffersonville, IN",
    "New Albany, IN",

    # Kansas
    "Kansas City, KS",
    "Leavenworth, KS",
    "Leawood, KS",
    "Lenexa, KS",
    "Olathe, KS",
    "Overland Park, KS",
    "Wichita, KS",

    # Kentucky
    "Covington, KY",
    "Florence, KY",
    "Independence, KY",
    "Louisville, KY",
    "Jeffersontown, KY",
    "Mount Washington, KY",

    # Louisiana
    "Baton Rouge, LA",
    "Baker, LA",
    "Central, LA",
    "Denham Springs, LA",
    "Gonzales, LA",
    "Zachary, LA",
    "New Orleans, LA",
    "Kenner, LA",
    "Laplace, LA",
    "Marrero, LA",
    "Metairie, LA",
    "Slidell, LA",

    # Massachusetts
    "Boston, MA",
    "Brockton, MA",
    "Cambridge, MA",
    "Framingham, MA",
    "Haverhill, MA",
    "Lawrence, MA",
    "Lowell, MA",
    "Lynn, MA",
    "Newton, MA",
    "Quincy, MA",
    "Somerville, MA",
    "Attleboro, MA",
    "New Bedford, MA",

    # Maryland
    "Baltimore, MD",
    "Annapolis, MD",
    "Bel Air, MD",
    "Catonsville, MD",
    "Columbia, MD",
    "Dundalk, MD",
    "Ellicott City, MD",
    "Glen Burnie, MD",
    "Odenton, MD",
    "Severn, MD",
    "Towson, MD",
    "Salisbury, MD",
    "Ocean City, MD",
    "Bethesda, MD",
    "Frederick, MD",
    "Gaithersburg, MD",
    "Germantown, MD",
    "Silver Spring, MD",
    "Waldorf, MD",

    # Michigan
    "Detroit, MI",
    "Ann Arbor, MI",
    "Dearborn, MI",
    "Farmington Hills, MI",
    "Livonia, MI",
    "Rochester Hills, MI",
    "Southfield, MI",
    "Sterling Heights, MI",
    "Troy, MI",
    "Warren, MI",
    "Westland, MI",

    # Minnesota
    "Brainerd, MN",
    "Duluth, MN",
    "Minneapolis, MN",
    "Blaine, MN",
    "Bloomington, MN",
    "Brooklyn Park, MN",
    "Burnsville, MN",
    "Eagan, MN",
    "Lakeville, MN",
    "Maple Grove, MN",
    "Plymouth, MN",
    "St. Paul, MN",
    "Woodbury, MN",
    "Rochester, MN",
    "St. Cloud, MN",

    # Missouri
    "Jefferson City, MO",
    "Kansas City, MO",
    "Blue Springs, MO",
    "Independence, MO",
    "Lee's Summit, MO",
    "Liberty, MO",
    "Lake Ozark, MO",
    "Springfield, MO",
    "St. Joseph, MO",
    "St. Louis, MO",
    "Chesterfield, MO",
    "Florissant, MO",
    "O'Fallon, MO",
    "St. Charles, MO",
    "St. Peters, MO",
    "University City, MO",
    "Wentzville, MO",
    "Wildwood, MO",

    # Mississippi
    "Jackson, MS",
    "Olive Branch, MS",
    "Southaven, MS",

    # North Carolina
    "Charlotte, NC",
    "Concord, NC",
    "Gastonia, NC",
    "Hickory, NC",
    "Huntersville, NC",
    "Indian Trail, NC",
    "Kannapolis, NC",
    "Monroe, NC",
    "Mooresville, NC",
    "Salisbury, NC",
    "Greensboro, NC",
    "Asheboro, NC",
    "Burlington, NC",
    "Clemmons, NC",
    "Graham, NC",
    "High Point, NC",
    "Kernersville, NC",
    "Lexington, NC",
    "Reidsville, NC",
    "Thomasville, NC",
    "Winston-Salem, NC",
    "Raleigh, NC",
    "Apex, NC",
    "Cary, NC",
    "Chapel Hill, NC",
    "Clayton, NC",
    "Durham, NC",
    "Fuquay-Varina, NC",
    "Garner, NC",
    "Holly Springs, NC",
    "Morrisville, NC",
    "Wake Forest, NC",

    # New Jersey
    "Newark, NJ",
    "Bayonne, NJ",
    "Clifton, NJ",
    "East Orange, NJ",
    "Edison, NJ",
    "Elizabeth, NJ",
    "Hamilton, NJ",
    "Jersey City, NJ",
    "Lakewood, NJ",
    "Middletown, NJ",
    "Passaic, NJ",
    "Patterson, NJ",
    "Toms River, NJ",
    "Camden, NJ",
    "Cherry Hill, NJ",
    "Gloucester, NJ",

    # New Mexico
    "Albuquerque, NM",

    # Nevada
    "Las Vegas, NV",
    "Enterprise, NV",
    "Henderson, NV",
    "North Las Vegas, NV",
    "Spring Valley, NV",
    "Sunrise Manor, NV",
    "Reno, NV",

    # New York
    "Albany, NY",
    "Amsterdam, NY",
    "Glens Falls, NY",
    "Saratoga Springs, NY",
    "Schenectady, NY",
    "Troy, NY",
    "Buffalo, NY",
    "New York, NY",
    "Brentwood, NY",
    "Bronx, NY",
    "Brooklyn, NY",
    "Freeport, NY",
    "Hempstead, NY",
    "Hicksville, NY",
    "Levittown, NY",
    "Manhattan, NY",
    "Mount Vernon, NY",
    "New Rochelle, NY",
    "Queens, NY",
    "Staten Island, NY",
    "West Babylon, NY",
    "White Plains, NY",
    "Yonkers, NY",
    "Rochester, NY",
    "Greece, NY",
    "Henrietta, NY",
    "Irondequoit, NY",
    "Penfield, NY",
    "Webster, NY",

    # Ohio
    "Cincinnati, OH",
    "Fairfield, OH",
    "Forest Park, OH",
    "Hamilton, OH",
    "Lebanon, OH",
    "Mason, OH",
    "Middletown, OH",
    "Oxford, OH",
    "Cleveland, OH",
    "Akron, OH",
    "Canton, OH",
    "Cuyahoga Falls, OH",
    "Elyria, OH",
    "Euclid, OH",
    "Lakewood, OH",
    "Lorain, OH",
    "Mentor, OH",
    "Parma, OH",
    "Strongsville, OH",
    "Columbus, OH",
    "Delaware, OH",
    "Dublin, OH",
    "Gahanna, OH",
    "Grove City, OH",
    "Hilliard, OH",
    "Lancaster, OH",
    "Pickerington, OH",
    "Reynoldsburg, OH",
    "Upper Arlington, OH",
    "Westerville, OH",
    "Dayton, OH",
    "Beavercreek, OH",
    "Centerville, OH",
    "Fairborn, OH",
    "Huber Heights, OH",
    "Kettering, OH",
    "Riverside, OH",
    "Springfield, OH",
    "Trotwood, OH",
    "Troy, OH",
    "Xenia, OH",
    "Toledo, OH",
    "Bowling Green, OH",
    "Maumee, OH",
    "Oregon, OH",
    "Perrysburg, OH",
    "Sylvania, OH",

    # Oklahoma
    "Oklahoma City, OK",
    "Edmond, OK",
    "Midwest City, OK",
    "Moore, OK",
    "Norman, OK",
    "Shawnee, OK",
    "Tulsa, OK",
    "Bixby, OK",
    "Broken Arrow, OK",
    "Jenks, OK",
    "Owasso, OK",
    "Sapulpa, OK",

    # Oregon
    "Portland, OR",
    "Beaverton, OR",
    "Gresham, OR",
    "Hillsboro, OR",
    "Lake Oswego, OR",
    "Oregon City, OR",
    "Tigard, OR",
    "Tualatin, OR",
    "West Linn, OR",
    "Wilsonville, OR",

    # Pennsylvania
    "Allentown, PA",
    "Philadelphia, PA",
    "Abington, PA",
    "Bensalem, PA",
    "Bristol, PA",
    "Levittown, PA",
    "Lower Merion, PA",
    "Upper Darby, PA",
    "Pittsburgh, PA",
    "Bethel Park, PA",
    "Cranberry, PA",
    "Hempfield, PA",
    "McCandless, PA",
    "Monroeville, PA",
    "Moon, PA",
    "Mount Lebanon, PA",
    "North Huntingdon, PA",
    "Penn Hills, PA",
    "Ross, PA",

    # Rhode Island
    "Providence, RI",
    "Cranston, RI",
    "East Providence, RI",
    "Pawtucket, RI",
    "Warwick, RI",
    "Woonsocket, RI",

    # South Carolina
    "Charleston, SC",
    "Goose Creek, SC",
    "Hanahan, SC",
    "Mount Pleasant, SC",
    "North Charleston, SC",
    "Summerville, SC",
    "Rock Hill, SC",
    "Columbia, SC",
    "Lexington, SC",
    "Orangeburg, SC",
    "Pontiac, SC",
    "St. Andrews, SC",
    "Sumter, SC",
    "Greenville, SC",
    "Anderson, SC",
    "Easley, SC",
    "Greer, SC",
    "Mauldin, SC",
    "Spartanburg, SC",
    "Hilton Head Island, SC",
    "Myrtle Beach, SC",
    "Conway, SC",
    "Georgetown, SC",
    "Little River, SC",
    "Murrells Inlet, SC",
    "North Myrtle Beach, SC",

    # Tennessee
    "Chattanooga, TN",
    "Cleveland, TN",
    "Collegedale, TN",
    "East Ridge, TN",
    "Soddy-Daisy, TN",
    "Knoxville, TN",
    "Farragut, TN",
    "Maryville, TN",
    "Morristown, TN",
    "Oak Ridge, TN",
    "Sevierville, TN",
    "Memphis, TN",
    "Bartlett, TN",
    "Collierville, TN",
    "Germantown, TN",
    "Nashville, TN",
    "Brentwood, TN",
    "Columbia, TN",
    "Franklin, TN",
    "Gallatin, TN",
    "Hendersonville, TN",
    "Lebanon, TN",
    "Mt. Juliet, TN",
    "Murfreesboro, TN",
    "Smyrna, TN",
    "Spring Hill, TN",

    # Texas
    "Austin, TX",
    "Buda, TX",
    "Cedar Park, TX",
    "Georgetown, TX",
    "Hutto, TX",
    "Kyle, TX",
    "Lakeway, TX",
    "Leander, TX",
    "Manor, TX",
    "Pflugerville, TX",
    "Round Rock, TX",
    "Dallas, TX",
    "Arlington, TX",
    "Denton, TX",
    "Fort Worth, TX",
    "Frisco, TX",
    "Garland, TX",
    "Grand Prairie, TX",
    "Irving, TX",
    "McKinney, TX",
    "Mesquite, TX",
    "Plano, TX",
    "El Paso, TX",
    "Anthony, TX",
    "Canutillo, TX",
    "Horizon City, TX",
    "San Elizario, TX",
    "Socorro, TX",
    "Houston, TX",
    "Atascocita, TX",
    "Baytown, TX",
    "Conroe, TX",
    "League City, TX",
    "Missouri City, TX",
    "Pasadena, TX",
    "Pearland, TX",
    "Spring, TX",
    "Sugar Land, TX",
    "The Woodlands, TX",
    "San Antonio, TX",
    "Boerne, TX",
    "Canyon Lake, TX",
    "Cibolo, TX",
    "Converse, TX",
    "Leon Valley, TX",
    "Live Oak, TX",
    "New Braunfels, TX",
    "Schertz, TX",
    "Timberwood Park, TX",
    "Universal City, TX",

    # Utah
    "Salt Lake City, UT",
    "Layton, UT",
    "Lehi, UT",
    "Millcreek, UT",
    "Ogden, UT",
    "Orem, UT",
    "Provo, UT",
    "Sandy, UT",
    "South Jordan, UT",
    "West Jordan, UT",
    "West Valley City, UT",

    # Virginia
    "Charlottesville, VA",
    "Harrisonburg, VA",
    "Ruckersville, VA",
    "Staunton, VA",
    "Stuarts Draft, VA",
    "Waynesboro, VA",
    "Richmond, VA",
    "Chester, VA",
    "Colonial Heights, VA",
    "Glen Allen, VA",
    "Highland Springs, VA",
    "Hopewell, VA",
    "Mechanicsville, VA",
    "Midlothian, VA",
    "Petersburg, VA",
    "Short Pump, VA",
    "Tuckahoe, VA",
    "Roanoke, VA",
    "Blacksburg, VA",
    "Cave Spring, VA",
    "Christiansburg, VA",
    "Lynchburg, VA",
    "Salem, VA",
    "Virginia Beach, VA",
    "Chesapeake, VA",
    "Gloucester Point, VA",
    "Hampton, VA",
    "James City, VA",
    "Newport News, VA",
    "Norfolk, VA",
    "Poquoson, VA",
    "Portsmouth, VA",
    "Suffolk, VA",
    "Williamsburg, VA",
    "Alexandria, VA",
    "Arlington, VA",
    "Centreville, VA",
    "Dale City, VA",

    # Washington
    "Vancouver, WA",
    "Seattle, WA",
    "Auburn, WA",
    "Bellevue, WA",
    "Federal Way, WA",
    "Kent, WA",
    "Kirkland, WA",
    "Lakewood, WA",
    "Redmond, WA",
    "Renton, WA",
    "Sammamish, WA",
    "Tacoma, WA",

    # Wisconsin
    "Milwaukee, WI",
    "Kenosha, WI",
    "Racine, WI",
    "Waukesha, WI",
    "Wauwatosa, WI",
    "West Allis, WI"
]

## MSA Mapping Process - Goshare
Mapped each location to OMB-defined Metropolitan Statistical Areas using 2020 delineations and Census population data for market sizing analysis.

In [7]:
# GoShare Cities mapped to Official OMB Metropolitan Statistical Areas (MSAs)
# Corrected to match official OMB naming conventions from Census data
# MAJOR CORRECTIONS: Added "Metro Area" suffix to ALL names + Fixed incorrect MSA names

goshare_city_to_msa_mapping = {
    # Alabama
    "Birmingham, AL": "Birmingham-Hoover, AL Metro Area",
    "Alabaster, AL": "Birmingham-Hoover, AL Metro Area",
    "Homewood, AL": "Birmingham-Hoover, AL Metro Area",
    "Hoover, AL": "Birmingham-Hoover, AL Metro Area",
    "Trussville, AL": "Birmingham-Hoover, AL Metro Area",
    "Vestavia Hills, AL": "Birmingham-Hoover, AL Metro Area",
    "Huntsville, AL": "Huntsville, AL Metro Area",
    "Athens, AL": "Huntsville, AL Metro Area",
    "Decatur, AL": "Decatur, AL Metro Area",
    "Hartselle, AL": "Decatur, AL Metro Area",
    "Harvest, AL": "Huntsville, AL Metro Area",
    "Madison, AL": "Huntsville, AL Metro Area",
    "Mobile, AL": "Mobile, AL Metro Area",
    "Daphne, AL": "Daphne-Fairhope-Foley, AL Metro Area",
    "Fairhope, AL": "Daphne-Fairhope-Foley, AL Metro Area",
    "Foley, AL": "Daphne-Fairhope-Foley, AL Metro Area",
    "Prichard, AL": "Mobile, AL Metro Area",
    "Saraland, AL": "Mobile, AL Metro Area",
    "Montgomery, AL": "Montgomery, AL Metro Area",

    # Arkansas
    "Little Rock, AR": "Little Rock-North Little Rock-Conway, AR Metro Area",
    "Benton, AR": "Little Rock-North Little Rock-Conway, AR Metro Area",
    "Conway, AR": "Little Rock-North Little Rock-Conway, AR Metro Area",
    "Jacksonville, AR": "Little Rock-North Little Rock-Conway, AR Metro Area",
    "North Little Rock, AR": "Little Rock-North Little Rock-Conway, AR Metro Area",
    "Sherwood, AR": "Little Rock-North Little Rock-Conway, AR Metro Area",

    # Arizona
    "Phoenix, AZ": "Phoenix-Mesa-Chandler, AZ Metro Area",
    "Chandler, AZ": "Phoenix-Mesa-Chandler, AZ Metro Area",
    "Gilbert, AZ": "Phoenix-Mesa-Chandler, AZ Metro Area",
    "Glendale, AZ": "Phoenix-Mesa-Chandler, AZ Metro Area",
    "Goodyear, AZ": "Phoenix-Mesa-Chandler, AZ Metro Area",
    "Mesa, AZ": "Phoenix-Mesa-Chandler, AZ Metro Area",
    "Peoria, AZ": "Phoenix-Mesa-Chandler, AZ Metro Area",
    "San Tan Valley, AZ": "Phoenix-Mesa-Chandler, AZ Metro Area",
    "Scottsdale, AZ": "Phoenix-Mesa-Chandler, AZ Metro Area",
    "Surprise, AZ": "Phoenix-Mesa-Chandler, AZ Metro Area",
    "Tempe, AZ": "Phoenix-Mesa-Chandler, AZ Metro Area",
    "Sierra Vista, AZ": "Sierra Vista-Douglas, AZ Metro Area",
    "Tucson, AZ": "Tucson, AZ Metro Area",
    "Casas Adobes, AZ": "Tucson, AZ Metro Area",
    "Catalina Foothills, AZ": "Tucson, AZ Metro Area",
    "Marana, AZ": "Tucson, AZ Metro Area",
    "Oro Valley, AZ": "Tucson, AZ Metro Area",
    "Sahuarita, AZ": "Tucson, AZ Metro Area",
    "Yuma, AZ": "Yuma, AZ Metro Area",

    # California
    "Anaheim, CA": "Los Angeles-Long Beach-Anaheim, CA Metro Area",
    "Costa Mesa, CA": "Los Angeles-Long Beach-Anaheim, CA Metro Area",
    "Fullerton, CA": "Los Angeles-Long Beach-Anaheim, CA Metro Area",
    "Garden Grove, CA": "Los Angeles-Long Beach-Anaheim, CA Metro Area",
    "Huntington Beach, CA": "Los Angeles-Long Beach-Anaheim, CA Metro Area",
    "Irvine, CA": "Los Angeles-Long Beach-Anaheim, CA Metro Area",
    "Lake Forest, CA": "Los Angeles-Long Beach-Anaheim, CA Metro Area",
    "Mission Viejo, CA": "Los Angeles-Long Beach-Anaheim, CA Metro Area",
    "Orange, CA": "Los Angeles-Long Beach-Anaheim, CA Metro Area",
    "Santa Ana, CA": "Los Angeles-Long Beach-Anaheim, CA Metro Area",
    "Westminster, CA": "Los Angeles-Long Beach-Anaheim, CA Metro Area",
    "Bakersfield, CA": "Bakersfield, CA Metro Area",
    "Chico, CA": "Chico, CA Metro Area",
    "El Centro, CA": "El Centro, CA Metro Area",
    "Fresno, CA": "Fresno, CA Metro Area",
    "Clovis, CA": "Fresno, CA Metro Area",
    "Fowler, CA": "Fresno, CA Metro Area",
    "Madera, CA": "Madera, CA Metro Area",
    "Sanger, CA": "Fresno, CA Metro Area",
    "Selma, CA": "Fresno, CA Metro Area",
    "Los Angeles, CA": "Los Angeles-Long Beach-Anaheim, CA Metro Area",
    "Downey, CA": "Los Angeles-Long Beach-Anaheim, CA Metro Area",
    "Glendale, CA": "Los Angeles-Long Beach-Anaheim, CA Metro Area",
    "Long Beach, CA": "Los Angeles-Long Beach-Anaheim, CA Metro Area",
    "Oxnard, CA": "Oxnard-Thousand Oaks-Ventura, CA Metro Area",
    "Pasadena, CA": "Los Angeles-Long Beach-Anaheim, CA Metro Area",
    "Pomona, CA": "Los Angeles-Long Beach-Anaheim, CA Metro Area",
    "Santa Clarita, CA": "Los Angeles-Long Beach-Anaheim, CA Metro Area",
    "Simi Valley, CA": "Oxnard-Thousand Oaks-Ventura, CA Metro Area",
    "Thousand Oaks, CA": "Oxnard-Thousand Oaks-Ventura, CA Metro Area",
    "Torrance, CA": "Los Angeles-Long Beach-Anaheim, CA Metro Area",
    "Ventura, CA": "Oxnard-Thousand Oaks-Ventura, CA Metro Area",
    "Monterey, CA": "Salinas, CA Metro Area",
    "Palm Springs, CA": "Riverside-San Bernardino-Ontario, CA Metro Area",
    "Redding, CA": "Redding, CA Metro Area",
    "Sacramento, CA": "Sacramento-Roseville-Folsom, CA Metro Area",
    "Arden-Arcade, CA": "Sacramento-Roseville-Folsom, CA Metro Area",
    "Carmichael, CA": "Sacramento-Roseville-Folsom, CA Metro Area",
    "Citrus Heights, CA": "Sacramento-Roseville-Folsom, CA Metro Area",
    "Davis, CA": "Sacramento-Roseville-Folsom, CA Metro Area",
    "Elk Grove, CA": "Sacramento-Roseville-Folsom, CA Metro Area",
    "Folsom, CA": "Sacramento-Roseville-Folsom, CA Metro Area",
    "Rancho Cordova, CA": "Sacramento-Roseville-Folsom, CA Metro Area",
    "Rocklin, CA": "Sacramento-Roseville-Folsom, CA Metro Area",
    "Roseville, CA": "Sacramento-Roseville-Folsom, CA Metro Area",
    "Woodland, CA": "Sacramento-Roseville-Folsom, CA Metro Area",
    "San Bernardino, CA": "Riverside-San Bernardino-Ontario, CA Metro Area",
    "Corona, CA": "Riverside-San Bernardino-Ontario, CA Metro Area",
    "Fontana, CA": "Riverside-San Bernardino-Ontario, CA Metro Area",
    "Jurupa Valley, CA": "Riverside-San Bernardino-Ontario, CA Metro Area",
    "Menifee, CA": "Riverside-San Bernardino-Ontario, CA Metro Area",
    "Moreno Valley, CA": "Riverside-San Bernardino-Ontario, CA Metro Area",
    "Murrieta, CA": "Riverside-San Bernardino-Ontario, CA Metro Area",
    "Ontario, CA": "Riverside-San Bernardino-Ontario, CA Metro Area",
    "Rancho Cucamonga, CA": "Riverside-San Bernardino-Ontario, CA Metro Area",
    "Riverside, CA": "Riverside-San Bernardino-Ontario, CA Metro Area",
    "Temecula, CA": "Riverside-San Bernardino-Ontario, CA Metro Area",
    "San Diego, CA": "San Diego-Chula Vista-Carlsbad, CA Metro Area",
    "Carlsbad, CA": "San Diego-Chula Vista-Carlsbad, CA Metro Area",
    "Chula Vista, CA": "San Diego-Chula Vista-Carlsbad, CA Metro Area",
    "El Cajon, CA": "San Diego-Chula Vista-Carlsbad, CA Metro Area",
    "Encinitas, CA": "San Diego-Chula Vista-Carlsbad, CA Metro Area",
    "Escondido, CA": "San Diego-Chula Vista-Carlsbad, CA Metro Area",
    "La Mesa, CA": "San Diego-Chula Vista-Carlsbad, CA Metro Area",
    "Oceanside, CA": "San Diego-Chula Vista-Carlsbad, CA Metro Area",
    "San Marcos, CA": "San Diego-Chula Vista-Carlsbad, CA Metro Area",
    "Santee, CA": "San Diego-Chula Vista-Carlsbad, CA Metro Area",
    "Vista, CA": "San Diego-Chula Vista-Carlsbad, CA Metro Area",
    "San Francisco, CA": "San Francisco-Oakland-Berkeley, CA Metro Area",
    "Antioch, CA": "San Francisco-Oakland-Berkeley, CA Metro Area",
    "Berkeley, CA": "San Francisco-Oakland-Berkeley, CA Metro Area",
    "Concord, CA": "San Francisco-Oakland-Berkeley, CA Metro Area",
    "Fremont, CA": "San Francisco-Oakland-Berkeley, CA Metro Area",
    "Hayward, CA": "San Francisco-Oakland-Berkeley, CA Metro Area",
    "Oakland, CA": "San Francisco-Oakland-Berkeley, CA Metro Area",
    "Richmond, CA": "San Francisco-Oakland-Berkeley, CA Metro Area",
    "San Jose, CA": "San Jose-Sunnyvale-Santa Clara, CA Metro Area",
    "Santa Clara, CA": "San Jose-Sunnyvale-Santa Clara, CA Metro Area",
    "Sunnyvale, CA": "San Jose-Sunnyvale-Santa Clara, CA Metro Area",
    "San Luis Obispo, CA": "San Luis Obispo-Paso Robles, CA Metro Area",
    "Santa Barbara, CA": "Santa Maria-Santa Barbara, CA Metro Area",
    "Santa Rosa, CA": "Santa Rosa-Petaluma, CA Metro Area",
    "Stockton, CA": "Stockton, CA Metro Area",
    "Lodi, CA": "Stockton, CA Metro Area",
    "Manteca, CA": "Stockton, CA Metro Area",
    "Modesto, CA": "Modesto, CA Metro Area",
    "Tracy, CA": "Stockton, CA Metro Area",
    "Turlock, CA": "Modesto, CA Metro Area",
    "Vallejo, CA": "Vallejo, CA Metro Area",
    "Visalia, CA": "Visalia, CA Metro Area",

    # Colorado
    "Colorado Springs, CO": "Colorado Springs, CO Metro Area",
    "Denver, CO": "Denver-Aurora-Lakewood, CO Metro Area",
    "Arvada, CO": "Denver-Aurora-Lakewood, CO Metro Area",
    "Aurora, CO": "Denver-Aurora-Lakewood, CO Metro Area",
    "Boulder, CO": "Boulder, CO Metro Area",
    "Broomfield, CO": "Denver-Aurora-Lakewood, CO Metro Area",
    "Centennial, CO": "Denver-Aurora-Lakewood, CO Metro Area",
    "Highlands Ranch, CO": "Denver-Aurora-Lakewood, CO Metro Area",
    "Lakewood, CO": "Denver-Aurora-Lakewood, CO Metro Area",
    "Longmont, CO": "Boulder, CO Metro Area",
    "Thornton, CO": "Denver-Aurora-Lakewood, CO Metro Area",
    "Westminster, CO": "Denver-Aurora-Lakewood, CO Metro Area",

    # Connecticut
    "Bridgeport, CT": "Bridgeport-Stamford-Norwalk, CT Metro Area",
    "Danbury, CT": "Bridgeport-Stamford-Norwalk, CT Metro Area",
    "Fairfield, CT": "Bridgeport-Stamford-Norwalk, CT Metro Area",
    "Greenwich, CT": "Bridgeport-Stamford-Norwalk, CT Metro Area",
    "New Haven, CT": "New Haven-Milford, CT Metro Area",
    "Norwalk, CT": "Bridgeport-Stamford-Norwalk, CT Metro Area",
    "Stamford, CT": "Bridgeport-Stamford-Norwalk, CT Metro Area",
    "Hartford, CT": "Hartford-East Hartford-Middletown, CT Metro Area",
    "Bristol, CT": "Hartford-East Hartford-Middletown, CT Metro Area",
    "East Hartford, CT": "Hartford-East Hartford-Middletown, CT Metro Area",
    "Enfield, CT": "Hartford-East Hartford-Middletown, CT Metro Area",
    "Manchester, CT": "Hartford-East Hartford-Middletown, CT Metro Area",
    "Meriden, CT": "Hartford-East Hartford-Middletown, CT Metro Area",
    "Middletown, CT": "Hartford-East Hartford-Middletown, CT Metro Area",
    "New Britain, CT": "Hartford-East Hartford-Middletown, CT Metro Area",
    "Southington, CT": "Hartford-East Hartford-Middletown, CT Metro Area",
    "Waterbury, CT": "Waterbury, CT Metro Area",
    "West Hartford, CT": "Hartford-East Hartford-Middletown, CT Metro Area",

    # District of Columbia
    "Washington, DC": "Washington-Arlington-Alexandria, DC-VA-MD-WV Metro Area",

    # Delaware
    "Wilmington, DE": "Philadelphia-Camden-Wilmington, PA-NJ-DE-MD Metro Area",
    "Georgetown, DE": "Seaford, DE Micro Area",
    "Milford, DE": "Dover, DE Metro Area",
    "Millsboro, DE": "Seaford, DE Micro Area",
    "Seaford, DE": "Seaford, DE Micro Area",

    # Florida
    "Crestview, FL": "Crestview-Fort Walton Beach-Destin, FL Metro Area",
    "Fort Myers, FL": "Cape Coral-Fort Myers, FL Metro Area",
    "Bonita Springs, FL": "Cape Coral-Fort Myers, FL Metro Area",
    "Cape Coral, FL": "Cape Coral-Fort Myers, FL Metro Area",
    "Estero, FL": "Cape Coral-Fort Myers, FL Metro Area",
    "Lehigh Acres, FL": "Cape Coral-Fort Myers, FL Metro Area",
    "Naples, FL": "Naples-Marco Island, FL Metro Area",
    "Gainesville, FL": "Gainesville, FL Metro Area",
    "Jacksonville, FL": "Jacksonville, FL Metro Area",
    "Fleming Island, FL": "Jacksonville, FL Metro Area",
    "Fruit Cove, FL": "Jacksonville, FL Metro Area",
    "Jacksonville Beach, FL": "Jacksonville, FL Metro Area",
    "Lakeside, FL": "Jacksonville, FL Metro Area",
    "Oakleaf Plantation, FL": "Jacksonville, FL Metro Area",
    "Lake City, FL": "Lake City, FL Micro Area",
    "Marianna, FL": "Tallahassee, FL Metro Area",
    "Miami, FL": "Miami-Fort Lauderdale-Pompano Beach, FL Metro Area",
    "Coral Springs, FL": "Miami-Fort Lauderdale-Pompano Beach, FL Metro Area",
    "Davie, FL": "Miami-Fort Lauderdale-Pompano Beach, FL Metro Area",
    "Fort Lauderdale, FL": "Miami-Fort Lauderdale-Pompano Beach, FL Metro Area",
    "Hialeah, FL": "Miami-Fort Lauderdale-Pompano Beach, FL Metro Area",
    "Hollywood, FL": "Miami-Fort Lauderdale-Pompano Beach, FL Metro Area",
    "Miami Gardens, FL": "Miami-Fort Lauderdale-Pompano Beach, FL Metro Area",
    "Miramar, FL": "Miami-Fort Lauderdale-Pompano Beach, FL Metro Area",
    "Pembroke Pines, FL": "Miami-Fort Lauderdale-Pompano Beach, FL Metro Area",
    "Pompano Beach, FL": "Miami-Fort Lauderdale-Pompano Beach, FL Metro Area",
    "West Palm Beach, FL": "Miami-Fort Lauderdale-Pompano Beach, FL Metro Area",
    "North Port, FL": "North Port-Sarasota-Bradenton, FL Metro Area",
    "Orlando, FL": "Orlando-Kissimmee-Sanford, FL Metro Area",
    "Alafaya, FL": "Orlando-Kissimmee-Sanford, FL Metro Area",
    "Altamonte Springs, FL": "Orlando-Kissimmee-Sanford, FL Metro Area",
    "Apopka, FL": "Orlando-Kissimmee-Sanford, FL Metro Area",
    "Kissimmee, FL": "Orlando-Kissimmee-Sanford, FL Metro Area",
    "Ocoee, FL": "Orlando-Kissimmee-Sanford, FL Metro Area",
    "Pine Hills, FL": "Orlando-Kissimmee-Sanford, FL Metro Area",
    "Poinciana, FL": "Orlando-Kissimmee-Sanford, FL Metro Area",
    "Sanford, FL": "Orlando-Kissimmee-Sanford, FL Metro Area",
    "St. Cloud, FL": "Orlando-Kissimmee-Sanford, FL Metro Area",
    "Winter Garden, FL": "Orlando-Kissimmee-Sanford, FL Metro Area",
    "Panama City, FL": "Panama City, FL Metro Area",
    "Pensacola, FL": "Pensacola-Ferry Pass-Brent, FL Metro Area",
    "Tallahassee, FL": "Tallahassee, FL Metro Area",
    "Tampa, FL": "Tampa-St. Petersburg-Clearwater, FL Metro Area",
    "Brandon, FL": "Tampa-St. Petersburg-Clearwater, FL Metro Area",
    "Clearwater, FL": "Tampa-St. Petersburg-Clearwater, FL Metro Area",
    "Largo, FL": "Tampa-St. Petersburg-Clearwater, FL Metro Area",
    "Palm Harbor, FL": "Tampa-St. Petersburg-Clearwater, FL Metro Area",
    "Pinellas Park, FL": "Tampa-St. Petersburg-Clearwater, FL Metro Area",
    "Riverview, FL": "Tampa-St. Petersburg-Clearwater, FL Metro Area",
    "Spring Hill, FL": "Homosassa Springs, FL Metro Area",
    "St. Petersburg, FL": "Tampa-St. Petersburg-Clearwater, FL Metro Area",
    "Town 'n' Country, FL": "Tampa-St. Petersburg-Clearwater, FL Metro Area",
    "Wesley Chapel, FL": "Tampa-St. Petersburg-Clearwater, FL Metro Area",

    # Georgia
    "Atlanta, GA": "Atlanta-Sandy Springs-Alpharetta, GA Metro Area",
    "Alpharetta, GA": "Atlanta-Sandy Springs-Alpharetta, GA Metro Area",
    "Brookhaven, GA": "Atlanta-Sandy Springs-Alpharetta, GA Metro Area",
    "Dunwoody, GA": "Atlanta-Sandy Springs-Alpharetta, GA Metro Area",
    "Johns Creek, GA": "Atlanta-Sandy Springs-Alpharetta, GA Metro Area",
    "Marietta, GA": "Atlanta-Sandy Springs-Alpharetta, GA Metro Area",
    "Roswell, GA": "Atlanta-Sandy Springs-Alpharetta, GA Metro Area",
    "Sandy Springs, GA": "Atlanta-Sandy Springs-Alpharetta, GA Metro Area",
    "Smyrna, GA": "Atlanta-Sandy Springs-Alpharetta, GA Metro Area",
    "South Fulton, GA": "Atlanta-Sandy Springs-Alpharetta, GA Metro Area",
    "Stonecrest, GA": "Atlanta-Sandy Springs-Alpharetta, GA Metro Area",
    "Dalton, GA": "Dalton, GA Metro Area",

    # Idaho
    "Boise, ID": "Boise City, ID Metro Area",

    # Illinois
    "Chicago, IL": "Chicago-Naperville-Elgin, IL-IN-WI Metro Area",
    "Arlington Heights, IL": "Chicago-Naperville-Elgin, IL-IN-WI Metro Area",
    "Aurora, IL": "Chicago-Naperville-Elgin, IL-IN-WI Metro Area",
    "Cicero, IL": "Chicago-Naperville-Elgin, IL-IN-WI Metro Area",
    "Elgin, IL": "Chicago-Naperville-Elgin, IL-IN-WI Metro Area",
    "Evanston, IL": "Chicago-Naperville-Elgin, IL-IN-WI Metro Area",
    "Joliet, IL": "Chicago-Naperville-Elgin, IL-IN-WI Metro Area",
    "Naperville, IL": "Chicago-Naperville-Elgin, IL-IN-WI Metro Area",
    "Schaumburg, IL": "Chicago-Naperville-Elgin, IL-IN-WI Metro Area",
    "Waukegan, IL": "Chicago-Naperville-Elgin, IL-IN-WI Metro Area",
    "Peoria, IL": "Peoria, IL Metro Area",
    "Belleville, IL": "St. Louis, MO-IL Metro Area",
    "Edwardsville, IL": "St. Louis, MO-IL Metro Area",

    # Indiana
    "Hammond, IN": "Chicago-Naperville-Elgin, IL-IN-WI Metro Area",
    "Indianapolis, IN": "Indianapolis-Carmel-Anderson, IN Metro Area",
    "Carmel, IN": "Indianapolis-Carmel-Anderson, IN Metro Area",
    "Fishers, IN": "Indianapolis-Carmel-Anderson, IN Metro Area",
    "Greenwood, IN": "Indianapolis-Carmel-Anderson, IN Metro Area",
    "Lawrence, IN": "Indianapolis-Carmel-Anderson, IN Metro Area",
    "Noblesville, IN": "Indianapolis-Carmel-Anderson, IN Metro Area",
    "Clarksville, IN": "Louisville/Jefferson County, KY-IN Metro Area",
    "Jeffersonville, IN": "Louisville/Jefferson County, KY-IN Metro Area",
    "New Albany, IN": "Louisville/Jefferson County, KY-IN Metro Area",

    # Kansas
    "Kansas City, KS": "Kansas City, MO-KS Metro Area",
    "Leavenworth, KS": "Kansas City, MO-KS Metro Area",
    "Leawood, KS": "Kansas City, MO-KS Metro Area",
    "Lenexa, KS": "Kansas City, MO-KS Metro Area",
    "Olathe, KS": "Kansas City, MO-KS Metro Area",
    "Overland Park, KS": "Kansas City, MO-KS Metro Area",
    "Wichita, KS": "Wichita, KS Metro Area",

    # Kentucky
    "Covington, KY": "Cincinnati, OH-KY-IN Metro Area",
    "Florence, KY": "Cincinnati, OH-KY-IN Metro Area",
    "Independence, KY": "Cincinnati, OH-KY-IN Metro Area",
    "Louisville, KY": "Louisville/Jefferson County, KY-IN Metro Area",
    "Jeffersontown, KY": "Louisville/Jefferson County, KY-IN Metro Area",
    "Mount Washington, KY": "Louisville/Jefferson County, KY-IN Metro Area",

    # Louisiana
    "Baton Rouge, LA": "Baton Rouge, LA Metro Area",
    "Baker, LA": "Baton Rouge, LA Metro Area",
    "Central, LA": "Baton Rouge, LA Metro Area",
    "Denham Springs, LA": "Baton Rouge, LA Metro Area",
    "Gonzales, LA": "Baton Rouge, LA Metro Area",
    "Zachary, LA": "Baton Rouge, LA Metro Area",
    "New Orleans, LA": "New Orleans-Metairie, LA Metro Area",
    "Kenner, LA": "New Orleans-Metairie, LA Metro Area",
    "Laplace, LA": "New Orleans-Metairie, LA Metro Area",
    "Marrero, LA": "New Orleans-Metairie, LA Metro Area",
    "Metairie, LA": "New Orleans-Metairie, LA Metro Area",
    "Slidell, LA": "New Orleans-Metairie, LA Metro Area",

    # Massachusetts
    "Boston, MA": "Boston-Cambridge-Newton, MA-NH Metro Area",
    "Brockton, MA": "Boston-Cambridge-Newton, MA-NH Metro Area",
    "Cambridge, MA": "Boston-Cambridge-Newton, MA-NH Metro Area",
    "Framingham, MA": "Boston-Cambridge-Newton, MA-NH Metro Area",
    "Haverhill, MA": "Boston-Cambridge-Newton, MA-NH Metro Area",
    "Lawrence, MA": "Boston-Cambridge-Newton, MA-NH Metro Area",
    "Lowell, MA": "Boston-Cambridge-Newton, MA-NH Metro Area",
    "Lynn, MA": "Boston-Cambridge-Newton, MA-NH Metro Area",
    "Newton, MA": "Boston-Cambridge-Newton, MA-NH Metro Area",
    "Quincy, MA": "Boston-Cambridge-Newton, MA-NH Metro Area",
    "Somerville, MA": "Boston-Cambridge-Newton, MA-NH Metro Area",
    "Attleboro, MA": "Providence-Warwick, RI-MA Metro Area",
    "New Bedford, MA": "New Bedford, MA Metro Area",

    # Maryland
    "Baltimore, MD": "Baltimore-Columbia-Towson, MD Metro Area",
    "Annapolis, MD": "Baltimore-Columbia-Towson, MD Metro Area",
    "Bel Air, MD": "Baltimore-Columbia-Towson, MD Metro Area",
    "Catonsville, MD": "Baltimore-Columbia-Towson, MD Metro Area",
    "Columbia, MD": "Baltimore-Columbia-Towson, MD Metro Area",
    "Dundalk, MD": "Baltimore-Columbia-Towson, MD Metro Area",
    "Ellicott City, MD": "Baltimore-Columbia-Towson, MD Metro Area",
    "Glen Burnie, MD": "Baltimore-Columbia-Towson, MD Metro Area",
    "Odenton, MD": "Baltimore-Columbia-Towson, MD Metro Area",
    "Severn, MD": "Baltimore-Columbia-Towson, MD Metro Area",
    "Towson, MD": "Baltimore-Columbia-Towson, MD Metro Area",
    "Salisbury, MD": "Salisbury, MD-DE Metro Area",
    "Ocean City, MD": "Ocean City, NJ Metro Area",
    "Bethesda, MD": "Washington-Arlington-Alexandria, DC-VA-MD-WV Metro Area",
    "Frederick, MD": "Washington-Arlington-Alexandria, DC-VA-MD-WV Metro Area",
    "Gaithersburg, MD": "Washington-Arlington-Alexandria, DC-VA-MD-WV Metro Area",
    "Germantown, MD": "Washington-Arlington-Alexandria, DC-VA-MD-WV Metro Area",
    "Silver Spring, MD": "Washington-Arlington-Alexandria, DC-VA-MD-WV Metro Area",
    "Waldorf, MD": "Washington-Arlington-Alexandria, DC-VA-MD-WV Metro Area",

    # Michigan
    "Detroit, MI": "Detroit-Warren-Dearborn, MI Metro Area",
    "Ann Arbor, MI": "Ann Arbor, MI Metro Area",
    "Dearborn, MI": "Detroit-Warren-Dearborn, MI Metro Area",
    "Farmington Hills, MI": "Detroit-Warren-Dearborn, MI Metro Area",
    "Livonia, MI": "Detroit-Warren-Dearborn, MI Metro Area",
    "Rochester Hills, MI": "Detroit-Warren-Dearborn, MI Metro Area",
    "Southfield, MI": "Detroit-Warren-Dearborn, MI Metro Area",
    "Sterling Heights, MI": "Detroit-Warren-Dearborn, MI Metro Area",
    "Troy, MI": "Detroit-Warren-Dearborn, MI Metro Area",
    "Warren, MI": "Detroit-Warren-Dearborn, MI Metro Area",
    "Westland, MI": "Detroit-Warren-Dearborn, MI Metro Area",

    # Minnesota
    "Brainerd, MN": "Brainerd, MN Micro Area",
    "Duluth, MN": "Duluth, MN-WI Metro Area",
    "Minneapolis, MN": "Minneapolis-St. Paul-Bloomington, MN-WI Metro Area",
    "Blaine, MN": "Minneapolis-St. Paul-Bloomington, MN-WI Metro Area",
    "Bloomington, MN": "Minneapolis-St. Paul-Bloomington, MN-WI Metro Area",
    "Brooklyn Park, MN": "Minneapolis-St. Paul-Bloomington, MN-WI Metro Area",
    "Burnsville, MN": "Minneapolis-St. Paul-Bloomington, MN-WI Metro Area",
    "Eagan, MN": "Minneapolis-St. Paul-Bloomington, MN-WI Metro Area",
    "Lakeville, MN": "Minneapolis-St. Paul-Bloomington, MN-WI Metro Area",
    "Maple Grove, MN": "Minneapolis-St. Paul-Bloomington, MN-WI Metro Area",
    "Plymouth, MN": "Minneapolis-St. Paul-Bloomington, MN-WI Metro Area",
    "St. Paul, MN": "Minneapolis-St. Paul-Bloomington, MN-WI Metro Area",
    "Woodbury, MN": "Minneapolis-St. Paul-Bloomington, MN-WI Metro Area",
    "Rochester, MN": "Rochester, MN Metro Area",
    "St. Cloud, MN": "St. Cloud, MN Metro Area",

    # Missouri
    "Jefferson City, MO": "Jefferson City, MO Metro Area",
    "Kansas City, MO": "Kansas City, MO-KS Metro Area",
    "Blue Springs, MO": "Kansas City, MO-KS Metro Area",
    "Independence, MO": "Kansas City, MO-KS Metro Area",
    "Lee's Summit, MO": "Kansas City, MO-KS Metro Area",
    "Liberty, MO": "Kansas City, MO-KS Metro Area",
    "Lake Ozark, MO": "Jefferson City, MO Metro Area",
    "Springfield, MO": "Springfield, MO Metro Area",
    "St. Joseph, MO": "St. Joseph, MO-KS Metro Area",
    "St. Louis, MO": "St. Louis, MO-IL Metro Area",
    "Chesterfield, MO": "St. Louis, MO-IL Metro Area",
    "Florissant, MO": "St. Louis, MO-IL Metro Area",
    "O'Fallon, MO": "St. Louis, MO-IL Metro Area",
    "St. Charles, MO": "St. Louis, MO-IL Metro Area",
    "St. Peters, MO": "St. Louis, MO-IL Metro Area",
    "University City, MO": "St. Louis, MO-IL Metro Area",
    "Wentzville, MO": "St. Louis, MO-IL Metro Area",
    "Wildwood, MO": "St. Louis, MO-IL Metro Area",

    # Mississippi
    "Jackson, MS": "Jackson, MS Metro Area",
    "Olive Branch, MS": "Memphis, TN-MS-AR Metro Area",
    "Southaven, MS": "Memphis, TN-MS-AR Metro Area",

    # North Carolina
    "Charlotte, NC": "Charlotte-Concord-Gastonia, NC-SC Metro Area",
    "Concord, NC": "Charlotte-Concord-Gastonia, NC-SC Metro Area",
    "Gastonia, NC": "Charlotte-Concord-Gastonia, NC-SC Metro Area",
    "Hickory, NC": "Hickory-Lenoir-Morganton, NC Metro Area",
    "Huntersville, NC": "Charlotte-Concord-Gastonia, NC-SC Metro Area",
    "Indian Trail, NC": "Charlotte-Concord-Gastonia, NC-SC Metro Area",
    "Kannapolis, NC": "Charlotte-Concord-Gastonia, NC-SC Metro Area",
    "Monroe, NC": "Charlotte-Concord-Gastonia, NC-SC Metro Area",
    "Mooresville, NC": "Charlotte-Concord-Gastonia, NC-SC Metro Area",
    "Salisbury, NC": "Charlotte-Concord-Gastonia, NC-SC Metro Area",
    "Greensboro, NC": "Greensboro-High Point, NC Metro Area",
    "Asheboro, NC": "Greensboro-High Point, NC Metro Area",
    "Burlington, NC": "Greensboro-High Point, NC Metro Area",
    "Clemmons, NC": "Winston-Salem, NC Metro Area",
    "Graham, NC": "Greensboro-High Point, NC Metro Area",
    "High Point, NC": "Greensboro-High Point, NC Metro Area",
    "Kernersville, NC": "Winston-Salem, NC Metro Area",
    "Lexington, NC": "Greensboro-High Point, NC Metro Area",
    "Reidsville, NC": "Greensboro-High Point, NC Metro Area",
    "Thomasville, NC": "Greensboro-High Point, NC Metro Area",
    "Winston-Salem, NC": "Winston-Salem, NC Metro Area",
    "Raleigh, NC": "Raleigh-Cary, NC Metro Area",
    "Apex, NC": "Raleigh-Cary, NC Metro Area",
    "Cary, NC": "Raleigh-Cary, NC Metro Area",
    "Chapel Hill, NC": "Durham-Chapel Hill, NC Metro Area",
    "Clayton, NC": "Raleigh-Cary, NC Metro Area",
    "Durham, NC": "Durham-Chapel Hill, NC Metro Area",
    "Fuquay-Varina, NC": "Raleigh-Cary, NC Metro Area",
    "Garner, NC": "Raleigh-Cary, NC Metro Area",
    "Holly Springs, NC": "Raleigh-Cary, NC Metro Area",
    "Morrisville, NC": "Durham-Chapel Hill, NC Metro Area",
    "Wake Forest, NC": "Raleigh-Cary, NC Metro Area",

    # New Jersey
    "Newark, NJ": "New York-Newark-Jersey City, NY-NJ-PA Metro Area",
    "Bayonne, NJ": "New York-Newark-Jersey City, NY-NJ-PA Metro Area",
    "Clifton, NJ": "New York-Newark-Jersey City, NY-NJ-PA Metro Area",
    "East Orange, NJ": "New York-Newark-Jersey City, NY-NJ-PA Metro Area",
    "Edison, NJ": "New York-Newark-Jersey City, NY-NJ-PA Metro Area",
    "Elizabeth, NJ": "New York-Newark-Jersey City, NY-NJ-PA Metro Area",
    "Hamilton, NJ": "Trenton-Princeton, NJ Metro Area",
    "Jersey City, NJ": "New York-Newark-Jersey City, NY-NJ-PA Metro Area",
    "Lakewood, NJ": "New York-Newark-Jersey City, NY-NJ-PA Metro Area",
    "Middletown, NJ": "New York-Newark-Jersey City, NY-NJ-PA Metro Area",
    "Passaic, NJ": "New York-Newark-Jersey City, NY-NJ-PA Metro Area",
    "Patterson, NJ": "New York-Newark-Jersey City, NY-NJ-PA Metro Area",
    "Toms River, NJ": "New York-Newark-Jersey City, NY-NJ-PA Metro Area",
    "Camden, NJ": "Philadelphia-Camden-Wilmington, PA-NJ-DE-MD Metro Area",
    "Cherry Hill, NJ": "Philadelphia-Camden-Wilmington, PA-NJ-DE-MD Metro Area",
    "Gloucester, NJ": "Philadelphia-Camden-Wilmington, PA-NJ-DE-MD Metro Area",

    # New Mexico
    "Albuquerque, NM": "Albuquerque, NM Metro Area",

    # Nevada
    "Las Vegas, NV": "Las Vegas-Henderson-Paradise, NV Metro Area",
    "Enterprise, NV": "Las Vegas-Henderson-Paradise, NV Metro Area",
    "Henderson, NV": "Las Vegas-Henderson-Paradise, NV Metro Area",
    "North Las Vegas, NV": "Las Vegas-Henderson-Paradise, NV Metro Area",
    "Spring Valley, NV": "Las Vegas-Henderson-Paradise, NV Metro Area",
    "Sunrise Manor, NV": "Las Vegas-Henderson-Paradise, NV Metro Area",
    "Reno, NV": "Reno, NV Metro Area",

    # New York
    "Albany, NY": "Albany-Schenectady-Troy, NY Metro Area",
    "Amsterdam, NY": "Albany-Schenectady-Troy, NY Metro Area",
    "Glens Falls, NY": "Glens Falls, NY Metro Area",
    "Saratoga Springs, NY": "Albany-Schenectady-Troy, NY Metro Area",
    "Schenectady, NY": "Albany-Schenectady-Troy, NY Metro Area",
    "Troy, NY": "Albany-Schenectady-Troy, NY Metro Area",
    "Buffalo, NY": "Buffalo-Cheektowaga, NY Metro Area",
    "New York, NY": "New York-Newark-Jersey City, NY-NJ-PA Metro Area",
    "Brentwood, NY": "New York-Newark-Jersey City, NY-NJ-PA Metro Area",
    "Bronx, NY": "New York-Newark-Jersey City, NY-NJ-PA Metro Area",
    "Brooklyn, NY": "New York-Newark-Jersey City, NY-NJ-PA Metro Area",
    "Freeport, NY": "New York-Newark-Jersey City, NY-NJ-PA Metro Area",
    "Hempstead, NY": "New York-Newark-Jersey City, NY-NJ-PA Metro Area",
    "Hicksville, NY": "New York-Newark-Jersey City, NY-NJ-PA Metro Area",
    "Levittown, NY": "New York-Newark-Jersey City, NY-NJ-PA Metro Area",
    "Manhattan, NY": "New York-Newark-Jersey City, NY-NJ-PA Metro Area",
    "Mount Vernon, NY": "New York-Newark-Jersey City, NY-NJ-PA Metro Area",
    "New Rochelle, NY": "New York-Newark-Jersey City, NY-NJ-PA Metro Area",
    "Queens, NY": "New York-Newark-Jersey City, NY-NJ-PA Metro Area",
    "Staten Island, NY": "New York-Newark-Jersey City, NY-NJ-PA Metro Area",
    "West Babylon, NY": "New York-Newark-Jersey City, NY-NJ-PA Metro Area",
    "White Plains, NY": "New York-Newark-Jersey City, NY-NJ-PA Metro Area",
    "Yonkers, NY": "New York-Newark-Jersey City, NY-NJ-PA Metro Area",
    "Rochester, NY": "Rochester, NY Metro Area",
    "Greece, NY": "Rochester, NY Metro Area",
    "Henrietta, NY": "Rochester, NY Metro Area",
    "Irondequoit, NY": "Rochester, NY Metro Area",
    "Penfield, NY": "Rochester, NY Metro Area",
    "Webster, NY": "Rochester, NY Metro Area",

    # Ohio
    "Cincinnati, OH": "Cincinnati, OH-KY-IN Metro Area",
    "Fairfield, OH": "Cincinnati, OH-KY-IN Metro Area",
    "Forest Park, OH": "Cincinnati, OH-KY-IN Metro Area",
    "Hamilton, OH": "Cincinnati, OH-KY-IN Metro Area",
    "Lebanon, OH": "Cincinnati, OH-KY-IN Metro Area",
    "Mason, OH": "Cincinnati, OH-KY-IN Metro Area",
    "Middletown, OH": "Cincinnati, OH-KY-IN Metro Area",
    "Oxford, OH": "Cincinnati, OH-KY-IN Metro Area",
    "Cleveland, OH": "Cleveland-Elyria, OH Metro Area",
    "Akron, OH": "Akron, OH Metro Area",
    "Canton, OH": "Canton-Massillon, OH Metro Area",
    "Cuyahoga Falls, OH": "Akron, OH Metro Area",
    "Elyria, OH": "Cleveland-Elyria, OH Metro Area",
    "Euclid, OH": "Cleveland-Elyria, OH Metro Area",
    "Lakewood, OH": "Cleveland-Elyria, OH Metro Area",
    "Lorain, OH": "Cleveland-Elyria, OH Metro Area",
    "Mentor, OH": "Cleveland-Elyria, OH Metro Area",
    "Parma, OH": "Cleveland-Elyria, OH Metro Area",
    "Strongsville, OH": "Cleveland-Elyria, OH Metro Area",
    "Columbus, OH": "Columbus, OH Metro Area",
    "Delaware, OH": "Columbus, OH Metro Area",
    "Dublin, OH": "Columbus, OH Metro Area",
    "Gahanna, OH": "Columbus, OH Metro Area",
    "Grove City, OH": "Columbus, OH Metro Area",
    "Hilliard, OH": "Columbus, OH Metro Area",
    "Lancaster, OH": "Columbus, OH Metro Area",
    "Pickerington, OH": "Columbus, OH Metro Area",
    "Reynoldsburg, OH": "Columbus, OH Metro Area",
    "Upper Arlington, OH": "Columbus, OH Metro Area",
    "Westerville, OH": "Columbus, OH Metro Area",
    "Dayton, OH": "Dayton-Kettering, OH Metro Area",
    "Beavercreek, OH": "Dayton-Kettering, OH Metro Area",
    "Centerville, OH": "Dayton-Kettering, OH Metro Area",
    "Fairborn, OH": "Dayton-Kettering, OH Metro Area",
    "Huber Heights, OH": "Dayton-Kettering, OH Metro Area",
    "Kettering, OH": "Dayton-Kettering, OH Metro Area",
    "Riverside, OH": "Dayton-Kettering, OH Metro Area",
    "Springfield, OH": "Springfield, OH Metro Area",
    "Trotwood, OH": "Dayton-Kettering, OH Metro Area",
    "Troy, OH": "Dayton-Kettering, OH Metro Area",
    "Xenia, OH": "Dayton-Kettering, OH Metro Area",
    "Toledo, OH": "Toledo, OH Metro Area",
    "Bowling Green, OH": "Toledo, OH Metro Area",
    "Maumee, OH": "Toledo, OH Metro Area",
    "Oregon, OH": "Toledo, OH Metro Area",
    "Perrysburg, OH": "Toledo, OH Metro Area",
    "Sylvania, OH": "Toledo, OH Metro Area",

    # Oklahoma
    "Oklahoma City, OK": "Oklahoma City, OK Metro Area",
    "Edmond, OK": "Oklahoma City, OK Metro Area",
    "Midwest City, OK": "Oklahoma City, OK Metro Area",
    "Moore, OK": "Oklahoma City, OK Metro Area",
    "Norman, OK": "Oklahoma City, OK Metro Area",
    "Shawnee, OK": "Shawnee, OK Micro Area",
    "Tulsa, OK": "Tulsa, OK Metro Area",
    "Bixby, OK": "Tulsa, OK Metro Area",
    "Broken Arrow, OK": "Tulsa, OK Metro Area",
    "Jenks, OK": "Tulsa, OK Metro Area",
    "Owasso, OK": "Tulsa, OK Metro Area",
    "Sapulpa, OK": "Tulsa, OK Metro Area",

    # Oregon
    "Portland, OR": "Portland-Vancouver-Hillsboro, OR-WA Metro Area",
    "Beaverton, OR": "Portland-Vancouver-Hillsboro, OR-WA Metro Area",
    "Gresham, OR": "Portland-Vancouver-Hillsboro, OR-WA Metro Area",
    "Hillsboro, OR": "Portland-Vancouver-Hillsboro, OR-WA Metro Area",
    "Lake Oswego, OR": "Portland-Vancouver-Hillsboro, OR-WA Metro Area",
    "Oregon City, OR": "Portland-Vancouver-Hillsboro, OR-WA Metro Area",
    "Tigard, OR": "Portland-Vancouver-Hillsboro, OR-WA Metro Area",
    "Tualatin, OR": "Portland-Vancouver-Hillsboro, OR-WA Metro Area",
    "West Linn, OR": "Portland-Vancouver-Hillsboro, OR-WA Metro Area",
    "Wilsonville, OR": "Portland-Vancouver-Hillsboro, OR-WA Metro Area",

    # Pennsylvania
    "Allentown, PA": "Allentown-Bethlehem-Easton, PA-NJ Metro Area",
    "Philadelphia, PA": "Philadelphia-Camden-Wilmington, PA-NJ-DE-MD Metro Area",
    "Abington, PA": "Philadelphia-Camden-Wilmington, PA-NJ-DE-MD Metro Area",
    "Bensalem, PA": "Philadelphia-Camden-Wilmington, PA-NJ-DE-MD Metro Area",
    "Bristol, PA": "Philadelphia-Camden-Wilmington, PA-NJ-DE-MD Metro Area",
    "Levittown, PA": "Philadelphia-Camden-Wilmington, PA-NJ-DE-MD Metro Area",
    "Lower Merion, PA": "Philadelphia-Camden-Wilmington, PA-NJ-DE-MD Metro Area",
    "Upper Darby, PA": "Philadelphia-Camden-Wilmington, PA-NJ-DE-MD Metro Area",
    "Pittsburgh, PA": "Pittsburgh, PA Metro Area",
    "Bethel Park, PA": "Pittsburgh, PA Metro Area",
    "Cranberry, PA": "Pittsburgh, PA Metro Area",
    "Hempfield, PA": "Pittsburgh, PA Metro Area",
    "McCandless, PA": "Pittsburgh, PA Metro Area",
    "Monroeville, PA": "Pittsburgh, PA Metro Area",
    "Moon, PA": "Pittsburgh, PA Metro Area",
    "Mount Lebanon, PA": "Pittsburgh, PA Metro Area",
    "North Huntingdon, PA": "Pittsburgh, PA Metro Area",
    "Penn Hills, PA": "Pittsburgh, PA Metro Area",
    "Ross, PA": "Pittsburgh, PA Metro Area",

    # Rhode Island
    "Providence, RI": "Providence-Warwick, RI-MA Metro Area",
    "Cranston, RI": "Providence-Warwick, RI-MA Metro Area",
    "East Providence, RI": "Providence-Warwick, RI-MA Metro Area",
    "Pawtucket, RI": "Providence-Warwick, RI-MA Metro Area",
    "Warwick, RI": "Providence-Warwick, RI-MA Metro Area",
    "Woonsocket, RI": "Providence-Warwick, RI-MA Metro Area",

    # South Carolina
    "Charleston, SC": "Charleston-North Charleston, SC Metro Area",
    "Goose Creek, SC": "Charleston-North Charleston, SC Metro Area",
    "Hanahan, SC": "Charleston-North Charleston, SC Metro Area",
    "Mount Pleasant, SC": "Charleston-North Charleston, SC Metro Area",
    "North Charleston, SC": "Charleston-North Charleston, SC Metro Area",
    "Summerville, SC": "Charleston-North Charleston, SC Metro Area",
    "Rock Hill, SC": "Charlotte-Concord-Gastonia, NC-SC Metro Area",
    "Columbia, SC": "Columbia, SC Metro Area",
    "Lexington, SC": "Columbia, SC Metro Area",
    "Orangeburg, SC": "Orangeburg, SC Micro Area",
    "Pontiac, SC": "Columbia, SC Metro Area",
    "St. Andrews, SC": "Columbia, SC Metro Area",
    "Sumter, SC": "Sumter, SC Metro Area",
    "Greenville, SC": "Greenville-Anderson, SC Metro Area",
    "Anderson, SC": "Greenville-Anderson, SC Metro Area",
    "Easley, SC": "Greenville-Anderson, SC Metro Area",
    "Greer, SC": "Greenville-Anderson, SC Metro Area",
    "Mauldin, SC": "Greenville-Anderson, SC Metro Area",
    "Spartanburg, SC": "Spartanburg, SC Metro Area",
    "Hilton Head Island, SC": "Hilton Head Island-Bluffton, SC Metro Area",
    "Myrtle Beach, SC": "Myrtle Beach-Conway-North Myrtle Beach, SC-NC Metro Area",
    "Conway, SC": "Myrtle Beach-Conway-North Myrtle Beach, SC-NC Metro Area",
    "Georgetown, SC": "Georgetown, SC Micro Area",
    "Little River, SC": "Myrtle Beach-Conway-North Myrtle Beach, SC-NC Metro Area",
    "Murrells Inlet, SC": "Georgetown, SC Micro Area",
    "North Myrtle Beach, SC": "Myrtle Beach-Conway-North Myrtle Beach, SC-NC Metro Area",

    # Tennessee
    "Chattanooga, TN": "Chattanooga, TN-GA Metro Area",
    "Cleveland, TN": "Cleveland, TN Metro Area",
    "Collegedale, TN": "Chattanooga, TN-GA Metro Area",
    "East Ridge, TN": "Chattanooga, TN-GA Metro Area",
    "Soddy-Daisy, TN": "Chattanooga, TN-GA Metro Area",
    "Knoxville, TN": "Knoxville, TN Metro Area",
    "Farragut, TN": "Knoxville, TN Metro Area",
    "Maryville, TN": "Knoxville, TN Metro Area",
    "Morristown, TN": "Morristown, TN Metro Area",
    "Oak Ridge, TN": "Knoxville, TN Metro Area",
    "Sevierville, TN": "Sevierville, TN Micro Area",
    "Memphis, TN": "Memphis, TN-MS-AR Metro Area",
    "Bartlett, TN": "Memphis, TN-MS-AR Metro Area",
    "Collierville, TN": "Memphis, TN-MS-AR Metro Area",
    "Germantown, TN": "Memphis, TN-MS-AR Metro Area",
    "Nashville, TN": "Nashville-Davidson--Murfreesboro--Franklin, TN Metro Area",
    "Brentwood, TN": "Nashville-Davidson--Murfreesboro--Franklin, TN Metro Area",
    "Columbia, TN": "Nashville-Davidson--Murfreesboro--Franklin, TN Metro Area",
    "Franklin, TN": "Nashville-Davidson--Murfreesboro--Franklin, TN Metro Area",
    "Gallatin, TN": "Nashville-Davidson--Murfreesboro--Franklin, TN Metro Area",
    "Hendersonville, TN": "Nashville-Davidson--Murfreesboro--Franklin, TN Metro Area",
    "Lebanon, TN": "Nashville-Davidson--Murfreesboro--Franklin, TN Metro Area",
    "Mt. Juliet, TN": "Nashville-Davidson--Murfreesboro--Franklin, TN Metro Area",
    "Murfreesboro, TN": "Nashville-Davidson--Murfreesboro--Franklin, TN Metro Area",
    "Smyrna, TN": "Nashville-Davidson--Murfreesboro--Franklin, TN Metro Area",
    "Spring Hill, TN": "Nashville-Davidson--Murfreesboro--Franklin, TN Metro Area",

    # Texas
    "Austin, TX": "Austin-Round Rock-Georgetown, TX Metro Area",
    "Buda, TX": "Austin-Round Rock-Georgetown, TX Metro Area",
    "Cedar Park, TX": "Austin-Round Rock-Georgetown, TX Metro Area",
    "Georgetown, TX": "Austin-Round Rock-Georgetown, TX Metro Area",
    "Hutto, TX": "Austin-Round Rock-Georgetown, TX Metro Area",
    "Kyle, TX": "Austin-Round Rock-Georgetown, TX Metro Area",
    "Lakeway, TX": "Austin-Round Rock-Georgetown, TX Metro Area",
    "Leander, TX": "Austin-Round Rock-Georgetown, TX Metro Area",
    "Manor, TX": "Austin-Round Rock-Georgetown, TX Metro Area",
    "Pflugerville, TX": "Austin-Round Rock-Georgetown, TX Metro Area",
    "Round Rock, TX": "Austin-Round Rock-Georgetown, TX Metro Area",
    "Dallas, TX": "Dallas-Fort Worth-Arlington, TX Metro Area",
    "Arlington, TX": "Dallas-Fort Worth-Arlington, TX Metro Area",
    "Denton, TX": "Dallas-Fort Worth-Arlington, TX Metro Area",
    "Fort Worth, TX": "Dallas-Fort Worth-Arlington, TX Metro Area",
    "Frisco, TX": "Dallas-Fort Worth-Arlington, TX Metro Area",
    "Garland, TX": "Dallas-Fort Worth-Arlington, TX Metro Area",
    "Grand Prairie, TX": "Dallas-Fort Worth-Arlington, TX Metro Area",
    "Irving, TX": "Dallas-Fort Worth-Arlington, TX Metro Area",
    "McKinney, TX": "Dallas-Fort Worth-Arlington, TX Metro Area",
    "Mesquite, TX": "Dallas-Fort Worth-Arlington, TX Metro Area",
    "Plano, TX": "Dallas-Fort Worth-Arlington, TX Metro Area",
    "El Paso, TX": "El Paso, TX Metro Area",
    "Anthony, TX": "El Paso, TX Metro Area",
    "Canutillo, TX": "El Paso, TX Metro Area",
    "Horizon City, TX": "El Paso, TX Metro Area",
    "San Elizario, TX": "El Paso, TX Metro Area",
    "Socorro, TX": "El Paso, TX Metro Area",
    "Houston, TX": "Houston-The Woodlands-Sugar Land, TX Metro Area",
    "Atascocita, TX": "Houston-The Woodlands-Sugar Land, TX Metro Area",
    "Baytown, TX": "Houston-The Woodlands-Sugar Land, TX Metro Area",
    "Conroe, TX": "Houston-The Woodlands-Sugar Land, TX Metro Area",
    "League City, TX": "Houston-The Woodlands-Sugar Land, TX Metro Area",
    "Missouri City, TX": "Houston-The Woodlands-Sugar Land, TX Metro Area",
    "Pasadena, TX": "Houston-The Woodlands-Sugar Land, TX Metro Area",
    "Pearland, TX": "Houston-The Woodlands-Sugar Land, TX Metro Area",
    "Spring, TX": "Houston-The Woodlands-Sugar Land, TX Metro Area",
    "Sugar Land, TX": "Houston-The Woodlands-Sugar Land, TX Metro Area",
    "The Woodlands, TX": "Houston-The Woodlands-Sugar Land, TX Metro Area",
    "San Antonio, TX": "San Antonio-New Braunfels, TX Metro Area",
    "Boerne, TX": "San Antonio-New Braunfels, TX Metro Area",
    "Canyon Lake, TX": "San Antonio-New Braunfels, TX Metro Area",
    "Cibolo, TX": "San Antonio-New Braunfels, TX Metro Area",
    "Converse, TX": "San Antonio-New Braunfels, TX Metro Area",
    "Leon Valley, TX": "San Antonio-New Braunfels, TX Metro Area",
    "Live Oak, TX": "San Antonio-New Braunfels, TX Metro Area",
    "New Braunfels, TX": "San Antonio-New Braunfels, TX Metro Area",
    "Schertz, TX": "San Antonio-New Braunfels, TX Metro Area",
    "Timberwood Park, TX": "San Antonio-New Braunfels, TX Metro Area",
    "Universal City, TX": "San Antonio-New Braunfels, TX Metro Area",

    # Utah
    "Salt Lake City, UT": "Salt Lake City, UT Metro Area",
    "Layton, UT": "Ogden-Clearfield, UT Metro Area",
    "Lehi, UT": "Provo-Orem, UT Metro Area",
    "Millcreek, UT": "Salt Lake City, UT Metro Area",
    "Ogden, UT": "Ogden-Clearfield, UT Metro Area",
    "Orem, UT": "Provo-Orem, UT Metro Area",
    "Provo, UT": "Provo-Orem, UT Metro Area",
    "Sandy, UT": "Salt Lake City, UT Metro Area",
    "South Jordan, UT": "Salt Lake City, UT Metro Area",
    "West Jordan, UT": "Salt Lake City, UT Metro Area",
    "West Valley City, UT": "Salt Lake City, UT Metro Area",

    # Virginia
    "Charlottesville, VA": "Charlottesville, VA Metro Area",
    "Harrisonburg, VA": "Harrisonburg, VA Metro Area",
    "Ruckersville, VA": "Charlottesville, VA Metro Area",
    "Staunton, VA": "Staunton, VA Metro Area",
    "Stuarts Draft, VA": "Staunton, VA Metro Area",
    "Waynesboro, VA": "Staunton, VA Metro Area",
    "Richmond, VA": "Richmond, VA Metro Area",
    "Chester, VA": "Richmond, VA Metro Area",
    "Colonial Heights, VA": "Richmond, VA Metro Area",
    "Glen Allen, VA": "Richmond, VA Metro Area",
    "Highland Springs, VA": "Richmond, VA Metro Area",
    "Hopewell, VA": "Richmond, VA Metro Area",
    "Mechanicsville, VA": "Richmond, VA Metro Area",
    "Midlothian, VA": "Richmond, VA Metro Area",
    "Petersburg, VA": "Richmond, VA Metro Area",
    "Short Pump, VA": "Richmond, VA Metro Area",
    "Tuckahoe, VA": "Richmond, VA Metro Area",
    "Roanoke, VA": "Roanoke, VA Metro Area",
    "Blacksburg, VA": "Blacksburg-Christiansburg, VA Metro Area",
    "Cave Spring, VA": "Roanoke, VA Metro Area",
    "Christiansburg, VA": "Blacksburg-Christiansburg, VA Metro Area",
    "Lynchburg, VA": "Lynchburg, VA Metro Area",
    "Salem, VA": "Roanoke, VA Metro Area",
    "Virginia Beach, VA": "Virginia Beach-Norfolk-Newport News, VA-NC Metro Area",
    "Chesapeake, VA": "Virginia Beach-Norfolk-Newport News, VA-NC Metro Area",
    "Gloucester Point, VA": "Virginia Beach-Norfolk-Newport News, VA-NC Metro Area",
    "Hampton, VA": "Virginia Beach-Norfolk-Newport News, VA-NC Metro Area",
    "James City, VA": "Virginia Beach-Norfolk-Newport News, VA-NC Metro Area",
    "Newport News, VA": "Virginia Beach-Norfolk-Newport News, VA-NC Metro Area",
    "Norfolk, VA": "Virginia Beach-Norfolk-Newport News, VA-NC Metro Area",
    "Poquoson, VA": "Virginia Beach-Norfolk-Newport News, VA-NC Metro Area",
    "Portsmouth, VA": "Virginia Beach-Norfolk-Newport News, VA-NC Metro Area",
    "Suffolk, VA": "Virginia Beach-Norfolk-Newport News, VA-NC Metro Area",
    "Williamsburg, VA": "Virginia Beach-Norfolk-Newport News, VA-NC Metro Area",
    "Alexandria, VA": "Washington-Arlington-Alexandria, DC-VA-MD-WV Metro Area",
    "Arlington, VA": "Washington-Arlington-Alexandria, DC-VA-MD-WV Metro Area",
    "Centreville, VA": "Washington-Arlington-Alexandria, DC-VA-MD-WV Metro Area",
    "Dale City, VA": "Washington-Arlington-Alexandria, DC-VA-MD-WV Metro Area",

    # Washington
    "Vancouver, WA": "Portland-Vancouver-Hillsboro, OR-WA Metro Area",
    "Seattle, WA": "Seattle-Tacoma-Bellevue, WA Metro Area",
    "Auburn, WA": "Seattle-Tacoma-Bellevue, WA Metro Area",
    "Bellevue, WA": "Seattle-Tacoma-Bellevue, WA Metro Area",
    "Federal Way, WA": "Seattle-Tacoma-Bellevue, WA Metro Area",
    "Kent, WA": "Seattle-Tacoma-Bellevue, WA Metro Area",
    "Kirkland, WA": "Seattle-Tacoma-Bellevue, WA Metro Area",
    "Lakewood, WA": "Seattle-Tacoma-Bellevue, WA Metro Area",
    "Redmond, WA": "Seattle-Tacoma-Bellevue, WA Metro Area",
    "Renton, WA": "Seattle-Tacoma-Bellevue, WA Metro Area",
    "Sammamish, WA": "Seattle-Tacoma-Bellevue, WA Metro Area",
    "Tacoma, WA": "Seattle-Tacoma-Bellevue, WA Metro Area",

    # Wisconsin
    "Milwaukee, WI": "Milwaukee-Waukesha, WI Metro Area",
    "Kenosha, WI": "Chicago-Naperville-Elgin, IL-IN-WI Metro Area",
    "Racine, WI": "Milwaukee-Waukesha, WI Metro Area",
    "Waukesha, WI": "Milwaukee-Waukesha, WI Metro Area",
    "Wauwatosa, WI": "Milwaukee-Waukesha, WI Metro Area",
    "West Allis, WI": "Milwaukee-Waukesha, WI Metro Area"
}

# Create a summary by MSA
msa_summary = {}
for city, msa in goshare_city_to_msa_mapping.items():
    if msa not in msa_summary:
        msa_summary[msa] = []
    msa_summary[msa].append(city)

# Print summary
print(f"Total GoShare cities: {len(goshare_city_to_msa_mapping)}")
print(f"Total unique MSAs covered: {len(msa_summary)}")

# Show MSAs with multiple cities (top 10)
print("\nTop MSAs with most GoShare cities:")
sorted_msas = sorted(msa_summary.items(), key=lambda x: len(x[1]), reverse=True)
for msa, cities in sorted_msas[:10]:
    print(f"{msa}: {len(cities)} cities")

Total GoShare cities: 711
Total unique MSAs covered: 149

Top MSAs with most GoShare cities:
New York-Newark-Jersey City, NY-NJ-PA Metro Area: 28 cities
Los Angeles-Long Beach-Anaheim, CA Metro Area: 19 cities
Riverside-San Bernardino-Ontario, CA Metro Area: 12 cities
Chicago-Naperville-Elgin, IL-IN-WI Metro Area: 12 cities
Phoenix-Mesa-Chandler, AZ Metro Area: 11 cities
Sacramento-Roseville-Folsom, CA Metro Area: 11 cities
San Diego-Chula Vista-Carlsbad, CA Metro Area: 11 cities
Washington-Arlington-Alexandria, DC-VA-MD-WV Metro Area: 11 cities
Philadelphia-Camden-Wilmington, PA-NJ-DE-MD Metro Area: 11 cities
Miami-Fort Lauderdale-Pompano Beach, FL Metro Area: 11 cities


## Location Data Collection - Lugg
 Extracted service locations from official website for Lugg,  standardizing city/state formats and removing duplicates.

In [8]:
lugg_cities = [
    # Arizona - 27 cities
    "Anthem, AZ",
    "Apache Junction, AZ",
    "Avondale, AZ",
    "Buckeye, AZ",
    "Casa Grande, AZ",
    "Chandler, AZ",
    "El Mirage, AZ",
    "Eloy, AZ",
    "Florence, AZ",
    "Fountain Hills, AZ",
    "Gilbert, AZ",
    "Glendale, AZ",
    "Goodyear, AZ",
    "Marana, AZ",
    "Maricopa, AZ",
    "Mesa, AZ",
    "New River, AZ",
    "Peoria, AZ",
    "Phoenix, AZ",
    "Queen Creek, AZ",
    "San Tan Valley, AZ",
    "Scottsdale, AZ",
    "Sun City, AZ",
    "Sun City West, AZ",
    "Sun Lakes, AZ",
    "Surprise, AZ",
    "Tempe, AZ",

    # California - 173 cities
    "Alameda, CA",
    "Aliso Viejo, CA",
    "Alpine, CA",
    "Alum Rock, CA",
    "Anaheim, CA",
    "Antelope, CA",
    "Antioch, CA",
    "Arden-Arcade, CA",
    "Aromas, CA",
    "Berkeley, CA",
    "Beverly Hills, CA",
    "Bostonia, CA",
    "Brea, CA",
    "Brentwood, CA",
    "Buena Park, CA",
    "Burbank, CA",
    "Cambrian Park, CA",
    "Cameron Park, CA",
    "Campbell, CA",
    "Carlsbad, CA",
    "Carmichael, CA",
    "Carson, CA",
    "Castro Valley, CA",
    "Chula Vista, CA",
    "Citrus Heights, CA",
    "Concord, CA",
    "Coronado, CA",
    "Costa Mesa, CA",
    "Cupertino, CA",
    "Cypress, CA",
    "Daly City, CA",
    "Danville, CA",
    "Davis, CA",
    "Downey, CA",
    "Dublin, CA",
    "East Foothills, CA",
    "East Los Angeles, CA",
    "El Cajon, CA",
    "El Dorado Hills, CA",
    "El Monte, CA",
    "Elk Grove, CA",
    "Encinitas, CA",
    "Escondido, CA",
    "Fair Oaks, CA",
    "Fallbrook, CA",
    "Florin, CA",
    "Folsom, CA",
    "Foothill Farms, CA",
    "Fountain Valley, CA",
    "Fremont, CA",
    "Fullerton, CA",
    "Galt, CA",
    "Garden Grove, CA",
    "Gilroy, CA",
    "Glendale, CA",
    "Granite Bay, CA",
    "Hayward, CA",
    "Hermosa Beach, CA",
    "Hollister, CA",
    "Huntington Beach, CA",
    "Imperial Beach, CA",
    "Inglewood, CA",
    "Irvine, CA",
    "La Habra, CA",
    "La Mesa, CA",
    "La Presa, CA",
    "Laguna Niguel, CA",
    "Lake Forest, CA",
    "Lakeside, CA",
    "Lancaster, CA",
    "Lemon Grove, CA",
    "Lexington Hills, CA",
    "Lincoln, CA",
    "Livermore, CA",
    "Long Beach, CA",
    "Los Altos, CA",
    "Los Altos Hills, CA",
    "Los Angeles, CA",
    "Los Gatos, CA",
    "Loyola, CA",
    "Milpitas, CA",
    "Mission Viejo, CA",
    "Monte Sereno, CA",
    "Morgan Hill, CA",
    "Mountain View, CA",
    "National City, CA",
    "Newark, CA",
    "Newport Beach, CA",
    "North Highlands, CA",
    "Norwalk, CA",
    "Novato, CA",
    "Oakland, CA",
    "Oceanside, CA",
    "Orange, CA",
    "Orange County, CA",
    "Orangevale, CA",
    "Palo Alto, CA",
    "Pasadena, CA",
    "Pittsburg, CA",
    "Placentia, CA",
    "Pleasanton, CA",
    "Pomona, CA",
    "Poway, CA",
    "Ramona, CA",
    "Rancho Cordova, CA",
    "Rancho San Diego, CA",
    "Rancho Santa Margarita, CA",
    "Redwood City, CA",
    "Richmond, CA",
    "Ridgemark, CA",
    "Rocklin, CA",
    "Rosemont, CA",
    "Roseville, CA",
    "Sacramento, CA",
    "San Clemente, CA",
    "San Diego, CA",
    "San Francisco, CA",
    "San Jose, CA",
    "San Leandro, CA",
    "San Marcos, CA",
    "San Martin, CA",
    "San Mateo, CA",
    "San Rafael, CA",
    "San Ramon, CA",
    "Santa Ana, CA",
    "Santa Clara, CA",
    "Santa Clarita, CA",
    "Santa Cruz, CA",
    "Santa Monica, CA",
    "Santa Rosa, CA",
    "Santee, CA",
    "Saratoga, CA",
    "Solana Beach, CA",
    "South Lake Tahoe, CA",
    "Spring Valley, CA",
    "Stanford, CA",
    "Stanton, CA",
    "Sunnyvale, CA",
    "Torrance, CA",
    "Tustin, CA",
    "Union City, CA",
    "Vineyard, CA",
    "Vista, CA",
    "Walnut Creek, CA",
    "West Covina, CA",
    "West Sacramento, CA",
    "Westminster, CA",
    "Winter Gardens, CA",
    "Woodland, CA",
    "Yorba Linda, CA",

    # Colorado - 25 cities
    "Arvada, CO",
    "Aurora, CO",
    "Boulder, CO",
    "Brighton, CO",
    "Broomfield, CO",
    "Castle Rock, CO",
    "Centennial, CO",
    "Columbine, CO",
    "Commerce City, CO",
    "Dakota Ridge, CO",
    "Denver, CO",
    "Englewood, CO",
    "Fort Collins, CO",
    "Golden, CO",
    "Greenwood Village, CO",
    "Highlands Ranch, CO",
    "Ken Caryl, CO",
    "Lakewood, CO",
    "Littleton, CO",
    "Northglenn, CO",
    "Parker, CO",
    "Sherrelwood, CO",
    "Thornton, CO",
    "Welby, CO",
    "Westminster, CO",
    "Wheat Ridge, CO",

    # Connecticut - 51 cities
    "Ansonia, CT",
    "Bolton, CT",
    "Branford, CT",
    "Bridgewater, CT",
    "Bristol, CT",
    "Broad Brook, CT",
    "Cheshire, CT",
    "Clinton, CT",
    "Collinsville, CT",
    "Derby, CT",
    "Dover Plains, CT",
    "East Hartford, CT",
    "East Haven, CT",
    "Glastonbury, CT",
    "Guilford, CT",
    "Hamden, CT",
    "Hartford, CT",
    "Hazardville, CT",
    "Heritage, CT",
    "Kensington, CT",
    "Madison, CT",
    "Manchester, CT",
    "Meriden, CT",
    "Middletown, CT",
    "Milford, CT",
    "Milton, CT",
    "Naugatuck, CT",
    "New Britain, CT",
    "New Haven, CT",
    "New Milford, CT",
    "Newington, CT",
    "Newtown, CT",
    "North Haven, CT",
    "Orange, CT",
    "Portland, CT",
    "Riverside, CT",
    "Rockville, CT",
    "Shelton, CT",
    "Sherwood Manor, CT",
    "Shirley, CT",
    "Simsbury, CT",
    "Southwood Acres, CT",
    "Stafford Springs, CT",
    "Stamford, CT",
    "Storrs, CT",
    "Thompsonville, CT",
    "Wallingford, CT",
    "Waterbury, CT",
    "West Hartford, CT",
    "West Haven, CT",
    "Wethersfield, CT",
    "Windsor Locks, CT",
    "Woodmon, CT",

    # Delaware - 14 cities
    "Bear, DE",
    "Bellefonte, DE",
    "Christiana, DE",
    "Claymont, DE",
    "Delaware City, DE",
    "Elsmere, DE",
    "Glasgow, DE",
    "Hockessin, DE",
    "Middletown, DE",
    "Minquadale, DE",
    "Montchanin, DE",
    "New Castle, DE",
    "Willow Grove, DE",
    "Wilmington, DE",

    # Florida - 26 cities
    "Boca Raton, FL",
    "Boynton Beach, FL",
    "Coconut Creek, FL",
    "Coral Springs, FL",
    "Davie, FL",
    "Deerfield Beach, FL",
    "Delray Beach, FL",
    "Fort Lauderdale, FL",
    "Hialeah, FL",
    "Hollywood, FL",
    "Homestead, FL",
    "Kendall, FL",
    "Lauderhill, FL",
    "Miami, FL",
    "Miami Beach, FL",
    "Miami Gardens, FL",
    "Miramar, FL",
    "North Miami, FL",
    "Pembroke Pines, FL",
    "Plantation, FL",
    "Pompano Beach, FL",
    "Sunrise, FL",
    "Tamarac, FL",
    "Wellington, FL",
    "West Palm Beach, FL",
    "Weston, FL",

    # Georgia - 26 cities
    "Alpharetta, GA",
    "Atlanta, GA",
    "Brookhaven, GA",
    "Canton, GA",
    "Carrollton, GA",
    "Chamblee, GA",
    "Douglasville, GA",
    "Duluth, GA",
    "Dunwoody, GA",
    "East Point, GA",
    "Johns Creek, GA",
    "Kennesaw, GA",
    "Lawrenceville, GA",
    "Mableton, GA",
    "Marietta, GA",
    "Milton, GA",
    "Newnan, GA",
    "Peachtree City, GA",
    "Peachtree Corners, GA",
    "Redan, GA",
    "Roswell, GA",
    "Sandy Springs, GA",
    "Smyrna, GA",
    "Stockbridge, GA",
    "Tucker, GA",
    "Woodstock, GA",

    # Illinois - 24 cities
    "Arlington Heights, IL",
    "Aurora, IL",
    "Berwyn, IL",
    "Bolingbrook, IL",
    "Chicago, IL",
    "Cicero, IL",
    "Des Plaines, IL",
    "Downers Grove, IL",
    "Elgin, IL",
    "Evanston, IL",
    "Glenview, IL",
    "Joliet, IL",
    "Mount Prospect, IL",
    "Naperville, IL",
    "Oak Lawn, IL",
    "Oak Park, IL",
    "Orland Park, IL",
    "Palatine, IL",
    "Schaumburg, IL",
    "Skokie, IL",
    "Tinley Park, IL",
    "Waukegan, IL",
    "Wheaton, IL",
    "Yorkville, IL",

    # Indiana - 2 cities
    "Gary, IN",
    "Hammond, IN",

    # Maryland - 49 cities
    "Aberdeen, MD",
    "Annapolis, MD",
    "Annapolis Neck, MD",
    "Arnold, MD",
    "Baltimore, MD",
    "Bel Air, MD",
    "Brooklyn Park, MD",
    "Cape St. Claire, MD",
    "Chase, MD",
    "Chestertown, MD",
    "Crofton, MD",
    "Deale, MD",
    "Dundalk, MD",
    "Edgewater, MD",
    "Edgewood, MD",
    "Essex, MD",
    "Ferndale, MD",
    "Finksburg, MD",
    "Fort Meade, MD",
    "Fredericksburg, MD",
    "Freeland, MD",
    "Glen Burnie, MD",
    "Glenville, MD",
    "Gunpowder, MD",
    "Halethorpe, MD",
    "Herald Harbor, MD",
    "Hunt Valley, MD",
    "Jarrettsville, MD",
    "Jessup, MD",
    "Kingsville, MD",
    "Lake Shore, MD",
    "Linthicum Heights, MD",
    "Manchester, MD",
    "Maryland City, MD",
    "Mayo, MD",
    "Mount Rainier, MD",
    "Naval Academy, MD",
    "Odenton, MD",
    "Parkville, MD",
    "Parole, MD",
    "Pasadena, MD",
    "Randallstown, MD",
    "Riva, MD",
    "Riviera Beach, MD",
    "Severn, MD",
    "Severna Park, MD",
    "Shady Side, MD",
    "Stevenson, MD",
    "Towson, MD",
    "White Marsh, MD",
    "Woodstock, MD",

    # Massachusetts - 75 cities
    "Acushnet, MA",
    "Andover, MA",
    "Attleboro, MA",
    "Barnstable, MA",
    "Berkley, MA",
    "Beverly, MA",
    "Boston, MA",
    "Boxford, MA",
    "Braintree, MA",
    "Brockton, MA",
    "Brookline, MA",
    "Burlington, MA",
    "Cambridge, MA",
    "Chatham, MA",
    "Chelsea, MA",
    "Danvers, MA",
    "Dartmouth, MA",
    "Dedham, MA",
    "Dighton, MA",
    "Everett, MA",
    "Fall River, MA",
    "Fitchburg, MA",
    "Foxborough, MA",
    "Framingham, MA",
    "Franklin, MA",
    "Haverhill, MA",
    "Lawrence, MA",
    "Lowell, MA",
    "Lynn, MA",
    "Lynnfield, MA",
    "Malden, MA",
    "Mansfield, MA",
    "Marlborough, MA",
    "Medford, MA",
    "Melrose, MA",
    "Methuen, MA",
    "Milford, MA",
    "Natick, MA",
    "Needham, MA",
    "New Bedford, MA",
    "Newburyport, MA",
    "Newton, MA",
    "Norfolk, MA",
    "North Attleborough, MA",
    "Northampton, MA",
    "Norton, MA",
    "Peabody, MA",
    "Plainville, MA",
    "Quincy, MA",
    "Raynham, MA",
    "Reading, MA",
    "Rehoboth, MA",
    "Revere, MA",
    "Seekonk, MA",
    "Somerset, MA",
    "Somerville, MA",
    "Springfield, MA",
    "Stoughton, MA",
    "Swansea, MA",
    "Taunton, MA",
    "Tewksbury, MA",
    "Wakefield, MA",
    "Walpole, MA",
    "Waltham, MA",
    "Watertown, MA",
    "Wayland, MA",
    "Wellesley, MA",
    "Westport, MA",
    "Weymouth, MA",
    "Wilmington, MA",
    "Winchester, MA",
    "Winthrop, MA",
    "Woburn, MA",
    "Worcester, MA",
    "Wrentham, MA",

    # New Hampshire - 19 cities
    "Atkinson, NH",
    "Derry, NH",
    "East Derry, NH",
    "East Hampstead, NH",
    "East Kingston, NH",
    "Fremont, NH",
    "Hampstead, NH",
    "Hampton, NH",
    "Hampton Falls, NH",
    "Hollis, NH",
    "Litchfield, NH",
    "Londonderry, NH",
    "Merrimack, NH",
    "Nashua, NH",
    "North Hampton, NH",
    "Pelham, NH",
    "Plaistow, NH",
    "Sandown, NH",
    "Seabrook, NH",

    # New Jersey - 52 cities
    "Bayonne, NJ",
    "Brentwood, NJ",
    "Camden, NJ",
    "Cherry Hill, NJ",
    "Clifton, NJ",
    "Collingswood, NJ",
    "Deptford, NJ",
    "East Orange, NJ",
    "Elizabeth, NJ",
    "Glassboro, NJ",
    "Groveville, NJ",
    "Hackensack, NJ",
    "Hamilton Square, NJ",
    "Hamilton Township, NJ",
    "Hempstead, NJ",
    "Hightstown, NJ",
    "Hoboken, NJ",
    "Hopewell, NJ",
    "Jersey City, NJ",
    "Lakewood, NJ",
    "Lawrenceville, NJ",
    "Lindenwold, NJ",
    "Mercerville, NJ",
    "Merchantville, NJ",
    "Moorestown, NJ",
    "Mount Vernon, NJ",
    "New Brunswick, NJ",
    "New Rochelle, NJ",
    "Newark, NJ",
    "Palmyra, NJ",
    "Paramus, NJ",
    "Passaic, NJ",
    "Paterson, NJ",
    "Pennington, NJ",
    "Pennsauken Township, NJ",
    "Perth Amboy, NJ",
    "Plainfield, NJ",
    "Princeton, NJ",
    "Robbinsville, NJ",
    "Sayreville, NJ",
    "Somerdale, NJ",
    "Toms River, NJ",
    "Trenton, NJ",
    "Twin Rivers, NJ",
    "Union, NJ",
    "Union City, NJ",
    "West New York, NJ",
    "White Horse, NJ",
    "White Plains, NJ",
    "Yardville, NJ",

    # New York - 25 cities
    "Astoria, NY",
    "Bedford Stuyvesant, NY",
    "Bronx, NY",
    "Brooklyn, NY",
    "Bushwick, NY",
    "Crown Heights, NY",
    "East Flatbush, NY",
    "Flatbush, NY",
    "Gramercy, NY",
    "Harlem, NY",
    "Jamaica, NY",
    "Long Island, NY",
    "Lower East Side, NY",
    "Manhattan, NY",
    "Midwood, NY",
    "New York City, NY",
    "Park Slope, NY",
    "Queens, NY",
    "Ridgewood, NY",
    "SoHo, NY",
    "Staten Island, NY",
    "Upper East Side, NY",
    "Upper West Side, NY",
    "Williamsbridge, NY",
    "Williamsburg, NY",
    "Yonkers, NY",

    # Oregon - 33 cities
    "Aloha, OR",
    "Aurora, OR",
    "Barlow, OR",
    "Battin, OR",
    "Beaverton, OR",
    "Bethany, OR",
    "Canby, OR",
    "Chehalem, OR",
    "Clackamas, OR",
    "Cornelius, OR",
    "Damascus, OR",
    "Dunthorpe, OR",
    "Fairview, OR",
    "Forest Grove, OR",
    "Gaston, OR",
    "Gladstone, OR",
    "Gresham, OR",
    "Happy Valley, OR",
    "Helvetia, OR",
    "Hillsboro, OR",
    "Lake Oswego, OR",
    "McMinnville, OR",
    "Milwaukie, OR",
    "Newberg, OR",
    "North Plains, OR",
    "Orchards, OR",
    "Oregon City, OR",
    "Portland, OR",
    "Sherwood, OR",
    "Tigard, OR",
    "Troutdale, OR",
    "Tualatin, OR",
    "West Linn, OR",
    "Wilsonville, OR",
    "Wood Village, OR",

    # Pennsylvania - 31 cities
    "Ambler, PA",
    "Blue Bell, PA",
    "Bridgeport, PA",
    "Brookside, PA",
    "Broomall, PA",
    "Chester, PA",
    "Conshohocken, PA",
    "Doylestown, PA",
    "Drexel Hill, PA",
    "Exton, PA",
    "Fishtown, PA",
    "Gladwyne, PA",
    "Havertown, PA",
    "Horsham, PA",
    "King of Prussia, PA",
    "Lansdale, PA",
    "Levittown, PA",
    "Malvern, PA",
    "New Hope, PA",
    "Newtown, PA",
    "Norristown, PA",
    "Paoli, PA",
    "Philadelphia, PA",
    "Phoenixville, PA",
    "Plymouth Meeting, PA",
    "Pottstown, PA",
    "Southampton, PA",
    "Springfield, PA",
    "Swarthmore, PA",
    "Villanova, PA",
    "Wayne, PA",
    "West Chester, PA",
    "Willow Grove, PA",

    # Rhode Island - 15 cities
    "Barrington, RI",
    "Central Falls, RI",
    "Cranston, RI",
    "Cumberland, RI",
    "East Greenwich, RI",
    "Lincoln, RI",
    "Narragansett, RI",
    "Newport, RI",
    "Pawtucket, RI",
    "Portsmouth, RI",
    "Providence, RI",
    "Smithfield, RI",
    "Tiverton, RI",
    "Warwick, RI",
    "Woonsocket, RI",

    # Texas - 107 cities
    "Adkins, TX",
    "Allen, TX",
    "Alvin, TX",
    "Angleto, TX",
    "Arlington, TX",
    "Atascocita, TX",
    "Austin, TX",
    "Bastrop, TX",
    "Baytown, TX",
    "Bedford, TX",
    "Bee Cave, TX",
    "Bellaire, TX",
    "Brushy Creek, TX",
    "Buda, TX",
    "Camp Swift, TX",
    "Carrollton, TX",
    "Cedar Hill, TX",
    "Cedar Park, TX",
    "Channelview, TX",
    "Cinco Ranch, TX",
    "Cloverleaf, TX",
    "Conroe, TX",
    "Cypress, TX",
    "Dallas, TX",
    "DeSoto, TX",
    "Deer Park, TX",
    "Denton, TX",
    "Dickinson, TX",
    "Elgin, TX",
    "Euless, TX",
    "Flower Mound, TX",
    "Fort Worth, TX",
    "Fresno, TX",
    "Friendswood, TX",
    "Frisco, TX",
    "Fulshear, TX",
    "Galveston, TX",
    "Garland, TX",
    "Georgetown, TX",
    "Grand Prairie, TX",
    "Grapevine, TX",
    "Hornsby Bend, TX",
    "Houston, TX",
    "Humble, TX",
    "Hutto, TX",
    "Irving, TX",
    "Jersey Village, TX",
    "Katy, TX",
    "Kindwood, TX",
    "Kyle, TX",
    "La Porte, TX",
    "Lago Vista, TX",
    "Lake Jackson, TX",
    "Lakeway, TX",
    "League City, TX",
    "Leander, TX",
    "Lewisville, TX",
    "Lockhart, TX",
    "Lost Creek, TX",
    "Luling, TX",
    "Manor, TX",
    "Mansfield, TX",
    "McKinney, TX",
    "Mesquite, TX",
    "Mission Bend, TX",
    "Missouri City, TX",
    "Montrose, TX",
    "North Austin, TX",
    "North Richland Hills, TX",
    "Pasadena, TX",
    "Pearland, TX",
    "Pflugerville, TX",
    "Plano, TX",
    "Richardson, TX",
    "Richmond, TX",
    "Rosenberg, TX",
    "Round Rock, TX",
    "Rowlett, TX",
    "San Antonio, TX",
    "San Marcos, TX",
    "Shady Hollow, TX",
    "Smithville, TX",
    "Spring, TX",
    "Sugar Land, TX",
    "Taylor, TX",
    "Texas City, TX",
    "The Colony, TX",
    "The Woodlands, TX",
    "Tomball, TX",
    "Wells Branch, TX",
    "Wylie, TX",

    # Utah - 54 cities
    "Alpine, UT",
    "American Fork, UT",
    "Bingham Canyon, UT",
    "Bountiful, UT",
    "Cedar Valley, UT",
    "Centerville, UT",
    "Clearfield, UT",
    "Coalville, UT",
    "Croydon, UT",
    "Draper, UT",
    "Eagle Mountain, UT",
    "Echo, UT",
    "Eden, UT",
    "Farmington, UT",
    "Grantsville, UT",
    "Heber City, UT",
    "Henefer, UT",
    "Herriman, UT",
    "Hill Afb, UT",
    "Hooper, UT",
    "Huntsville, UT",
    "Kamas, UT",
    "Kaysville, UT",
    "Layton, UT",
    "Lehi, UT",
    "Lindon, UT",
    "Magna, UT",
    "Mapleton, UT",
    "Midvale, UT",
    "Midway, UT",
    "Morgan, UT",
    "North Salt Lake, UT",
    "Oakley, UT",
    "Ogden, UT",
    "Orem, UT",
    "Park City, UT",
    "Payson, UT",
    "Peoa, UT",
    "Pleasant Grove, UT",
    "Provo, UT",
    "Riverton, UT",
    "Roy, UT",
    "Rush Valley, UT",
    "Salt Lake City, UT",
    "Sandy, UT",
    "Saratoga Springs, UT",
    "South Jordan, UT",
    "Spanish Fork, UT",
    "Springville, UT",
    "Stockton, UT",
    "Syracuse, UT",
    "Tooele, UT",
    "Vineyard, UT",
    "Wallsburg, UT",
    "West Jordan, UT",
    "West Valley City, UT",
    "Willard, UT",
    "Woods Cross, UT",

    # Virginia - 23 cities
    "Alexandria, VA",
    "Annandale, VA",
    "Arlington, VA",
    "Burke, VA",
    "Centreville, VA",
    "Chantilly, VA",
    "Dunn Loring, VA",
    "Fairfax, VA",
    "Falls Church, VA",
    "Great Falls, VA",
    "Greenway, VA",
    "Herndon, VA",
    "Indian Head, VA",
    "Lorton, VA",
    "Newington, VA",
    "Oakton, VA",
    "Occoquan, VA",
    "Reston, VA",
    "Springfield, VA",
    "Sterling, VA",
    "Vienna, VA",
    "West Mclean, VA",
    "Woodbridge, VA",

    # Washington - 35 cities
    "Auburn, WA",
    "Barberton, WA",
    "Battle Ground, WA",
    "Bellevue, WA",
    "Bothell, WA",
    "Brush Prairie, WA",
    "Burien, WA",
    "Camas, WA",
    "Des Moines, WA",
    "Edmonds, WA",
    "Everett, WA",
    "Federal Way, WA",
    "Five Corners, WA",
    "Hazel Dell, WA",
    "Issaquah, WA",
    "Kent, WA",
    "Kirkland, WA",
    "Lake Stevens, WA",
    "Lakewood, WA",
    "Lynnwood, WA",
    "Marysville, WA",
    "Minnehaha, WA",
    "Olympia, WA",
    "Orchards, WA",
    "Parkland, WA",
    "Puyallup, WA",
    "Redmond, WA",
    "Renton, WA",
    "Ridgefield, WA",
    "Salmon Creek, WA",
    "Sammamish, WA",
    "Seattle, WA",
    "Shoreline, WA",
    "South Hill, WA",
    "Spanaway, WA",
    "Tacoma, WA",
    "University Place, WA",
    "Vancouver, WA",
    "Walnut Grove, WA",
    "West Linn, WA",

    # Wisconsin - 2 cities
    "Kenosha, WI",
    "Pleasant Prairie, WI",

    # District of Columbia - 25 neighborhoods
    "Adams Morgan, DC",
    "American University Park, DC",
    "Brightwood, DC",
    "Capitol Hill, DC",
    "Chevy Chase, DC",
    "Cleveland Park, DC",
    "Columbia Heights, DC",
    "Congress Heights, DC",
    "Dupont Circle, DC",
    "Foggy Bottom, DC",
    "Forest Hills, DC",
    "Georgetown, DC",
    "Glover Park, DC",
    "Kingman Park, DC",
    "Logan Circle, DC",
    "Manor Park, DC",
    "Mount Pleasant, DC",
    "Near Northeast, DC",
    "Petworth, DC",
    "Riggs Park, DC",
    "Shaw, DC",
    "Shipley Terrace, DC",
    "Sixteenth St Heights, DC",
    "Southwest Waterfront, DC",
    "Washington, D.C., DC",
    "Washington Highlands, DC"
]

## MSA Mapping Process - Lugg
Mapped each location to OMB-defined Metropolitan Statistical Areas using 2020 delineations and Census population data for market sizing analysis.

In [9]:
# Lugg Cities mapped to Official OMB Metropolitan Statistical Areas (MSAs)
# Corrected to match official OMB naming conventions from Census data
# MAJOR CORRECTIONS: Added "Metro Area" suffix to ALL names + Fixed incorrect MSA names

lugg_city_to_msa_mapping = {
    # Arizona - 27 cities (Phoenix MSA)
    "Anthem, AZ": "Phoenix-Mesa-Chandler, AZ Metro Area",
    "Apache Junction, AZ": "Phoenix-Mesa-Chandler, AZ Metro Area",
    "Avondale, AZ": "Phoenix-Mesa-Chandler, AZ Metro Area",
    "Buckeye, AZ": "Phoenix-Mesa-Chandler, AZ Metro Area",
    "Casa Grande, AZ": "Phoenix-Mesa-Chandler, AZ Metro Area",
    "Chandler, AZ": "Phoenix-Mesa-Chandler, AZ Metro Area",
    "El Mirage, AZ": "Phoenix-Mesa-Chandler, AZ Metro Area",
    "Eloy, AZ": "Phoenix-Mesa-Chandler, AZ Metro Area",
    "Florence, AZ": "Phoenix-Mesa-Chandler, AZ Metro Area",
    "Fountain Hills, AZ": "Phoenix-Mesa-Chandler, AZ Metro Area",
    "Gilbert, AZ": "Phoenix-Mesa-Chandler, AZ Metro Area",
    "Glendale, AZ": "Phoenix-Mesa-Chandler, AZ Metro Area",
    "Goodyear, AZ": "Phoenix-Mesa-Chandler, AZ Metro Area",
    "Marana, AZ": "Tucson, AZ Metro Area",
    "Maricopa, AZ": "Phoenix-Mesa-Chandler, AZ Metro Area",
    "Mesa, AZ": "Phoenix-Mesa-Chandler, AZ Metro Area",
    "New River, AZ": "Phoenix-Mesa-Chandler, AZ Metro Area",
    "Peoria, AZ": "Phoenix-Mesa-Chandler, AZ Metro Area",
    "Phoenix, AZ": "Phoenix-Mesa-Chandler, AZ Metro Area",
    "Queen Creek, AZ": "Phoenix-Mesa-Chandler, AZ Metro Area",
    "San Tan Valley, AZ": "Phoenix-Mesa-Chandler, AZ Metro Area",
    "Scottsdale, AZ": "Phoenix-Mesa-Chandler, AZ Metro Area",
    "Sun City, AZ": "Phoenix-Mesa-Chandler, AZ Metro Area",
    "Sun City West, AZ": "Phoenix-Mesa-Chandler, AZ Metro Area",
    "Sun Lakes, AZ": "Phoenix-Mesa-Chandler, AZ Metro Area",
    "Surprise, AZ": "Phoenix-Mesa-Chandler, AZ Metro Area",
    "Tempe, AZ": "Phoenix-Mesa-Chandler, AZ Metro Area",

    # California - 173 cities
    "Alameda, CA": "San Francisco-Oakland-Berkeley, CA Metro Area",
    "Aliso Viejo, CA": "Los Angeles-Long Beach-Anaheim, CA Metro Area",
    "Alpine, CA": "San Diego-Chula Vista-Carlsbad, CA Metro Area",
    "Alum Rock, CA": "San Jose-Sunnyvale-Santa Clara, CA Metro Area",
    "Anaheim, CA": "Los Angeles-Long Beach-Anaheim, CA Metro Area",
    "Antelope, CA": "Sacramento-Roseville-Folsom, CA Metro Area",
    "Antioch, CA": "San Francisco-Oakland-Berkeley, CA Metro Area",
    "Arden-Arcade, CA": "Sacramento-Roseville-Folsom, CA Metro Area",
    "Aromas, CA": "Salinas, CA Metro Area",
    "Berkeley, CA": "San Francisco-Oakland-Berkeley, CA Metro Area",
    "Beverly Hills, CA": "Los Angeles-Long Beach-Anaheim, CA Metro Area",
    "Bostonia, CA": "San Diego-Chula Vista-Carlsbad, CA Metro Area",
    "Brea, CA": "Los Angeles-Long Beach-Anaheim, CA Metro Area",
    "Brentwood, CA": "San Francisco-Oakland-Berkeley, CA Metro Area",
    "Buena Park, CA": "Los Angeles-Long Beach-Anaheim, CA Metro Area",
    "Burbank, CA": "Los Angeles-Long Beach-Anaheim, CA Metro Area",
    "Cambrian Park, CA": "San Jose-Sunnyvale-Santa Clara, CA Metro Area",
    "Cameron Park, CA": "Sacramento-Roseville-Folsom, CA Metro Area",
    "Campbell, CA": "San Jose-Sunnyvale-Santa Clara, CA Metro Area",
    "Carlsbad, CA": "San Diego-Chula Vista-Carlsbad, CA Metro Area",
    "Carmichael, CA": "Sacramento-Roseville-Folsom, CA Metro Area",
    "Carson, CA": "Los Angeles-Long Beach-Anaheim, CA Metro Area",
    "Castro Valley, CA": "San Francisco-Oakland-Berkeley, CA Metro Area",
    "Chula Vista, CA": "San Diego-Chula Vista-Carlsbad, CA Metro Area",
    "Citrus Heights, CA": "Sacramento-Roseville-Folsom, CA Metro Area",
    "Concord, CA": "San Francisco-Oakland-Berkeley, CA Metro Area",
    "Coronado, CA": "San Diego-Chula Vista-Carlsbad, CA Metro Area",
    "Costa Mesa, CA": "Los Angeles-Long Beach-Anaheim, CA Metro Area",
    "Cupertino, CA": "San Jose-Sunnyvale-Santa Clara, CA Metro Area",
    "Cypress, CA": "Los Angeles-Long Beach-Anaheim, CA Metro Area",
    "Daly City, CA": "San Francisco-Oakland-Berkeley, CA Metro Area",
    "Danville, CA": "San Francisco-Oakland-Berkeley, CA Metro Area",
    "Davis, CA": "Sacramento-Roseville-Folsom, CA Metro Area",
    "Downey, CA": "Los Angeles-Long Beach-Anaheim, CA Metro Area",
    "Dublin, CA": "San Francisco-Oakland-Berkeley, CA Metro Area",
    "East Foothills, CA": "San Jose-Sunnyvale-Santa Clara, CA Metro Area",
    "East Los Angeles, CA": "Los Angeles-Long Beach-Anaheim, CA Metro Area",
    "El Cajon, CA": "San Diego-Chula Vista-Carlsbad, CA Metro Area",
    "El Dorado Hills, CA": "Sacramento-Roseville-Folsom, CA Metro Area",
    "El Monte, CA": "Los Angeles-Long Beach-Anaheim, CA Metro Area",
    "Elk Grove, CA": "Sacramento-Roseville-Folsom, CA Metro Area",
    "Encinitas, CA": "San Diego-Chula Vista-Carlsbad, CA Metro Area",
    "Escondido, CA": "San Diego-Chula Vista-Carlsbad, CA Metro Area",
    "Fair Oaks, CA": "Sacramento-Roseville-Folsom, CA Metro Area",
    "Fallbrook, CA": "San Diego-Chula Vista-Carlsbad, CA Metro Area",
    "Florin, CA": "Sacramento-Roseville-Folsom, CA Metro Area",
    "Folsom, CA": "Sacramento-Roseville-Folsom, CA Metro Area",
    "Foothill Farms, CA": "Sacramento-Roseville-Folsom, CA Metro Area",
    "Fountain Valley, CA": "Los Angeles-Long Beach-Anaheim, CA Metro Area",
    "Fremont, CA": "San Francisco-Oakland-Berkeley, CA Metro Area",
    "Fullerton, CA": "Los Angeles-Long Beach-Anaheim, CA Metro Area",
    "Galt, CA": "Sacramento-Roseville-Folsom, CA Metro Area",
    "Garden Grove, CA": "Los Angeles-Long Beach-Anaheim, CA Metro Area",
    "Gilroy, CA": "San Jose-Sunnyvale-Santa Clara, CA Metro Area",
    "Glendale, CA": "Los Angeles-Long Beach-Anaheim, CA Metro Area",
    "Granite Bay, CA": "Sacramento-Roseville-Folsom, CA Metro Area",
    "Hayward, CA": "San Francisco-Oakland-Berkeley, CA Metro Area",
    "Hermosa Beach, CA": "Los Angeles-Long Beach-Anaheim, CA Metro Area",
    "Hollister, CA": "San Jose-Sunnyvale-Santa Clara, CA Metro Area",
    "Huntington Beach, CA": "Los Angeles-Long Beach-Anaheim, CA Metro Area",
    "Imperial Beach, CA": "San Diego-Chula Vista-Carlsbad, CA Metro Area",
    "Inglewood, CA": "Los Angeles-Long Beach-Anaheim, CA Metro Area",
    "Irvine, CA": "Los Angeles-Long Beach-Anaheim, CA Metro Area",
    "La Habra, CA": "Los Angeles-Long Beach-Anaheim, CA Metro Area",
    "La Mesa, CA": "San Diego-Chula Vista-Carlsbad, CA Metro Area",
    "La Presa, CA": "San Diego-Chula Vista-Carlsbad, CA Metro Area",
    "Laguna Niguel, CA": "Los Angeles-Long Beach-Anaheim, CA Metro Area",
    "Lake Forest, CA": "Los Angeles-Long Beach-Anaheim, CA Metro Area",
    "Lakeside, CA": "San Diego-Chula Vista-Carlsbad, CA Metro Area",
    "Lancaster, CA": "Los Angeles-Long Beach-Anaheim, CA Metro Area",
    "Lemon Grove, CA": "San Diego-Chula Vista-Carlsbad, CA Metro Area",
    "Lexington Hills, CA": "San Jose-Sunnyvale-Santa Clara, CA Metro Area",
    "Lincoln, CA": "Sacramento-Roseville-Folsom, CA Metro Area",
    "Livermore, CA": "San Francisco-Oakland-Berkeley, CA Metro Area",
    "Long Beach, CA": "Los Angeles-Long Beach-Anaheim, CA Metro Area",
    "Los Altos, CA": "San Jose-Sunnyvale-Santa Clara, CA Metro Area",
    "Los Altos Hills, CA": "San Jose-Sunnyvale-Santa Clara, CA Metro Area",
    "Los Angeles, CA": "Los Angeles-Long Beach-Anaheim, CA Metro Area",
    "Los Gatos, CA": "San Jose-Sunnyvale-Santa Clara, CA Metro Area",
    "Loyola, CA": "Los Angeles-Long Beach-Anaheim, CA Metro Area",
    "Milpitas, CA": "San Jose-Sunnyvale-Santa Clara, CA Metro Area",
    "Mission Viejo, CA": "Los Angeles-Long Beach-Anaheim, CA Metro Area",
    "Monte Sereno, CA": "San Jose-Sunnyvale-Santa Clara, CA Metro Area",
    "Morgan Hill, CA": "San Jose-Sunnyvale-Santa Clara, CA Metro Area",
    "Mountain View, CA": "San Jose-Sunnyvale-Santa Clara, CA Metro Area",
    "National City, CA": "San Diego-Chula Vista-Carlsbad, CA Metro Area",
    "Newark, CA": "San Francisco-Oakland-Berkeley, CA Metro Area",
    "Newport Beach, CA": "Los Angeles-Long Beach-Anaheim, CA Metro Area",
    "North Highlands, CA": "Sacramento-Roseville-Folsom, CA Metro Area",
    "Norwalk, CA": "Los Angeles-Long Beach-Anaheim, CA Metro Area",
    "Novato, CA": "San Francisco-Oakland-Berkeley, CA Metro Area",
    "Oakland, CA": "San Francisco-Oakland-Berkeley, CA Metro Area",
    "Oceanside, CA": "San Diego-Chula Vista-Carlsbad, CA Metro Area",
    "Orange, CA": "Los Angeles-Long Beach-Anaheim, CA Metro Area",
    "Orange County, CA": "Los Angeles-Long Beach-Anaheim, CA Metro Area",
    "Orangevale, CA": "Sacramento-Roseville-Folsom, CA Metro Area",
    "Palo Alto, CA": "San Jose-Sunnyvale-Santa Clara, CA Metro Area",
    "Pasadena, CA": "Los Angeles-Long Beach-Anaheim, CA Metro Area",
    "Pittsburg, CA": "San Francisco-Oakland-Berkeley, CA Metro Area",
    "Placentia, CA": "Los Angeles-Long Beach-Anaheim, CA Metro Area",
    "Pleasanton, CA": "San Francisco-Oakland-Berkeley, CA Metro Area",
    "Pomona, CA": "Los Angeles-Long Beach-Anaheim, CA Metro Area",
    "Poway, CA": "San Diego-Chula Vista-Carlsbad, CA Metro Area",
    "Ramona, CA": "San Diego-Chula Vista-Carlsbad, CA Metro Area",
    "Rancho Cordova, CA": "Sacramento-Roseville-Folsom, CA Metro Area",
    "Rancho San Diego, CA": "San Diego-Chula Vista-Carlsbad, CA Metro Area",
    "Rancho Santa Margarita, CA": "Los Angeles-Long Beach-Anaheim, CA Metro Area",
    "Redwood City, CA": "San Francisco-Oakland-Berkeley, CA Metro Area",
    "Richmond, CA": "San Francisco-Oakland-Berkeley, CA Metro Area",
    "Ridgemark, CA": "San Jose-Sunnyvale-Santa Clara, CA Metro Area",
    "Rocklin, CA": "Sacramento-Roseville-Folsom, CA Metro Area",
    "Rosemont, CA": "Sacramento-Roseville-Folsom, CA Metro Area",
    "Roseville, CA": "Sacramento-Roseville-Folsom, CA Metro Area",
    "Sacramento, CA": "Sacramento-Roseville-Folsom, CA Metro Area",
    "San Clemente, CA": "Los Angeles-Long Beach-Anaheim, CA Metro Area",
    "San Diego, CA": "San Diego-Chula Vista-Carlsbad, CA Metro Area",
    "San Francisco, CA": "San Francisco-Oakland-Berkeley, CA Metro Area",
    "San Jose, CA": "San Jose-Sunnyvale-Santa Clara, CA Metro Area",
    "San Leandro, CA": "San Francisco-Oakland-Berkeley, CA Metro Area",
    "San Marcos, CA": "San Diego-Chula Vista-Carlsbad, CA Metro Area",
    "San Martin, CA": "San Jose-Sunnyvale-Santa Clara, CA Metro Area",
    "San Mateo, CA": "San Francisco-Oakland-Berkeley, CA Metro Area",
    "San Rafael, CA": "San Francisco-Oakland-Berkeley, CA Metro Area",
    "San Ramon, CA": "San Francisco-Oakland-Berkeley, CA Metro Area",
    "Santa Ana, CA": "Los Angeles-Long Beach-Anaheim, CA Metro Area",
    "Santa Clara, CA": "San Jose-Sunnyvale-Santa Clara, CA Metro Area",
    "Santa Clarita, CA": "Los Angeles-Long Beach-Anaheim, CA Metro Area",
    "Santa Cruz, CA": "Santa Cruz-Watsonville, CA Metro Area",
    "Santa Monica, CA": "Los Angeles-Long Beach-Anaheim, CA Metro Area",
    "Santa Rosa, CA": "Santa Rosa-Petaluma, CA Metro Area",
    "Santee, CA": "San Diego-Chula Vista-Carlsbad, CA Metro Area",
    "Saratoga, CA": "San Jose-Sunnyvale-Santa Clara, CA Metro Area",
    "Solana Beach, CA": "San Diego-Chula Vista-Carlsbad, CA Metro Area",
    "South Lake Tahoe, CA": "Sacramento-Roseville-Folsom, CA Metro Area",
    "Spring Valley, CA": "San Diego-Chula Vista-Carlsbad, CA Metro Area",
    "Stanford, CA": "San Jose-Sunnyvale-Santa Clara, CA Metro Area",
    "Stanton, CA": "Los Angeles-Long Beach-Anaheim, CA Metro Area",
    "Sunnyvale, CA": "San Jose-Sunnyvale-Santa Clara, CA Metro Area",
    "Torrance, CA": "Los Angeles-Long Beach-Anaheim, CA Metro Area",
    "Tustin, CA": "Los Angeles-Long Beach-Anaheim, CA Metro Area",
    "Union City, CA": "San Francisco-Oakland-Berkeley, CA Metro Area",
    "Vineyard, CA": "Sacramento-Roseville-Folsom, CA Metro Area",
    "Vista, CA": "San Diego-Chula Vista-Carlsbad, CA Metro Area",
    "Walnut Creek, CA": "San Francisco-Oakland-Berkeley, CA Metro Area",
    "West Covina, CA": "Los Angeles-Long Beach-Anaheim, CA Metro Area",
    "West Sacramento, CA": "Sacramento-Roseville-Folsom, CA Metro Area",
    "Westminster, CA": "Los Angeles-Long Beach-Anaheim, CA Metro Area",
    "Winter Gardens, CA": "San Diego-Chula Vista-Carlsbad, CA Metro Area",
    "Woodland, CA": "Sacramento-Roseville-Folsom, CA Metro Area",
    "Yorba Linda, CA": "Los Angeles-Long Beach-Anaheim, CA Metro Area",

    # Colorado - 25 cities
    "Arvada, CO": "Denver-Aurora-Lakewood, CO Metro Area",
    "Aurora, CO": "Denver-Aurora-Lakewood, CO Metro Area",
    "Boulder, CO": "Boulder, CO Metro Area",
    "Brighton, CO": "Denver-Aurora-Lakewood, CO Metro Area",
    "Broomfield, CO": "Denver-Aurora-Lakewood, CO Metro Area",
    "Castle Rock, CO": "Denver-Aurora-Lakewood, CO Metro Area",
    "Centennial, CO": "Denver-Aurora-Lakewood, CO Metro Area",
    "Columbine, CO": "Denver-Aurora-Lakewood, CO Metro Area",
    "Commerce City, CO": "Denver-Aurora-Lakewood, CO Metro Area",
    "Dakota Ridge, CO": "Denver-Aurora-Lakewood, CO Metro Area",
    "Denver, CO": "Denver-Aurora-Lakewood, CO Metro Area",
    "Englewood, CO": "Denver-Aurora-Lakewood, CO Metro Area",
    "Fort Collins, CO": "Fort Collins, CO Metro Area",
    "Golden, CO": "Denver-Aurora-Lakewood, CO Metro Area",
    "Greenwood Village, CO": "Denver-Aurora-Lakewood, CO Metro Area",
    "Highlands Ranch, CO": "Denver-Aurora-Lakewood, CO Metro Area",
    "Ken Caryl, CO": "Denver-Aurora-Lakewood, CO Metro Area",
    "Lakewood, CO": "Denver-Aurora-Lakewood, CO Metro Area",
    "Littleton, CO": "Denver-Aurora-Lakewood, CO Metro Area",
    "Northglenn, CO": "Denver-Aurora-Lakewood, CO Metro Area",
    "Parker, CO": "Denver-Aurora-Lakewood, CO Metro Area",
    "Sherrelwood, CO": "Denver-Aurora-Lakewood, CO Metro Area",
    "Thornton, CO": "Denver-Aurora-Lakewood, CO Metro Area",
    "Welby, CO": "Denver-Aurora-Lakewood, CO Metro Area",
    "Westminster, CO": "Denver-Aurora-Lakewood, CO Metro Area",
    "Wheat Ridge, CO": "Denver-Aurora-Lakewood, CO Metro Area",

    # Connecticut - 51 cities
    "Ansonia, CT": "Bridgeport-Stamford-Norwalk, CT Metro Area",
    "Bolton, CT": "Hartford-East Hartford-Middletown, CT Metro Area",
    "Branford, CT": "New Haven-Milford, CT Metro Area",
    "Bridgewater, CT": "Bridgeport-Stamford-Norwalk, CT Metro Area",
    "Bristol, CT": "Hartford-East Hartford-Middletown, CT Metro Area",
    "Broad Brook, CT": "Hartford-East Hartford-Middletown, CT Metro Area",
    "Cheshire, CT": "New Haven-Milford, CT Metro Area",
    "Clinton, CT": "New Haven-Milford, CT Metro Area",
    "Collinsville, CT": "Hartford-East Hartford-Middletown, CT Metro Area",
    "Derby, CT": "Bridgeport-Stamford-Norwalk, CT Metro Area",
    "Dover Plains, CT": "New York-Newark-Jersey City, NY-NJ-PA Metro Area",
    "East Hartford, CT": "Hartford-East Hartford-Middletown, CT Metro Area",
    "East Haven, CT": "New Haven-Milford, CT Metro Area",
    "Glastonbury, CT": "Hartford-East Hartford-Middletown, CT Metro Area",
    "Guilford, CT": "New Haven-Milford, CT Metro Area",
    "Hamden, CT": "New Haven-Milford, CT Metro Area",
    "Hartford, CT": "Hartford-East Hartford-Middletown, CT Metro Area",
    "Hazardville, CT": "Hartford-East Hartford-Middletown, CT Metro Area",
    "Heritage, CT": "Hartford-East Hartford-Middletown, CT Metro Area",
    "Kensington, CT": "Hartford-East Hartford-Middletown, CT Metro Area",
    "Madison, CT": "New Haven-Milford, CT Metro Area",
    "Manchester, CT": "Hartford-East Hartford-Middletown, CT Metro Area",
    "Meriden, CT": "Hartford-East Hartford-Middletown, CT Metro Area",
    "Middletown, CT": "Hartford-East Hartford-Middletown, CT Metro Area",
    "Milford, CT": "Bridgeport-Stamford-Norwalk, CT Metro Area",
    "Milton, CT": "Hartford-East Hartford-Middletown, CT Metro Area",
    "Naugatuck, CT": "Waterbury, CT Metro Area",
    "New Britain, CT": "Hartford-East Hartford-Middletown, CT Metro Area",
    "New Haven, CT": "New Haven-Milford, CT Metro Area",
    "New Milford, CT": "Bridgeport-Stamford-Norwalk, CT Metro Area",
    "Newington, CT": "Hartford-East Hartford-Middletown, CT Metro Area",
    "Newtown, CT": "Bridgeport-Stamford-Norwalk, CT Metro Area",
    "North Haven, CT": "New Haven-Milford, CT Metro Area",
    "Orange, CT": "New Haven-Milford, CT Metro Area",
    "Portland, CT": "Hartford-East Hartford-Middletown, CT Metro Area",
    "Riverside, CT": "Bridgeport-Stamford-Norwalk, CT Metro Area",
    "Rockville, CT": "Hartford-East Hartford-Middletown, CT Metro Area",
    "Shelton, CT": "Bridgeport-Stamford-Norwalk, CT Metro Area",
    "Sherwood Manor, CT": "Hartford-East Hartford-Middletown, CT Metro Area",
    "Shirley, CT": "Hartford-East Hartford-Middletown, CT Metro Area",
    "Simsbury, CT": "Hartford-East Hartford-Middletown, CT Metro Area",
    "Southwood Acres, CT": "Hartford-East Hartford-Middletown, CT Metro Area",
    "Stafford Springs, CT": "Hartford-East Hartford-Middletown, CT Metro Area",
    "Stamford, CT": "Bridgeport-Stamford-Norwalk, CT Metro Area",
    "Storrs, CT": "Hartford-East Hartford-Middletown, CT Metro Area",
    "Thompsonville, CT": "Hartford-East Hartford-Middletown, CT Metro Area",
    "Wallingford, CT": "New Haven-Milford, CT Metro Area",
    "Waterbury, CT": "Waterbury, CT Metro Area",
    "West Hartford, CT": "Hartford-East Hartford-Middletown, CT Metro Area",
    "West Haven, CT": "New Haven-Milford, CT Metro Area",
    "Wethersfield, CT": "Hartford-East Hartford-Middletown, CT Metro Area",
    "Windsor Locks, CT": "Hartford-East Hartford-Middletown, CT Metro Area",
    "Woodmon, CT": "Hartford-East Hartford-Middletown, CT Metro Area",

    # Delaware - 14 cities
    "Bear, DE": "Philadelphia-Camden-Wilmington, PA-NJ-DE-MD Metro Area",
    "Bellefonte, DE": "Philadelphia-Camden-Wilmington, PA-NJ-DE-MD Metro Area",
    "Christiana, DE": "Philadelphia-Camden-Wilmington, PA-NJ-DE-MD Metro Area",
    "Claymont, DE": "Philadelphia-Camden-Wilmington, PA-NJ-DE-MD Metro Area",
    "Delaware City, DE": "Philadelphia-Camden-Wilmington, PA-NJ-DE-MD Metro Area",
    "Elsmere, DE": "Philadelphia-Camden-Wilmington, PA-NJ-DE-MD Metro Area",
    "Glasgow, DE": "Philadelphia-Camden-Wilmington, PA-NJ-DE-MD Metro Area",
    "Hockessin, DE": "Philadelphia-Camden-Wilmington, PA-NJ-DE-MD Metro Area",
    "Middletown, DE": "Philadelphia-Camden-Wilmington, PA-NJ-DE-MD Metro Area",
    "Minquadale, DE": "Philadelphia-Camden-Wilmington, PA-NJ-DE-MD Metro Area",
    "Montchanin, DE": "Philadelphia-Camden-Wilmington, PA-NJ-DE-MD Metro Area",
    "New Castle, DE": "Philadelphia-Camden-Wilmington, PA-NJ-DE-MD Metro Area",
    "Willow Grove, DE": "Philadelphia-Camden-Wilmington, PA-NJ-DE-MD Metro Area",
    "Wilmington, DE": "Philadelphia-Camden-Wilmington, PA-NJ-DE-MD Metro Area",

    # Florida - 26 cities (South Florida MSA)
    "Boca Raton, FL": "Miami-Fort Lauderdale-Pompano Beach, FL Metro Area",
    "Boynton Beach, FL": "Miami-Fort Lauderdale-Pompano Beach, FL Metro Area",
    "Coconut Creek, FL": "Miami-Fort Lauderdale-Pompano Beach, FL Metro Area",
    "Coral Springs, FL": "Miami-Fort Lauderdale-Pompano Beach, FL Metro Area",
    "Davie, FL": "Miami-Fort Lauderdale-Pompano Beach, FL Metro Area",
    "Deerfield Beach, FL": "Miami-Fort Lauderdale-Pompano Beach, FL Metro Area",
    "Delray Beach, FL": "Miami-Fort Lauderdale-Pompano Beach, FL Metro Area",
    "Fort Lauderdale, FL": "Miami-Fort Lauderdale-Pompano Beach, FL Metro Area",
    "Hialeah, FL": "Miami-Fort Lauderdale-Pompano Beach, FL Metro Area",
    "Hollywood, FL": "Miami-Fort Lauderdale-Pompano Beach, FL Metro Area",
    "Homestead, FL": "Miami-Fort Lauderdale-Pompano Beach, FL Metro Area",
    "Kendall, FL": "Miami-Fort Lauderdale-Pompano Beach, FL Metro Area",
    "Lauderhill, FL": "Miami-Fort Lauderdale-Pompano Beach, FL Metro Area",
    "Miami, FL": "Miami-Fort Lauderdale-Pompano Beach, FL Metro Area",
    "Miami Beach, FL": "Miami-Fort Lauderdale-Pompano Beach, FL Metro Area",
    "Miami Gardens, FL": "Miami-Fort Lauderdale-Pompano Beach, FL Metro Area",
    "Miramar, FL": "Miami-Fort Lauderdale-Pompano Beach, FL Metro Area",
    "North Miami, FL": "Miami-Fort Lauderdale-Pompano Beach, FL Metro Area",
    "Pembroke Pines, FL": "Miami-Fort Lauderdale-Pompano Beach, FL Metro Area",
    "Plantation, FL": "Miami-Fort Lauderdale-Pompano Beach, FL Metro Area",
    "Pompano Beach, FL": "Miami-Fort Lauderdale-Pompano Beach, FL Metro Area",
    "Sunrise, FL": "Miami-Fort Lauderdale-Pompano Beach, FL Metro Area",
    "Tamarac, FL": "Miami-Fort Lauderdale-Pompano Beach, FL Metro Area",
    "Wellington, FL": "Miami-Fort Lauderdale-Pompano Beach, FL Metro Area",
    "West Palm Beach, FL": "Miami-Fort Lauderdale-Pompano Beach, FL Metro Area",
    "Weston, FL": "Miami-Fort Lauderdale-Pompano Beach, FL Metro Area",

    # Georgia - 26 cities (Atlanta MSA)
    "Alpharetta, GA": "Atlanta-Sandy Springs-Alpharetta, GA Metro Area",
    "Atlanta, GA": "Atlanta-Sandy Springs-Alpharetta, GA Metro Area",
    "Brookhaven, GA": "Atlanta-Sandy Springs-Alpharetta, GA Metro Area",
    "Canton, GA": "Atlanta-Sandy Springs-Alpharetta, GA Metro Area",
    "Carrollton, GA": "Atlanta-Sandy Springs-Alpharetta, GA Metro Area",
    "Chamblee, GA": "Atlanta-Sandy Springs-Alpharetta, GA Metro Area",
    "Douglasville, GA": "Atlanta-Sandy Springs-Alpharetta, GA Metro Area",
    "Duluth, GA": "Atlanta-Sandy Springs-Alpharetta, GA Metro Area",
    "Dunwoody, GA": "Atlanta-Sandy Springs-Alpharetta, GA Metro Area",
    "East Point, GA": "Atlanta-Sandy Springs-Alpharetta, GA Metro Area",
    "Johns Creek, GA": "Atlanta-Sandy Springs-Alpharetta, GA Metro Area",
    "Kennesaw, GA": "Atlanta-Sandy Springs-Alpharetta, GA Metro Area",
    "Lawrenceville, GA": "Atlanta-Sandy Springs-Alpharetta, GA Metro Area",
    "Mableton, GA": "Atlanta-Sandy Springs-Alpharetta, GA Metro Area",
    "Marietta, GA": "Atlanta-Sandy Springs-Alpharetta, GA Metro Area",
    "Milton, GA": "Atlanta-Sandy Springs-Alpharetta, GA Metro Area",
    "Newnan, GA": "Atlanta-Sandy Springs-Alpharetta, GA Metro Area",
    "Peachtree City, GA": "Atlanta-Sandy Springs-Alpharetta, GA Metro Area",
    "Peachtree Corners, GA": "Atlanta-Sandy Springs-Alpharetta, GA Metro Area",
    "Redan, GA": "Atlanta-Sandy Springs-Alpharetta, GA Metro Area",
    "Roswell, GA": "Atlanta-Sandy Springs-Alpharetta, GA Metro Area",
    "Sandy Springs, GA": "Atlanta-Sandy Springs-Alpharetta, GA Metro Area",
    "Smyrna, GA": "Atlanta-Sandy Springs-Alpharetta, GA Metro Area",
    "Stockbridge, GA": "Atlanta-Sandy Springs-Alpharetta, GA Metro Area",
    "Tucker, GA": "Atlanta-Sandy Springs-Alpharetta, GA Metro Area",
    "Woodstock, GA": "Atlanta-Sandy Springs-Alpharetta, GA Metro Area",

    # Illinois - 24 cities (Chicago MSA)
    "Arlington Heights, IL": "Chicago-Naperville-Elgin, IL-IN-WI Metro Area",
    "Aurora, IL": "Chicago-Naperville-Elgin, IL-IN-WI Metro Area",
    "Berwyn, IL": "Chicago-Naperville-Elgin, IL-IN-WI Metro Area",
    "Bolingbrook, IL": "Chicago-Naperville-Elgin, IL-IN-WI Metro Area",
    "Chicago, IL": "Chicago-Naperville-Elgin, IL-IN-WI Metro Area",
    "Cicero, IL": "Chicago-Naperville-Elgin, IL-IN-WI Metro Area",
    "Des Plaines, IL": "Chicago-Naperville-Elgin, IL-IN-WI Metro Area",
    "Downers Grove, IL": "Chicago-Naperville-Elgin, IL-IN-WI Metro Area",
    "Elgin, IL": "Chicago-Naperville-Elgin, IL-IN-WI Metro Area",
    "Evanston, IL": "Chicago-Naperville-Elgin, IL-IN-WI Metro Area",
    "Glenview, IL": "Chicago-Naperville-Elgin, IL-IN-WI Metro Area",
    "Joliet, IL": "Chicago-Naperville-Elgin, IL-IN-WI Metro Area",
    "Mount Prospect, IL": "Chicago-Naperville-Elgin, IL-IN-WI Metro Area",
    "Naperville, IL": "Chicago-Naperville-Elgin, IL-IN-WI Metro Area",
    "Oak Lawn, IL": "Chicago-Naperville-Elgin, IL-IN-WI Metro Area",
    "Oak Park, IL": "Chicago-Naperville-Elgin, IL-IN-WI Metro Area",
    "Orland Park, IL": "Chicago-Naperville-Elgin, IL-IN-WI Metro Area",
    "Palatine, IL": "Chicago-Naperville-Elgin, IL-IN-WI Metro Area",
    "Schaumburg, IL": "Chicago-Naperville-Elgin, IL-IN-WI Metro Area",
    "Skokie, IL": "Chicago-Naperville-Elgin, IL-IN-WI Metro Area",
    "Tinley Park, IL": "Chicago-Naperville-Elgin, IL-IN-WI Metro Area",
    "Waukegan, IL": "Chicago-Naperville-Elgin, IL-IN-WI Metro Area",
    "Wheaton, IL": "Chicago-Naperville-Elgin, IL-IN-WI Metro Area",
    "Yorkville, IL": "Chicago-Naperville-Elgin, IL-IN-WI Metro Area",

    # Indiana - 2 cities (Chicago MSA)
    "Gary, IN": "Chicago-Naperville-Elgin, IL-IN-WI Metro Area",
    "Hammond, IN": "Chicago-Naperville-Elgin, IL-IN-WI Metro Area",

    # Maryland - 49 cities (Baltimore and DC MSAs)
    "Aberdeen, MD": "Baltimore-Columbia-Towson, MD Metro Area",
    "Annapolis, MD": "Baltimore-Columbia-Towson, MD Metro Area",
    "Annapolis Neck, MD": "Baltimore-Columbia-Towson, MD Metro Area",
    "Arnold, MD": "Baltimore-Columbia-Towson, MD Metro Area",
    "Baltimore, MD": "Baltimore-Columbia-Towson, MD Metro Area",
    "Bel Air, MD": "Baltimore-Columbia-Towson, MD Metro Area",
    "Brooklyn Park, MD": "Baltimore-Columbia-Towson, MD Metro Area",
    "Cape St. Claire, MD": "Baltimore-Columbia-Towson, MD Metro Area",
    "Chase, MD": "Baltimore-Columbia-Towson, MD Metro Area",
    "Chestertown, MD": "Baltimore-Columbia-Towson, MD Metro Area",
    "Crofton, MD": "Baltimore-Columbia-Towson, MD Metro Area",
    "Deale, MD": "Baltimore-Columbia-Towson, MD Metro Area",
    "Dundalk, MD": "Baltimore-Columbia-Towson, MD Metro Area",
    "Edgewater, MD": "Baltimore-Columbia-Towson, MD Metro Area",
    "Edgewood, MD": "Baltimore-Columbia-Towson, MD Metro Area",
    "Essex, MD": "Baltimore-Columbia-Towson, MD Metro Area",
    "Ferndale, MD": "Baltimore-Columbia-Towson, MD Metro Area",
    "Finksburg, MD": "Baltimore-Columbia-Towson, MD Metro Area",
    "Fort Meade, MD": "Baltimore-Columbia-Towson, MD Metro Area",
    "Fredericksburg, MD": "Washington-Arlington-Alexandria, DC-VA-MD-WV Metro Area",
    "Freeland, MD": "Baltimore-Columbia-Towson, MD Metro Area",
    "Glen Burnie, MD": "Baltimore-Columbia-Towson, MD Metro Area",
    "Glenville, MD": "Baltimore-Columbia-Towson, MD Metro Area",
    "Gunpowder, MD": "Baltimore-Columbia-Towson, MD Metro Area",
    "Halethorpe, MD": "Baltimore-Columbia-Towson, MD Metro Area",
    "Herald Harbor, MD": "Baltimore-Columbia-Towson, MD Metro Area",
    "Hunt Valley, MD": "Baltimore-Columbia-Towson, MD Metro Area",
    "Jarrettsville, MD": "Baltimore-Columbia-Towson, MD Metro Area",
    "Jessup, MD": "Baltimore-Columbia-Towson, MD Metro Area",
    "Kingsville, MD": "Baltimore-Columbia-Towson, MD Metro Area",
    "Lake Shore, MD": "Baltimore-Columbia-Towson, MD Metro Area",
    "Linthicum Heights, MD": "Baltimore-Columbia-Towson, MD Metro Area",
    "Manchester, MD": "Baltimore-Columbia-Towson, MD Metro Area",
    "Maryland City, MD": "Baltimore-Columbia-Towson, MD Metro Area",
    "Mayo, MD": "Baltimore-Columbia-Towson, MD Metro Area",
    "Mount Rainier, MD": "Washington-Arlington-Alexandria, DC-VA-MD-WV Metro Area",
    "Naval Academy, MD": "Baltimore-Columbia-Towson, MD Metro Area",
    "Odenton, MD": "Baltimore-Columbia-Towson, MD Metro Area",
    "Parkville, MD": "Baltimore-Columbia-Towson, MD Metro Area",
    "Parole, MD": "Baltimore-Columbia-Towson, MD Metro Area",
    "Pasadena, MD": "Baltimore-Columbia-Towson, MD Metro Area",
    "Randallstown, MD": "Baltimore-Columbia-Towson, MD Metro Area",
    "Riva, MD": "Baltimore-Columbia-Towson, MD Metro Area",
    "Riviera Beach, MD": "Baltimore-Columbia-Towson, MD Metro Area",
    "Severn, MD": "Baltimore-Columbia-Towson, MD Metro Area",
    "Severna Park, MD": "Baltimore-Columbia-Towson, MD Metro Area",
    "Shady Side, MD": "Baltimore-Columbia-Towson, MD Metro Area",
    "Stevenson, MD": "Baltimore-Columbia-Towson, MD Metro Area",
    "Towson, MD": "Baltimore-Columbia-Towson, MD Metro Area",
    "White Marsh, MD": "Baltimore-Columbia-Towson, MD Metro Area",
    "Woodstock, MD": "Baltimore-Columbia-Towson, MD Metro Area",

    # Massachusetts - 75 cities
    "Acushnet, MA": "New Bedford, MA Metro Area",
    "Andover, MA": "Boston-Cambridge-Newton, MA-NH Metro Area",
    "Attleboro, MA": "Providence-Warwick, RI-MA Metro Area",
    "Barnstable, MA": "Barnstable Town, MA Metro Area",
    "Berkley, MA": "New Bedford, MA Metro Area",
    "Beverly, MA": "Boston-Cambridge-Newton, MA-NH Metro Area",
    "Boston, MA": "Boston-Cambridge-Newton, MA-NH Metro Area",
    "Boxford, MA": "Boston-Cambridge-Newton, MA-NH Metro Area",
    "Braintree, MA": "Boston-Cambridge-Newton, MA-NH Metro Area",
    "Brockton, MA": "Boston-Cambridge-Newton, MA-NH Metro Area",
    "Brookline, MA": "Boston-Cambridge-Newton, MA-NH Metro Area",
    "Burlington, MA": "Boston-Cambridge-Newton, MA-NH Metro Area",
    "Cambridge, MA": "Boston-Cambridge-Newton, MA-NH Metro Area",
    "Chatham, MA": "Barnstable Town, MA Metro Area",
    "Chelsea, MA": "Boston-Cambridge-Newton, MA-NH Metro Area",
    "Danvers, MA": "Boston-Cambridge-Newton, MA-NH Metro Area",
    "Dartmouth, MA": "New Bedford, MA Metro Area",
    "Dedham, MA": "Boston-Cambridge-Newton, MA-NH Metro Area",
    "Dighton, MA": "Providence-Warwick, RI-MA Metro Area",
    "Everett, MA": "Boston-Cambridge-Newton, MA-NH Metro Area",
    "Fall River, MA": "Providence-Warwick, RI-MA Metro Area",
    "Fitchburg, MA": "Leominster-Gardner, MA Metro Area",
    "Foxborough, MA": "Boston-Cambridge-Newton, MA-NH Metro Area",
    "Framingham, MA": "Boston-Cambridge-Newton, MA-NH Metro Area",
    "Franklin, MA": "Boston-Cambridge-Newton, MA-NH Metro Area",
    "Haverhill, MA": "Boston-Cambridge-Newton, MA-NH Metro Area",
    "Lawrence, MA": "Boston-Cambridge-Newton, MA-NH Metro Area",
    "Lowell, MA": "Boston-Cambridge-Newton, MA-NH Metro Area",
    "Lynn, MA": "Boston-Cambridge-Newton, MA-NH Metro Area",
    "Lynnfield, MA": "Boston-Cambridge-Newton, MA-NH Metro Area",
    "Malden, MA": "Boston-Cambridge-Newton, MA-NH Metro Area",
    "Mansfield, MA": "Boston-Cambridge-Newton, MA-NH Metro Area",
    "Marlborough, MA": "Boston-Cambridge-Newton, MA-NH Metro Area",
    "Medford, MA": "Boston-Cambridge-Newton, MA-NH Metro Area",
    "Melrose, MA": "Boston-Cambridge-Newton, MA-NH Metro Area",
    "Methuen, MA": "Boston-Cambridge-Newton, MA-NH Metro Area",
    "Milford, MA": "Boston-Cambridge-Newton, MA-NH Metro Area",
    "Natick, MA": "Boston-Cambridge-Newton, MA-NH Metro Area",
    "Needham, MA": "Boston-Cambridge-Newton, MA-NH Metro Area",
    "New Bedford, MA": "New Bedford, MA Metro Area",
    "Newburyport, MA": "Boston-Cambridge-Newton, MA-NH Metro Area",
    "Newton, MA": "Boston-Cambridge-Newton, MA-NH Metro Area",
    "Norfolk, MA": "Boston-Cambridge-Newton, MA-NH Metro Area",
    "North Attleborough, MA": "Providence-Warwick, RI-MA Metro Area",
    "Northampton, MA": "Springfield, MA Metro Area",
    "Norton, MA": "Boston-Cambridge-Newton, MA-NH Metro Area",
    "Peabody, MA": "Boston-Cambridge-Newton, MA-NH Metro Area",
    "Plainville, MA": "Boston-Cambridge-Newton, MA-NH Metro Area",
    "Quincy, MA": "Boston-Cambridge-Newton, MA-NH Metro Area",
    "Raynham, MA": "Providence-Warwick, RI-MA Metro Area",
    "Reading, MA": "Boston-Cambridge-Newton, MA-NH Metro Area",
    "Rehoboth, MA": "Providence-Warwick, RI-MA Metro Area",
    "Revere, MA": "Boston-Cambridge-Newton, MA-NH Metro Area",
    "Seekonk, MA": "Providence-Warwick, RI-MA Metro Area",
    "Somerset, MA": "Providence-Warwick, RI-MA Metro Area",
    "Somerville, MA": "Boston-Cambridge-Newton, MA-NH Metro Area",
    "Springfield, MA": "Springfield, MA Metro Area",
    "Stoughton, MA": "Boston-Cambridge-Newton, MA-NH Metro Area",
    "Swansea, MA": "Providence-Warwick, RI-MA Metro Area",
    "Taunton, MA": "Providence-Warwick, RI-MA Metro Area",
    "Tewksbury, MA": "Boston-Cambridge-Newton, MA-NH Metro Area",
    "Wakefield, MA": "Boston-Cambridge-Newton, MA-NH Metro Area",
    "Walpole, MA": "Boston-Cambridge-Newton, MA-NH Metro Area",
    "Waltham, MA": "Boston-Cambridge-Newton, MA-NH Metro Area",
    "Watertown, MA": "Boston-Cambridge-Newton, MA-NH Metro Area",
    "Wayland, MA": "Boston-Cambridge-Newton, MA-NH Metro Area",
    "Wellesley, MA": "Boston-Cambridge-Newton, MA-NH Metro Area",
    "Westport, MA": "Providence-Warwick, RI-MA Metro Area",
    "Weymouth, MA": "Boston-Cambridge-Newton, MA-NH Metro Area",
    "Wilmington, MA": "Boston-Cambridge-Newton, MA-NH Metro Area",
    "Winchester, MA": "Boston-Cambridge-Newton, MA-NH Metro Area",
    "Winthrop, MA": "Boston-Cambridge-Newton, MA-NH Metro Area",
    "Woburn, MA": "Boston-Cambridge-Newton, MA-NH Metro Area",
    "Worcester, MA": "Worcester, MA-CT Metro Area",
    "Wrentham, MA": "Boston-Cambridge-Newton, MA-NH Metro Area",

    # New Hampshire - 19 cities (Boston MSA)
    "Atkinson, NH": "Boston-Cambridge-Newton, MA-NH Metro Area",
    "Derry, NH": "Boston-Cambridge-Newton, MA-NH Metro Area",
    "East Derry, NH": "Boston-Cambridge-Newton, MA-NH Metro Area",
    "East Hampstead, NH": "Boston-Cambridge-Newton, MA-NH Metro Area",
    "East Kingston, NH": "Boston-Cambridge-Newton, MA-NH Metro Area",
    "Fremont, NH": "Boston-Cambridge-Newton, MA-NH Metro Area",
    "Hampstead, NH": "Boston-Cambridge-Newton, MA-NH Metro Area",
    "Hampton, NH": "Boston-Cambridge-Newton, MA-NH Metro Area",
    "Hampton Falls, NH": "Boston-Cambridge-Newton, MA-NH Metro Area",
    "Hollis, NH": "Boston-Cambridge-Newton, MA-NH Metro Area",
    "Litchfield, NH": "Boston-Cambridge-Newton, MA-NH Metro Area",
    "Londonderry, NH": "Boston-Cambridge-Newton, MA-NH Metro Area",
    "Merrimack, NH": "Boston-Cambridge-Newton, MA-NH Metro Area",
    "Nashua, NH": "Boston-Cambridge-Newton, MA-NH Metro Area",
    "North Hampton, NH": "Boston-Cambridge-Newton, MA-NH Metro Area",
    "Pelham, NH": "Boston-Cambridge-Newton, MA-NH Metro Area",
    "Plaistow, NH": "Boston-Cambridge-Newton, MA-NH Metro Area",
    "Sandown, NH": "Boston-Cambridge-Newton, MA-NH Metro Area",
    "Seabrook, NH": "Boston-Cambridge-Newton, MA-NH Metro Area",

    # New Jersey - 52 cities
    "Bayonne, NJ": "New York-Newark-Jersey City, NY-NJ-PA Metro Area",
    "Brentwood, NJ": "New York-Newark-Jersey City, NY-NJ-PA Metro Area",
    "Camden, NJ": "Philadelphia-Camden-Wilmington, PA-NJ-DE-MD Metro Area",
    "Cherry Hill, NJ": "Philadelphia-Camden-Wilmington, PA-NJ-DE-MD Metro Area",
    "Clifton, NJ": "New York-Newark-Jersey City, NY-NJ-PA Metro Area",
    "Collingswood, NJ": "Philadelphia-Camden-Wilmington, PA-NJ-DE-MD Metro Area",
    "Deptford, NJ": "Philadelphia-Camden-Wilmington, PA-NJ-DE-MD Metro Area",
    "East Orange, NJ": "New York-Newark-Jersey City, NY-NJ-PA Metro Area",
    "Elizabeth, NJ": "New York-Newark-Jersey City, NY-NJ-PA Metro Area",
    "Glassboro, NJ": "Philadelphia-Camden-Wilmington, PA-NJ-DE-MD Metro Area",
    "Groveville, NJ": "Trenton-Princeton, NJ Metro Area",
    "Hackensack, NJ": "New York-Newark-Jersey City, NY-NJ-PA Metro Area",
    "Hamilton Square, NJ": "Trenton-Princeton, NJ Metro Area",
    "Hamilton Township, NJ": "Trenton-Princeton, NJ Metro Area",
    "Hempstead, NJ": "New York-Newark-Jersey City, NY-NJ-PA Metro Area",
    "Hightstown, NJ": "Trenton-Princeton, NJ Metro Area",
    "Hoboken, NJ": "New York-Newark-Jersey City, NY-NJ-PA Metro Area",
    "Hopewell, NJ": "Trenton-Princeton, NJ Metro Area",
    "Jersey City, NJ": "New York-Newark-Jersey City, NY-NJ-PA Metro Area",
    "Lakewood, NJ": "New York-Newark-Jersey City, NY-NJ-PA Metro Area",
    "Lawrenceville, NJ": "Trenton-Princeton, NJ Metro Area",
    "Lindenwold, NJ": "Philadelphia-Camden-Wilmington, PA-NJ-DE-MD Metro Area",
    "Mercerville, NJ": "Trenton-Princeton, NJ Metro Area",
    "Merchantville, NJ": "Philadelphia-Camden-Wilmington, PA-NJ-DE-MD Metro Area",
    "Moorestown, NJ": "Philadelphia-Camden-Wilmington, PA-NJ-DE-MD Metro Area",
    "Mount Vernon, NJ": "New York-Newark-Jersey City, NY-NJ-PA Metro Area",
    "New Brunswick, NJ": "New York-Newark-Jersey City, NY-NJ-PA Metro Area",
    "New Rochelle, NJ": "New York-Newark-Jersey City, NY-NJ-PA Metro Area",
    "Newark, NJ": "New York-Newark-Jersey City, NY-NJ-PA Metro Area",
    "Palmyra, NJ": "Philadelphia-Camden-Wilmington, PA-NJ-DE-MD Metro Area",
    "Paramus, NJ": "New York-Newark-Jersey City, NY-NJ-PA Metro Area",
    "Passaic, NJ": "New York-Newark-Jersey City, NY-NJ-PA Metro Area",
    "Paterson, NJ": "New York-Newark-Jersey City, NY-NJ-PA Metro Area",
    "Pennington, NJ": "Trenton-Princeton, NJ Metro Area",
    "Pennsauken Township, NJ": "Philadelphia-Camden-Wilmington, PA-NJ-DE-MD Metro Area",
    "Perth Amboy, NJ": "New York-Newark-Jersey City, NY-NJ-PA Metro Area",
    "Plainfield, NJ": "New York-Newark-Jersey City, NY-NJ-PA Metro Area",
    "Princeton, NJ": "Trenton-Princeton, NJ Metro Area",
    "Robbinsville, NJ": "Trenton-Princeton, NJ Metro Area",
    "Sayreville, NJ": "New York-Newark-Jersey City, NY-NJ-PA Metro Area",
    "Somerdale, NJ": "Philadelphia-Camden-Wilmington, PA-NJ-DE-MD Metro Area",
    "Toms River, NJ": "New York-Newark-Jersey City, NY-NJ-PA Metro Area",
    "Trenton, NJ": "Trenton-Princeton, NJ Metro Area",
    "Twin Rivers, NJ": "Trenton-Princeton, NJ Metro Area",
    "Union, NJ": "New York-Newark-Jersey City, NY-NJ-PA Metro Area",
    "Union City, NJ": "New York-Newark-Jersey City, NY-NJ-PA Metro Area",
    "West New York, NJ": "New York-Newark-Jersey City, NY-NJ-PA Metro Area",
    "White Horse, NJ": "Trenton-Princeton, NJ Metro Area",
    "White Plains, NJ": "New York-Newark-Jersey City, NY-NJ-PA Metro Area",
    "Yardville, NJ": "Trenton-Princeton, NJ Metro Area",

    # New York - 25 cities (NYC MSA)
    "Astoria, NY": "New York-Newark-Jersey City, NY-NJ-PA Metro Area",
    "Bedford Stuyvesant, NY": "New York-Newark-Jersey City, NY-NJ-PA Metro Area",
    "Bronx, NY": "New York-Newark-Jersey City, NY-NJ-PA Metro Area",
    "Brooklyn, NY": "New York-Newark-Jersey City, NY-NJ-PA Metro Area",
    "Bushwick, NY": "New York-Newark-Jersey City, NY-NJ-PA Metro Area",
    "Crown Heights, NY": "New York-Newark-Jersey City, NY-NJ-PA Metro Area",
    "East Flatbush, NY": "New York-Newark-Jersey City, NY-NJ-PA Metro Area",
    "Flatbush, NY": "New York-Newark-Jersey City, NY-NJ-PA Metro Area",
    "Gramercy, NY": "New York-Newark-Jersey City, NY-NJ-PA Metro Area",
    "Harlem, NY": "New York-Newark-Jersey City, NY-NJ-PA Metro Area",
    "Jamaica, NY": "New York-Newark-Jersey City, NY-NJ-PA Metro Area",
    "Long Island, NY": "New York-Newark-Jersey City, NY-NJ-PA Metro Area",
    "Lower East Side, NY": "New York-Newark-Jersey City, NY-NJ-PA Metro Area",
    "Manhattan, NY": "New York-Newark-Jersey City, NY-NJ-PA Metro Area",
    "Midwood, NY": "New York-Newark-Jersey City, NY-NJ-PA Metro Area",
    "New York City, NY": "New York-Newark-Jersey City, NY-NJ-PA Metro Area",
    "Park Slope, NY": "New York-Newark-Jersey City, NY-NJ-PA Metro Area",
    "Queens, NY": "New York-Newark-Jersey City, NY-NJ-PA Metro Area",
    "Ridgewood, NY": "New York-Newark-Jersey City, NY-NJ-PA Metro Area",
    "SoHo, NY": "New York-Newark-Jersey City, NY-NJ-PA Metro Area",
    "Staten Island, NY": "New York-Newark-Jersey City, NY-NJ-PA Metro Area",
    "Upper East Side, NY": "New York-Newark-Jersey City, NY-NJ-PA Metro Area",
    "Upper West Side, NY": "New York-Newark-Jersey City, NY-NJ-PA Metro Area",
    "Williamsbridge, NY": "New York-Newark-Jersey City, NY-NJ-PA Metro Area",
    "Williamsburg, NY": "New York-Newark-Jersey City, NY-NJ-PA Metro Area",
    "Yonkers, NY": "New York-Newark-Jersey City, NY-NJ-PA Metro Area",

    # Oregon - 33 cities (Portland MSA)
    "Aloha, OR": "Portland-Vancouver-Hillsboro, OR-WA Metro Area",
    "Aurora, OR": "Portland-Vancouver-Hillsboro, OR-WA Metro Area",
    "Barlow, OR": "Portland-Vancouver-Hillsboro, OR-WA Metro Area",
    "Battin, OR": "Portland-Vancouver-Hillsboro, OR-WA Metro Area",
    "Beaverton, OR": "Portland-Vancouver-Hillsboro, OR-WA Metro Area",
    "Bethany, OR": "Portland-Vancouver-Hillsboro, OR-WA Metro Area",
    "Canby, OR": "Portland-Vancouver-Hillsboro, OR-WA Metro Area",
    "Chehalem, OR": "Portland-Vancouver-Hillsboro, OR-WA Metro Area",
    "Clackamas, OR": "Portland-Vancouver-Hillsboro, OR-WA Metro Area",
    "Cornelius, OR": "Portland-Vancouver-Hillsboro, OR-WA Metro Area",
    "Damascus, OR": "Portland-Vancouver-Hillsboro, OR-WA Metro Area",
    "Dunthorpe, OR": "Portland-Vancouver-Hillsboro, OR-WA Metro Area",
    "Fairview, OR": "Portland-Vancouver-Hillsboro, OR-WA Metro Area",
    "Forest Grove, OR": "Portland-Vancouver-Hillsboro, OR-WA Metro Area",
    "Gaston, OR": "Portland-Vancouver-Hillsboro, OR-WA Metro Area",
    "Gladstone, OR": "Portland-Vancouver-Hillsboro, OR-WA Metro Area",
    "Gresham, OR": "Portland-Vancouver-Hillsboro, OR-WA Metro Area",
    "Happy Valley, OR": "Portland-Vancouver-Hillsboro, OR-WA Metro Area",
    "Helvetia, OR": "Portland-Vancouver-Hillsboro, OR-WA Metro Area",
    "Hillsboro, OR": "Portland-Vancouver-Hillsboro, OR-WA Metro Area",
    "Lake Oswego, OR": "Portland-Vancouver-Hillsboro, OR-WA Metro Area",
    "McMinnville, OR": "Portland-Vancouver-Hillsboro, OR-WA Metro Area",
    "Milwaukie, OR": "Portland-Vancouver-Hillsboro, OR-WA Metro Area",
    "Newberg, OR": "Portland-Vancouver-Hillsboro, OR-WA Metro Area",
    "North Plains, OR": "Portland-Vancouver-Hillsboro, OR-WA Metro Area",
    "Orchards, OR": "Portland-Vancouver-Hillsboro, OR-WA Metro Area",
    "Oregon City, OR": "Portland-Vancouver-Hillsboro, OR-WA Metro Area",
    "Portland, OR": "Portland-Vancouver-Hillsboro, OR-WA Metro Area",
    "Sherwood, OR": "Portland-Vancouver-Hillsboro, OR-WA Metro Area",
    "Tigard, OR": "Portland-Vancouver-Hillsboro, OR-WA Metro Area",
    "Troutdale, OR": "Portland-Vancouver-Hillsboro, OR-WA Metro Area",
    "Tualatin, OR": "Portland-Vancouver-Hillsboro, OR-WA Metro Area",
    "West Linn, OR": "Portland-Vancouver-Hillsboro, OR-WA Metro Area",
    "Wilsonville, OR": "Portland-Vancouver-Hillsboro, OR-WA Metro Area",
    "Wood Village, OR": "Portland-Vancouver-Hillsboro, OR-WA Metro Area",

    # Pennsylvania - 31 cities (Philadelphia MSA)
    "Ambler, PA": "Philadelphia-Camden-Wilmington, PA-NJ-DE-MD Metro Area",
    "Blue Bell, PA": "Philadelphia-Camden-Wilmington, PA-NJ-DE-MD Metro Area",
    "Bridgeport, PA": "Philadelphia-Camden-Wilmington, PA-NJ-DE-MD Metro Area",
    "Brookside, PA": "Philadelphia-Camden-Wilmington, PA-NJ-DE-MD Metro Area",
    "Broomall, PA": "Philadelphia-Camden-Wilmington, PA-NJ-DE-MD Metro Area",
    "Chester, PA": "Philadelphia-Camden-Wilmington, PA-NJ-DE-MD Metro Area",
    "Conshohocken, PA": "Philadelphia-Camden-Wilmington, PA-NJ-DE-MD Metro Area",
    "Doylestown, PA": "Philadelphia-Camden-Wilmington, PA-NJ-DE-MD Metro Area",
    "Drexel Hill, PA": "Philadelphia-Camden-Wilmington, PA-NJ-DE-MD Metro Area",
    "Exton, PA": "Philadelphia-Camden-Wilmington, PA-NJ-DE-MD Metro Area",
    "Fishtown, PA": "Philadelphia-Camden-Wilmington, PA-NJ-DE-MD Metro Area",
    "Gladwyne, PA": "Philadelphia-Camden-Wilmington, PA-NJ-DE-MD Metro Area",
    "Havertown, PA": "Philadelphia-Camden-Wilmington, PA-NJ-DE-MD Metro Area",
    "Horsham, PA": "Philadelphia-Camden-Wilmington, PA-NJ-DE-MD Metro Area",
    "King of Prussia, PA": "Philadelphia-Camden-Wilmington, PA-NJ-DE-MD Metro Area",
    "Lansdale, PA": "Philadelphia-Camden-Wilmington, PA-NJ-DE-MD Metro Area",
    "Levittown, PA": "Philadelphia-Camden-Wilmington, PA-NJ-DE-MD Metro Area",
    "Malvern, PA": "Philadelphia-Camden-Wilmington, PA-NJ-DE-MD Metro Area",
    "New Hope, PA": "Philadelphia-Camden-Wilmington, PA-NJ-DE-MD Metro Area",
    "Newtown, PA": "Philadelphia-Camden-Wilmington, PA-NJ-DE-MD Metro Area",
    "Norristown, PA": "Philadelphia-Camden-Wilmington, PA-NJ-DE-MD Metro Area",
    "Paoli, PA": "Philadelphia-Camden-Wilmington, PA-NJ-DE-MD Metro Area",
    "Philadelphia, PA": "Philadelphia-Camden-Wilmington, PA-NJ-DE-MD Metro Area",
    "Phoenixville, PA": "Philadelphia-Camden-Wilmington, PA-NJ-DE-MD Metro Area",
    "Plymouth Meeting, PA": "Philadelphia-Camden-Wilmington, PA-NJ-DE-MD Metro Area",
    "Pottstown, PA": "Philadelphia-Camden-Wilmington, PA-NJ-DE-MD Metro Area",
    "Southampton, PA": "Philadelphia-Camden-Wilmington, PA-NJ-DE-MD Metro Area",
    "Springfield, PA": "Philadelphia-Camden-Wilmington, PA-NJ-DE-MD Metro Area",
    "Swarthmore, PA": "Philadelphia-Camden-Wilmington, PA-NJ-DE-MD Metro Area",
    "Villanova, PA": "Philadelphia-Camden-Wilmington, PA-NJ-DE-MD Metro Area",
    "Wayne, PA": "Philadelphia-Camden-Wilmington, PA-NJ-DE-MD Metro Area",
    "West Chester, PA": "Philadelphia-Camden-Wilmington, PA-NJ-DE-MD Metro Area",
    "Willow Grove, PA": "Philadelphia-Camden-Wilmington, PA-NJ-DE-MD Metro Area",

    # Rhode Island - 15 cities (Providence MSA)
    "Barrington, RI": "Providence-Warwick, RI-MA Metro Area",
    "Central Falls, RI": "Providence-Warwick, RI-MA Metro Area",
    "Cranston, RI": "Providence-Warwick, RI-MA Metro Area",
    "Cumberland, RI": "Providence-Warwick, RI-MA Metro Area",
    "East Greenwich, RI": "Providence-Warwick, RI-MA Metro Area",
    "Lincoln, RI": "Providence-Warwick, RI-MA Metro Area",
    "Narragansett, RI": "Providence-Warwick, RI-MA Metro Area",
    "Newport, RI": "Providence-Warwick, RI-MA Metro Area",
    "Pawtucket, RI": "Providence-Warwick, RI-MA Metro Area",
    "Portsmouth, RI": "Providence-Warwick, RI-MA Metro Area",
    "Providence, RI": "Providence-Warwick, RI-MA Metro Area",
    "Smithfield, RI": "Providence-Warwick, RI-MA Metro Area",
    "Tiverton, RI": "Providence-Warwick, RI-MA Metro Area",
    "Warwick, RI": "Providence-Warwick, RI-MA Metro Area",
    "Woonsocket, RI": "Providence-Warwick, RI-MA Metro Area",

    # Texas - 107 cities
    "Adkins, TX": "San Antonio-New Braunfels, TX Metro Area",
    "Allen, TX": "Dallas-Fort Worth-Arlington, TX Metro Area",
    "Alvin, TX": "Houston-The Woodlands-Sugar Land, TX Metro Area",
    "Angleto, TX": "Houston-The Woodlands-Sugar Land, TX Metro Area",
    "Arlington, TX": "Dallas-Fort Worth-Arlington, TX Metro Area",
    "Atascocita, TX": "Houston-The Woodlands-Sugar Land, TX Metro Area",
    "Austin, TX": "Austin-Round Rock-Georgetown, TX Metro Area",
    "Bastrop, TX": "Austin-Round Rock-Georgetown, TX Metro Area",
    "Baytown, TX": "Houston-The Woodlands-Sugar Land, TX Metro Area",
    "Bedford, TX": "Dallas-Fort Worth-Arlington, TX Metro Area",
    "Bee Cave, TX": "Austin-Round Rock-Georgetown, TX Metro Area",
    "Bellaire, TX": "Houston-The Woodlands-Sugar Land, TX Metro Area",
    "Brushy Creek, TX": "Austin-Round Rock-Georgetown, TX Metro Area",
    "Buda, TX": "Austin-Round Rock-Georgetown, TX Metro Area",
    "Camp Swift, TX": "Austin-Round Rock-Georgetown, TX Metro Area",
    "Carrollton, TX": "Dallas-Fort Worth-Arlington, TX Metro Area",
    "Cedar Hill, TX": "Dallas-Fort Worth-Arlington, TX Metro Area",
    "Cedar Park, TX": "Austin-Round Rock-Georgetown, TX Metro Area",
    "Channelview, TX": "Houston-The Woodlands-Sugar Land, TX Metro Area",
    "Cinco Ranch, TX": "Houston-The Woodlands-Sugar Land, TX Metro Area",
    "Cloverleaf, TX": "Houston-The Woodlands-Sugar Land, TX Metro Area",
    "Conroe, TX": "Houston-The Woodlands-Sugar Land, TX Metro Area",
    "Cypress, TX": "Houston-The Woodlands-Sugar Land, TX Metro Area",
    "Dallas, TX": "Dallas-Fort Worth-Arlington, TX Metro Area",
    "DeSoto, TX": "Dallas-Fort Worth-Arlington, TX Metro Area",
    "Deer Park, TX": "Houston-The Woodlands-Sugar Land, TX Metro Area",
    "Denton, TX": "Dallas-Fort Worth-Arlington, TX Metro Area",
    "Dickinson, TX": "Houston-The Woodlands-Sugar Land, TX Metro Area",
    "Elgin, TX": "Austin-Round Rock-Georgetown, TX Metro Area",
    "Euless, TX": "Dallas-Fort Worth-Arlington, TX Metro Area",
    "Flower Mound, TX": "Dallas-Fort Worth-Arlington, TX Metro Area",
    "Fort Worth, TX": "Dallas-Fort Worth-Arlington, TX Metro Area",
    "Fresno, TX": "Houston-The Woodlands-Sugar Land, TX Metro Area",
    "Friendswood, TX": "Houston-The Woodlands-Sugar Land, TX Metro Area",
    "Frisco, TX": "Dallas-Fort Worth-Arlington, TX Metro Area",
    "Fulshear, TX": "Houston-The Woodlands-Sugar Land, TX Metro Area",
    "Galveston, TX": "Houston-The Woodlands-Sugar Land, TX Metro Area",
    "Garland, TX": "Dallas-Fort Worth-Arlington, TX Metro Area",
    "Georgetown, TX": "Austin-Round Rock-Georgetown, TX Metro Area",
    "Grand Prairie, TX": "Dallas-Fort Worth-Arlington, TX Metro Area",
    "Grapevine, TX": "Dallas-Fort Worth-Arlington, TX Metro Area",
    "Hornsby Bend, TX": "Austin-Round Rock-Georgetown, TX Metro Area",
    "Houston, TX": "Houston-The Woodlands-Sugar Land, TX Metro Area",
    "Humble, TX": "Houston-The Woodlands-Sugar Land, TX Metro Area",
    "Hutto, TX": "Austin-Round Rock-Georgetown, TX Metro Area",
    "Irving, TX": "Dallas-Fort Worth-Arlington, TX Metro Area",
    "Jersey Village, TX": "Houston-The Woodlands-Sugar Land, TX Metro Area",
    "Katy, TX": "Houston-The Woodlands-Sugar Land, TX Metro Area",
    "Kindwood, TX": "Houston-The Woodlands-Sugar Land, TX Metro Area",
    "Kyle, TX": "Austin-Round Rock-Georgetown, TX Metro Area",
    "La Porte, TX": "Houston-The Woodlands-Sugar Land, TX Metro Area",
    "Lago Vista, TX": "Austin-Round Rock-Georgetown, TX Metro Area",
    "Lake Jackson, TX": "Houston-The Woodlands-Sugar Land, TX Metro Area",
    "Lakeway, TX": "Austin-Round Rock-Georgetown, TX Metro Area",
    "League City, TX": "Houston-The Woodlands-Sugar Land, TX Metro Area",
    "Leander, TX": "Austin-Round Rock-Georgetown, TX Metro Area",
    "Lewisville, TX": "Dallas-Fort Worth-Arlington, TX Metro Area",
    "Lockhart, TX": "Austin-Round Rock-Georgetown, TX Metro Area",
    "Lost Creek, TX": "Austin-Round Rock-Georgetown, TX Metro Area",
    "Luling, TX": "Austin-Round Rock-Georgetown, TX Metro Area",
    "Manor, TX": "Austin-Round Rock-Georgetown, TX Metro Area",
    "Mansfield, TX": "Dallas-Fort Worth-Arlington, TX Metro Area",
    "McKinney, TX": "Dallas-Fort Worth-Arlington, TX Metro Area",
    "Mesquite, TX": "Dallas-Fort Worth-Arlington, TX Metro Area",
    "Mission Bend, TX": "Houston-The Woodlands-Sugar Land, TX Metro Area",
    "Missouri City, TX": "Houston-The Woodlands-Sugar Land, TX Metro Area",
    "Montrose, TX": "Houston-The Woodlands-Sugar Land, TX Metro Area",
    "North Austin, TX": "Austin-Round Rock-Georgetown, TX Metro Area",
    "North Richland Hills, TX": "Dallas-Fort Worth-Arlington, TX Metro Area",
    "Pasadena, TX": "Houston-The Woodlands-Sugar Land, TX Metro Area",
    "Pearland, TX": "Houston-The Woodlands-Sugar Land, TX Metro Area",
    "Pflugerville, TX": "Austin-Round Rock-Georgetown, TX Metro Area",
    "Plano, TX": "Dallas-Fort Worth-Arlington, TX Metro Area",
    "Richardson, TX": "Dallas-Fort Worth-Arlington, TX Metro Area",
    "Richmond, TX": "Houston-The Woodlands-Sugar Land, TX Metro Area",
    "Rosenberg, TX": "Houston-The Woodlands-Sugar Land, TX Metro Area",
    "Round Rock, TX": "Austin-Round Rock-Georgetown, TX Metro Area",
    "Rowlett, TX": "Dallas-Fort Worth-Arlington, TX Metro Area",
    "San Antonio, TX": "San Antonio-New Braunfels, TX Metro Area",
    "San Marcos, TX": "Austin-Round Rock-Georgetown, TX Metro Area",
    "Shady Hollow, TX": "Austin-Round Rock-Georgetown, TX Metro Area",
    "Smithville, TX": "Austin-Round Rock-Georgetown, TX Metro Area",
    "Spring, TX": "Houston-The Woodlands-Sugar Land, TX Metro Area",
    "Sugar Land, TX": "Houston-The Woodlands-Sugar Land, TX Metro Area",
    "Taylor, TX": "Austin-Round Rock-Georgetown, TX Metro Area",
    "Texas City, TX": "Houston-The Woodlands-Sugar Land, TX Metro Area",
    "The Colony, TX": "Dallas-Fort Worth-Arlington, TX Metro Area",
    "The Woodlands, TX": "Houston-The Woodlands-Sugar Land, TX Metro Area",
    "Tomball, TX": "Houston-The Woodlands-Sugar Land, TX Metro Area",
    "Wells Branch, TX": "Austin-Round Rock-Georgetown, TX Metro Area",
    "Wylie, TX": "Dallas-Fort Worth-Arlington, TX Metro Area",

    # Utah - 54 cities
    "Alpine, UT": "Provo-Orem, UT Metro Area",
    "American Fork, UT": "Provo-Orem, UT Metro Area",
    "Bingham Canyon, UT": "Salt Lake City, UT Metro Area",
    "Bountiful, UT": "Salt Lake City, UT Metro Area",
    "Cedar Valley, UT": "Provo-Orem, UT Metro Area",
    "Centerville, UT": "Salt Lake City, UT Metro Area",
    "Clearfield, UT": "Ogden-Clearfield, UT Metro Area",
    "Coalville, UT": "Salt Lake City, UT Metro Area",
    "Croydon, UT": "Ogden-Clearfield, UT Metro Area",
    "Draper, UT": "Salt Lake City, UT Metro Area",
    "Eagle Mountain, UT": "Provo-Orem, UT Metro Area",
    "Echo, UT": "Salt Lake City, UT Metro Area",
    "Eden, UT": "Ogden-Clearfield, UT Metro Area",
    "Farmington, UT": "Salt Lake City, UT Metro Area",
    "Grantsville, UT": "Salt Lake City, UT Metro Area",
    "Heber City, UT": "Heber, UT Micro Area",
    "Henefer, UT": "Salt Lake City, UT Metro Area",
    "Herriman, UT": "Salt Lake City, UT Metro Area",
    "Hill Afb, UT": "Ogden-Clearfield, UT Metro Area",
    "Hooper, UT": "Ogden-Clearfield, UT Metro Area",
    "Huntsville, UT": "Ogden-Clearfield, UT Metro Area",
    "Kamas, UT": "Heber, UT Micro Area",
    "Kaysville, UT": "Salt Lake City, UT Metro Area",
    "Layton, UT": "Ogden-Clearfield, UT Metro Area",
    "Lehi, UT": "Provo-Orem, UT Metro Area",
    "Lindon, UT": "Provo-Orem, UT Metro Area",
    "Magna, UT": "Salt Lake City, UT Metro Area",
    "Mapleton, UT": "Provo-Orem, UT Metro Area",
    "Midvale, UT": "Salt Lake City, UT Metro Area",
    "Midway, UT": "Heber, UT Micro Area",
    "Morgan, UT": "Ogden-Clearfield, UT Metro Area",
    "North Salt Lake, UT": "Salt Lake City, UT Metro Area",
    "Oakley, UT": "Heber, UT Micro Area",
    "Ogden, UT": "Ogden-Clearfield, UT Metro Area",
    "Orem, UT": "Provo-Orem, UT Metro Area",
    "Park City, UT": "Salt Lake City, UT Metro Area",
    "Payson, UT": "Provo-Orem, UT Metro Area",
    "Peoa, UT": "Heber, UT Micro Area",
    "Pleasant Grove, UT": "Provo-Orem, UT Metro Area",
    "Provo, UT": "Provo-Orem, UT Metro Area",
    "Riverton, UT": "Salt Lake City, UT Metro Area",
    "Roy, UT": "Ogden-Clearfield, UT Metro Area",
    "Rush Valley, UT": "Salt Lake City, UT Metro Area",
    "Salt Lake City, UT": "Salt Lake City, UT Metro Area",
    "Sandy, UT": "Salt Lake City, UT Metro Area",
    "Saratoga Springs, UT": "Provo-Orem, UT Metro Area",
    "South Jordan, UT": "Salt Lake City, UT Metro Area",
    "Spanish Fork, UT": "Provo-Orem, UT Metro Area",
    "Springville, UT": "Provo-Orem, UT Metro Area",
    "Stockton, UT": "Salt Lake City, UT Metro Area",
    "Syracuse, UT": "Ogden-Clearfield, UT Metro Area",
    "Tooele, UT": "Salt Lake City, UT Metro Area",
    "Vineyard, UT": "Provo-Orem, UT Metro Area",
    "Wallsburg, UT": "Heber, UT Micro Area",
    "West Jordan, UT": "Salt Lake City, UT Metro Area",
    "West Valley City, UT": "Salt Lake City, UT Metro Area",
    "Willard, UT": "Ogden-Clearfield, UT Metro Area",
    "Woods Cross, UT": "Salt Lake City, UT Metro Area",

    # Virginia - 23 cities (DC MSA)
    "Alexandria, VA": "Washington-Arlington-Alexandria, DC-VA-MD-WV Metro Area",
    "Annandale, VA": "Washington-Arlington-Alexandria, DC-VA-MD-WV Metro Area",
    "Arlington, VA": "Washington-Arlington-Alexandria, DC-VA-MD-WV Metro Area",
    "Burke, VA": "Washington-Arlington-Alexandria, DC-VA-MD-WV Metro Area",
    "Centreville, VA": "Washington-Arlington-Alexandria, DC-VA-MD-WV Metro Area",
    "Chantilly, VA": "Washington-Arlington-Alexandria, DC-VA-MD-WV Metro Area",
    "Dunn Loring, VA": "Washington-Arlington-Alexandria, DC-VA-MD-WV Metro Area",
    "Fairfax, VA": "Washington-Arlington-Alexandria, DC-VA-MD-WV Metro Area",
    "Falls Church, VA": "Washington-Arlington-Alexandria, DC-VA-MD-WV Metro Area",
    "Great Falls, VA": "Washington-Arlington-Alexandria, DC-VA-MD-WV Metro Area",
    "Greenway, VA": "Washington-Arlington-Alexandria, DC-VA-MD-WV Metro Area",
    "Herndon, VA": "Washington-Arlington-Alexandria, DC-VA-MD-WV Metro Area",
    "Indian Head, VA": "Washington-Arlington-Alexandria, DC-VA-MD-WV Metro Area",
    "Lorton, VA": "Washington-Arlington-Alexandria, DC-VA-MD-WV Metro Area",
    "Newington, VA": "Washington-Arlington-Alexandria, DC-VA-MD-WV Metro Area",
    "Oakton, VA": "Washington-Arlington-Alexandria, DC-VA-MD-WV Metro Area",
    "Occoquan, VA": "Washington-Arlington-Alexandria, DC-VA-MD-WV Metro Area",
    "Reston, VA": "Washington-Arlington-Alexandria, DC-VA-MD-WV Metro Area",
    "Springfield, VA": "Washington-Arlington-Alexandria, DC-VA-MD-WV Metro Area",
    "Sterling, VA": "Washington-Arlington-Alexandria, DC-VA-MD-WV Metro Area",
    "Vienna, VA": "Washington-Arlington-Alexandria, DC-VA-MD-WV Metro Area",
    "West Mclean, VA": "Washington-Arlington-Alexandria, DC-VA-MD-WV Metro Area",
    "Woodbridge, VA": "Washington-Arlington-Alexandria, DC-VA-MD-WV Metro Area",

    # Washington - 35 cities (Seattle MSA)
    "Auburn, WA": "Seattle-Tacoma-Bellevue, WA Metro Area",
    "Barberton, WA": "Seattle-Tacoma-Bellevue, WA Metro Area",
    "Battle Ground, WA": "Portland-Vancouver-Hillsboro, OR-WA Metro Area",
    "Bellevue, WA": "Seattle-Tacoma-Bellevue, WA Metro Area",
    "Bothell, WA": "Seattle-Tacoma-Bellevue, WA Metro Area",
    "Brush Prairie, WA": "Portland-Vancouver-Hillsboro, OR-WA Metro Area",
    "Burien, WA": "Seattle-Tacoma-Bellevue, WA Metro Area",
    "Camas, WA": "Portland-Vancouver-Hillsboro, OR-WA Metro Area",
    "Des Moines, WA": "Seattle-Tacoma-Bellevue, WA Metro Area",
    "Edmonds, WA": "Seattle-Tacoma-Bellevue, WA Metro Area",
    "Everett, WA": "Seattle-Tacoma-Bellevue, WA Metro Area",
    "Federal Way, WA": "Seattle-Tacoma-Bellevue, WA Metro Area",
    "Five Corners, WA": "Portland-Vancouver-Hillsboro, OR-WA Metro Area",
    "Hazel Dell, WA": "Portland-Vancouver-Hillsboro, OR-WA Metro Area",
    "Issaquah, WA": "Seattle-Tacoma-Bellevue, WA Metro Area",
    "Kent, WA": "Seattle-Tacoma-Bellevue, WA Metro Area",
    "Kirkland, WA": "Seattle-Tacoma-Bellevue, WA Metro Area",
    "Lake Stevens, WA": "Seattle-Tacoma-Bellevue, WA Metro Area",
    "Lakewood, WA": "Seattle-Tacoma-Bellevue, WA Metro Area",
    "Lynnwood, WA": "Seattle-Tacoma-Bellevue, WA Metro Area",
    "Marysville, WA": "Seattle-Tacoma-Bellevue, WA Metro Area",
    "Minnehaha, WA": "Portland-Vancouver-Hillsboro, OR-WA Metro Area",
    "Olympia, WA": "Olympia-Lacey-Tumwater, WA Metro Area",
    "Orchards, WA": "Portland-Vancouver-Hillsboro, OR-WA Metro Area",
    "Parkland, WA": "Seattle-Tacoma-Bellevue, WA Metro Area",
    "Puyallup, WA": "Seattle-Tacoma-Bellevue, WA Metro Area",
    "Redmond, WA": "Seattle-Tacoma-Bellevue, WA Metro Area",
    "Renton, WA": "Seattle-Tacoma-Bellevue, WA Metro Area",
    "Ridgefield, WA": "Portland-Vancouver-Hillsboro, OR-WA Metro Area",
    "Salmon Creek, WA": "Portland-Vancouver-Hillsboro, OR-WA Metro Area",
    "Sammamish, WA": "Seattle-Tacoma-Bellevue, WA Metro Area",
    "Seattle, WA": "Seattle-Tacoma-Bellevue, WA Metro Area",
    "Shoreline, WA": "Seattle-Tacoma-Bellevue, WA Metro Area",
    "South Hill, WA": "Seattle-Tacoma-Bellevue, WA Metro Area",
    "Spanaway, WA": "Seattle-Tacoma-Bellevue, WA Metro Area",
    "Tacoma, WA": "Seattle-Tacoma-Bellevue, WA Metro Area",
    "University Place, WA": "Seattle-Tacoma-Bellevue, WA Metro Area",
    "Vancouver, WA": "Portland-Vancouver-Hillsboro, OR-WA Metro Area",
    "Walnut Grove, WA": "Portland-Vancouver-Hillsboro, OR-WA Metro Area",
    "West Linn, WA": "Portland-Vancouver-Hillsboro, OR-WA Metro Area",

    # Wisconsin - 2 cities
    "Kenosha, WI": "Chicago-Naperville-Elgin, IL-IN-WI Metro Area",
    "Pleasant Prairie, WI": "Chicago-Naperville-Elgin, IL-IN-WI Metro Area",

    # District of Columbia - 25 neighborhoods
    "Adams Morgan, DC": "Washington-Arlington-Alexandria, DC-VA-MD-WV Metro Area",
    "American University Park, DC": "Washington-Arlington-Alexandria, DC-VA-MD-WV Metro Area",
    "Brightwood, DC": "Washington-Arlington-Alexandria, DC-VA-MD-WV Metro Area",
    "Capitol Hill, DC": "Washington-Arlington-Alexandria, DC-VA-MD-WV Metro Area",
    "Chevy Chase, DC": "Washington-Arlington-Alexandria, DC-VA-MD-WV Metro Area",
    "Cleveland Park, DC": "Washington-Arlington-Alexandria, DC-VA-MD-WV Metro Area",
    "Columbia Heights, DC": "Washington-Arlington-Alexandria, DC-VA-MD-WV Metro Area",
    "Congress Heights, DC": "Washington-Arlington-Alexandria, DC-VA-MD-WV Metro Area",
    "Dupont Circle, DC": "Washington-Arlington-Alexandria, DC-VA-MD-WV Metro Area",
    "Foggy Bottom, DC": "Washington-Arlington-Alexandria, DC-VA-MD-WV Metro Area",
    "Forest Hills, DC": "Washington-Arlington-Alexandria, DC-VA-MD-WV Metro Area",
    "Georgetown, DC": "Washington-Arlington-Alexandria, DC-VA-MD-WV Metro Area",
    "Glover Park, DC": "Washington-Arlington-Alexandria, DC-VA-MD-WV Metro Area",
    "Kingman Park, DC": "Washington-Arlington-Alexandria, DC-VA-MD-WV Metro Area",
    "Logan Circle, DC": "Washington-Arlington-Alexandria, DC-VA-MD-WV Metro Area",
    "Manor Park, DC": "Washington-Arlington-Alexandria, DC-VA-MD-WV Metro Area",
    "Mount Pleasant, DC": "Washington-Arlington-Alexandria, DC-VA-MD-WV Metro Area",
    "Near Northeast, DC": "Washington-Arlington-Alexandria, DC-VA-MD-WV Metro Area",
    "Petworth, DC": "Washington-Arlington-Alexandria, DC-VA-MD-WV Metro Area",
    "Riggs Park, DC": "Washington-Arlington-Alexandria, DC-VA-MD-WV Metro Area",
    "Shaw, DC": "Washington-Arlington-Alexandria, DC-VA-MD-WV Metro Area",
    "Shipley Terrace, DC": "Washington-Arlington-Alexandria, DC-VA-MD-WV Metro Area",
    "Sixteenth St Heights, DC": "Washington-Arlington-Alexandria, DC-VA-MD-WV Metro Area",
    "Southwest Waterfront, DC": "Washington-Arlington-Alexandria, DC-VA-MD-WV Metro Area",
    "Washington, D.C., DC": "Washington-Arlington-Alexandria, DC-VA-MD-WV Metro Area",
    "Washington Highlands, DC": "Washington-Arlington-Alexandria, DC-VA-MD-WV Metro Area"
}

# Create summary by MSA
msa_summary = {}
for city, msa in lugg_city_to_msa_mapping.items():
    if msa not in msa_summary:
        msa_summary[msa] = []
    msa_summary[msa].append(city)

# Print summary
print(f"Total Lugg cities: {len(lugg_city_to_msa_mapping)}")
print(f"Total unique MSAs covered: {len(msa_summary)}")

# Show top MSAs with most cities
print("\nTop MSAs with most Lugg cities:")
sorted_msas = sorted(msa_summary.items(), key=lambda x: len(x[1]), reverse=True)
for msa, cities in sorted_msas[:10]:
    print(f"{msa}: {len(cities)} cities")

Total Lugg cities: 892
Total unique MSAs covered: 43

Top MSAs with most Lugg cities:
Boston-Cambridge-Newton, MA-NH Metro Area: 73 cities
Philadelphia-Camden-Wilmington, PA-NJ-DE-MD Metro Area: 58 cities
New York-Newark-Jersey City, NY-NJ-PA Metro Area: 52 cities
Washington-Arlington-Alexandria, DC-VA-MD-WV Metro Area: 51 cities
Baltimore-Columbia-Towson, MD Metro Area: 49 cities
Portland-Vancouver-Hillsboro, OR-WA Metro Area: 47 cities
Los Angeles-Long Beach-Anaheim, CA Metro Area: 46 cities
Houston-The Woodlands-Sugar Land, TX Metro Area: 36 cities
Hartford-East Hartford-Middletown, CT Metro Area: 29 cities
Chicago-Naperville-Elgin, IL-IN-WI Metro Area: 28 cities


## Loading Census MSA Data
This section loads the OMB-compliant Metropolitan Statistical Area data from the Census Bureau, which will serve as our reference for mapping company locations to standardized geographic units.

In [10]:
from google.colab import drive
import pandas as pd

# Mount Google Drive and load census data
drive.mount('/content/drive')
census_df = pd.read_csv('/content/drive/MyDrive/Lugg/census_metro_data.csv')

# Basic data exploration
print(f"Census data loaded: {len(census_df)} metro areas")
print(f"Columns: {list(census_df.columns)}")

print("\nSample census MSA names:")
for i in range(10):
    print(f"   {census_df.iloc[i]['metro_area_name']}")

# Analyze naming patterns for standardization
metro_areas = census_df['metro_area_name'].tolist()
patterns = set()
for name in metro_areas[:50]:
    if 'Metro Area' in name:
        patterns.add('Metro Area')
    if 'Micro Area' in name:
        patterns.add('Micro Area')
    if ', ' in name:
        patterns.add('Contains state codes')

print(f"\nNaming patterns found: {sorted(patterns)}")
print(f"Metro vs Micro breakdown:")
print(f"   Metro Areas: {sum('Metro Area' in name for name in metro_areas)}")
print(f"   Micro Areas: {sum('Micro Area' in name for name in metro_areas)}")

Mounted at /content/drive
Census data loaded: 939 metro areas
Columns: ['metro_area_name', 'total_population', 'median_household_income', 'cbsa_code']

Sample census MSA names:
   Aberdeen, SD Micro Area
   Aberdeen, WA Micro Area
   Abilene, TX Metro Area
   Ada, OK Micro Area
   Adrian, MI Micro Area
   Aguadilla-Isabela, PR Metro Area
   Akron, OH Metro Area
   Alamogordo, NM Micro Area
   Albany, GA Metro Area
   Albany-Lebanon, OR Metro Area

Naming patterns found: ['Contains state codes', 'Metro Area', 'Micro Area']
Metro vs Micro breakdown:
   Metro Areas: 392
   Micro Areas: 547


## MSA Coverage Analysis
This section analyzes the Metropolitan Statistical Area coverage for each company after mapping their service locations to OMB-compliant MSA names.

In [11]:
# Extract unique MSAs from each competitor
taskrabbit_msas = set(city_to_msa_mapping.values())
dolly_msas = set(dolly_city_to_msa_mapping.values())
goshare_msas = set(goshare_city_to_msa_mapping.values())
lugg_msas = set(lugg_city_to_msa_mapping.values())

# Display MSA counts per company
print("MSA Coverage by Company:")
print(f"   TaskRabbit: {len(taskrabbit_msas)} MSAs")
print(f"   Dolly: {len(dolly_msas)} MSAs")
print(f"   GoShare: {len(goshare_msas)} MSAs")
print(f"   Lugg: {len(lugg_msas)} MSAs")

# Calculate total unique MSAs across all competitors
all_competitor_msas = taskrabbit_msas | dolly_msas | goshare_msas | lugg_msas
print(f"\nTotal unique MSAs across all companies: {len(all_competitor_msas)}")

# Display sample MSA names to verify OMB compliance
print("\nSample OMB-compliant MSA names:")
for i, msa in enumerate(sorted(list(all_competitor_msas))[:5]):
    print(f"   {msa}")

MSA Coverage by Company:
   TaskRabbit: 52 MSAs
   Dolly: 42 MSAs
   GoShare: 149 MSAs
   Lugg: 43 MSAs

Total unique MSAs across all companies: 161

Sample OMB-compliant MSA names:
   Akron, OH Metro Area
   Albany-Schenectady-Troy, NY Metro Area
   Albuquerque, NM Metro Area
   Allentown-Bethlehem-Easton, PA-NJ Metro Area
   Ann Arbor, MI Metro Area


## OMB Compliance Validation
This section verifies that all mapped MSA names conform to OMB standards for proper integration with mapping tools.

In [12]:
# Check OMB compliance for all mapped MSAs
omb_compliant_count = 0
non_compliant_msas = []

for msa in all_competitor_msas:
    if "Metro Area" in msa or "Micro Area" in msa:
        omb_compliant_count += 1
    else:
        non_compliant_msas.append(msa)

compliance_rate = (omb_compliant_count / len(all_competitor_msas)) * 100

print("OMB Compliance Check:")
print(f"   Total MSAs: {len(all_competitor_msas)}")
print(f"   OMB compliant: {omb_compliant_count} ({compliance_rate:.1f}%)")
print(f"   Non-compliant: {len(non_compliant_msas)}")

# Display non-compliant MSAs if any exist
if non_compliant_msas:
    print("\nNon-compliant MSA names:")
    for msa in non_compliant_msas[:10]:
        print(f"   - {msa}")
    if len(non_compliant_msas) > 10:
        print(f"   ... and {len(non_compliant_msas) - 10} more")
    print(f"\nWARNING: {100 - compliance_rate:.1f}% of MSAs are not OMB compliant")
else:
    print("\nAll MSAs are OMB compliant")

# Display company-specific samples for verification
companies = [
    ("TaskRabbit", taskrabbit_msas),
    ("Dolly", dolly_msas),
    ("GoShare", goshare_msas),
    ("Lugg", lugg_msas)
]

for company_name, company_msas in companies:
    print(f"\nSample {company_name} MSAs ({len(company_msas)} total):")
    for msa in sorted(list(company_msas))[:3]:
        print(f"   {msa}")

OMB Compliance Check:
   Total MSAs: 161
   OMB compliant: 161 (100.0%)
   Non-compliant: 0

All MSAs are OMB compliant

Sample TaskRabbit MSAs (52 total):
   Ann Arbor, MI Metro Area
   Atlanta-Sandy Springs-Alpharetta, GA Metro Area
   Austin-Round Rock-Georgetown, TX Metro Area

Sample Dolly MSAs (42 total):
   Atlanta-Sandy Springs-Alpharetta, GA Metro Area
   Austin-Round Rock-Georgetown, TX Metro Area
   Baltimore-Columbia-Towson, MD Metro Area

Sample GoShare MSAs (149 total):
   Akron, OH Metro Area
   Albany-Schenectady-Troy, NY Metro Area
   Albuquerque, NM Metro Area

Sample Lugg MSAs (43 total):
   Atlanta-Sandy Springs-Alpharetta, GA Metro Area
   Austin-Round Rock-Georgetown, TX Metro Area
   Baltimore-Columbia-Towson, MD Metro Area


## MSA Matching with Census Data
Since all competitor MSA names are OMB-compliant, we use direct exact matching with the Census data for reliable results.

In [13]:
def find_census_match(competitor_msa, census_df):
    """
    Find exact census match for OMB-compliant competitor MSA names.
    Returns the matched row or None if no match found.
    """
    exact_match = census_df[census_df['metro_area_name'] == competitor_msa]

    if not exact_match.empty:
        return exact_match.iloc[0]
    else:
        return None

## Census Data Matching and Quality Analysis
This section matches competitor MSAs with Census data and identifies any unmatched areas for further investigation.

In [14]:
matching_results = []
match_quality = {'exact': 0, 'no_match': 0}

for competitor_msa in all_competitor_msas:
    census_match = find_census_match(competitor_msa, census_df)

    if census_match is not None:
        matching_results.append({
            'competitor_msa': competitor_msa,
            'census_msa': census_match['metro_area_name'],
            'match_type': 'exact',
            'population': census_match['total_population'],
            'income': census_match['median_household_income'],
            'cbsa_code': census_match['cbsa_code']
        })
        match_quality['exact'] += 1
    else:
        matching_results.append({
            'competitor_msa': competitor_msa,
            'census_msa': None,
            'match_type': 'no_match',
            'population': None,
            'income': None,
            'cbsa_code': None
        })
        match_quality['no_match'] += 1

# Display matching quality report
print("Matching quality report:")
print(f"   Exact matches: {match_quality['exact']}")
print(f"   No matches: {match_quality['no_match']}")
print(f"   Match rate: {(match_quality['exact'] / len(all_competitor_msas)) * 100:.1f}%")

# Display unmatched MSAs if any exist
if match_quality['no_match'] > 0:
    print(f"\nUnmatched MSAs ({match_quality['no_match']} total):")
    matching_df = pd.DataFrame(matching_results)
    no_matches = matching_df[matching_df['match_type'] == 'no_match']

    for i, row in no_matches.iterrows():
        print(f"   {i+1}. {row['competitor_msa']}")

    # Search for potential matches in census data
    print("\nSearching for potential matches:")
    for _, row in no_matches.iterrows():
        competitor_msa = row['competitor_msa']
        main_city = competitor_msa.split('-')[0].split(',')[0].strip()

        potential_matches = census_df[census_df['metro_area_name'].str.contains(main_city, case=False, na=False)]

        print(f"\n'{competitor_msa}':")
        if not potential_matches.empty:
            print(f"   Potential matches for '{main_city}':")
            for _, match in potential_matches.head(3).iterrows():
                print(f"      - {match['metro_area_name']}")
        else:
            print(f"   No potential matches found for '{main_city}'")

Matching quality report:
   Exact matches: 157
   No matches: 4
   Match rate: 97.5%

Unmatched MSAs (4 total):
   57. Leominster-Gardner, MA Metro Area
   88. New Bedford, MA Metro Area
   137. Waterbury, CT Metro Area
   148. Seaford, DE Micro Area

Searching for potential matches:

'Leominster-Gardner, MA Metro Area':
   No potential matches found for 'Leominster'

'New Bedford, MA Metro Area':
   No potential matches found for 'New Bedford'

'Waterbury, CT Metro Area':
   No potential matches found for 'Waterbury'

'Seaford, DE Micro Area':
   No potential matches found for 'Seaford'


## Competitor Activity Matrix Creation
This section creates a comprehensive activity matrix showing which competitors operate in each validated MSA, using Census-compliant MSA names for downstream analysis.

In [15]:
# Filter to validated matches only
validated_matches = matching_df[matching_df['match_type'] != 'no_match'].copy()
print(f"Proceeding with {len(validated_matches)} validated MSA matches")

# Create competitor activity matrix
competitor_activity = {}

for _, row in validated_matches.iterrows():
    competitor_msa = row['competitor_msa']
    census_msa = row['census_msa']

    # Initialize MSA entry with Census data (using Census MSA as key for OMB compliance)
    if census_msa not in competitor_activity:
        competitor_activity[census_msa] = {
            'competitor_msa_original': competitor_msa,
            'census_msa': census_msa,
            'population': row['population'],
            'income': row['income'],
            'cbsa_code': row['cbsa_code'],
            'taskrabbit_active': 0,
            'dolly_active': 0,
            'goshare_active': 0,
            'lugg_active': 0,
            'match_type': row['match_type']
        }

    # Set competitor presence flags
    if competitor_msa in taskrabbit_msas:
        competitor_activity[census_msa]['taskrabbit_active'] = 1
    if competitor_msa in dolly_msas:
        competitor_activity[census_msa]['dolly_active'] = 1
    if competitor_msa in goshare_msas:
        competitor_activity[census_msa]['goshare_active'] = 1
    if competitor_msa in lugg_msas:
        competitor_activity[census_msa]['lugg_active'] = 1

# Calculate total competitor count per MSA
for census_msa in competitor_activity:
    activity = competitor_activity[census_msa]
    activity['total_competitors'] = (
        activity['taskrabbit_active'] + activity['dolly_active'] +
        activity['goshare_active'] + activity['lugg_active']
    )

print(f"Competitor activity matrix created for {len(competitor_activity)} MSAs")

# Quick validation summary
activity_df = pd.DataFrame.from_dict(competitor_activity, orient='index')
print(f"\nActivity Summary:")
print(f"   MSAs with Lugg: {activity_df['lugg_active'].sum()}")
print(f"   MSAs with TaskRabbit: {activity_df['taskrabbit_active'].sum()}")
print(f"   MSAs with Dolly: {activity_df['dolly_active'].sum()}")
print(f"   MSAs with GoShare: {activity_df['goshare_active'].sum()}")
print(f"   Average competitors per MSA: {activity_df['total_competitors'].mean():.1f}")

Proceeding with 157 validated MSA matches
Competitor activity matrix created for 157 MSAs

Activity Summary:
   MSAs with Lugg: 40
   MSAs with TaskRabbit: 52
   MSAs with Dolly: 42
   MSAs with GoShare: 146
   Average competitors per MSA: 1.8


## Sample Mapping Verification
This section displays a sample of MSA mappings to verify the accuracy of the matching process.

In [16]:
# Display sample mappings for manual verification
print("SAMPLE MAPPINGS FOR VERIFICATION:")
print(f"{'Original MSA':<50} {'Census MSA':<50} {'Match Type':<15}")
print("-" * 115)

sample_mappings = activity_df[['competitor_msa_original', 'census_msa', 'match_type']].head(10)
for idx, row in sample_mappings.iterrows():
    # Truncate long MSA names for display formatting
    orig_short = str(row['competitor_msa_original'])
    if len(orig_short) > 47:
        orig_short = orig_short[:47] + "..."

    census_short = str(row['census_msa'])
    if len(census_short) > 47:
        census_short = census_short[:47] + "..."

    match_type_str = str(row['match_type'])

    print(f"{orig_short:<50} {census_short:<50} {match_type_str:<15}")

# Display mapping statistics by match type
print(f"\nMapping Statistics by Match Type:")
match_type_counts = activity_df['match_type'].value_counts()
for match_type, count in match_type_counts.items():
    print(f"   {match_type}: {count} MSAs")

SAMPLE MAPPINGS FOR VERIFICATION:
Original MSA                                       Census MSA                                         Match Type     
-------------------------------------------------------------------------------------------------------------------
New York-Newark-Jersey City, NY-NJ-PA Metro Are... New York-Newark-Jersey City, NY-NJ-PA Metro Are... exact          
Lake City, FL Micro Area                           Lake City, FL Micro Area                           exact          
Richmond, VA Metro Area                            Richmond, VA Metro Area                            exact          
Cincinnati, OH-KY-IN Metro Area                    Cincinnati, OH-KY-IN Metro Area                    exact          
Worcester, MA-CT Metro Area                        Worcester, MA-CT Metro Area                        exact          
Daphne-Fairhope-Foley, AL Metro Area               Daphne-Fairhope-Foley, AL Metro Area               exact          
St. Cloud, MN Metro Area

## Final Market Analysis Summary
This section provides comprehensive market presence analysis and key insights for strategic planning.

In [17]:
# Convert competitor activity to final analysis dataframe
activity_df = pd.DataFrame.from_dict(competitor_activity, orient='index')
activity_df.reset_index(inplace=True)
activity_df.rename(columns={'index': 'msa_name'}, inplace=True)

# Display competitor market presence
print("COMPETITOR MARKET PRESENCE:")
print(f"   TaskRabbit active in: {activity_df['taskrabbit_active'].sum()} MSAs")
print(f"   Dolly active in: {activity_df['dolly_active'].sum()} MSAs")
print(f"   GoShare active in: {activity_df['goshare_active'].sum()} MSAs")
print(f"   Lugg active in: {activity_df['lugg_active'].sum()} MSAs")

# Analyze competition intensity across markets
print("\nCOMPETITION INTENSITY:")
competition_summary = activity_df['total_competitors'].value_counts().sort_index()
for num_competitors, count in competition_summary.items():
    print(f"   {num_competitors} competitors: {count} MSAs")

# Market demographics analysis
markets_with_data = activity_df[activity_df['population'].notna()].copy()
print(f"\nMARKET DEMOGRAPHICS:")
print(f"   MSAs with complete data: {len(markets_with_data)}")

if len(markets_with_data) > 0:
    print(f"   Population range: {markets_with_data['population'].min():,.0f} to {markets_with_data['population'].max():,.0f}")
    print(f"   Income range: ${markets_with_data['income'].min():,.0f} to ${markets_with_data['income'].max():,.0f}")

    # Display top markets by population with competitor presence
    print("\nTOP MARKETS BY POPULATION:")
    top_markets = markets_with_data.nlargest(5, 'population')
    for _, row in top_markets.iterrows():
        competitors = row['total_competitors']
        pop_millions = row['population'] / 1_000_000
        print(f"   {row['msa_name']}")
        print(f"      Population: {pop_millions:.1f}M | Competitors: {competitors} | Income: ${row['income']:,.0f}")

print(f"\nData processing complete. Final dataset contains {len(activity_df)} MSAs ready for analysis.")

COMPETITOR MARKET PRESENCE:
   TaskRabbit active in: 52 MSAs
   Dolly active in: 42 MSAs
   GoShare active in: 146 MSAs
   Lugg active in: 40 MSAs

COMPETITION INTENSITY:
   1 competitors: 96 MSAs
   2 competitors: 20 MSAs
   3 competitors: 20 MSAs
   4 competitors: 21 MSAs

MARKET DEMOGRAPHICS:
   MSAs with complete data: 157
   Population range: 63,594 to 19,908,595
   Income range: $41,020 to $151,713

TOP MARKETS BY POPULATION:
   New York-Newark-Jersey City, NY-NJ-PA Metro Area
      Population: 19.9M | Competitors: 4 | Income: $93,610
   Los Angeles-Long Beach-Anaheim, CA Metro Area
      Population: 13.1M | Competitors: 4 | Income: $89,105
   Chicago-Naperville-Elgin, IL-IN-WI Metro Area
      Population: 9.6M | Competitors: 4 | Income: $85,087
   Dallas-Fort Worth-Arlington, TX Metro Area
      Population: 7.7M | Competitors: 4 | Income: $83,398
   Houston-The Woodlands-Sugar Land, TX Metro Area
      Population: 7.1M | Competitors: 4 | Income: $78,061

Data processing complete

## Market Gap Analysis
This section identifies significant metropolitan areas not served by any competitor, representing potential expansion opportunities.

In [18]:
# Identify markets not covered by any competitor
matched_census_msas = set(matching_df[matching_df['census_msa'].notna()]['census_msa'].tolist())
all_census_msas = set(census_df['metro_area_name'].tolist())
uncovered_markets = all_census_msas - matched_census_msas

# Filter to significant uncovered markets with strong demographics
significant_uncovered = census_df[
    (census_df['metro_area_name'].isin(uncovered_markets)) &
    ((census_df['total_population'] > 500000) | (census_df['median_household_income'] > 70000))
].sort_values('total_population', ascending=False)

print("MARKET GAP ANALYSIS:")
print(f"   Total census MSAs not served by any competitor: {len(uncovered_markets)}")
print(f"   Significant uncovered markets (Pop > 500K or Income > $70K): {len(significant_uncovered)}")

# Display top expansion opportunities
if len(significant_uncovered) > 0:
    print("\nTOP EXPANSION OPPORTUNITIES:")
    print(f"{'Rank':<4} {'Market Name':<50} {'Population':<12} {'Median Income':<15}")
    print("-" * 85)

    for i, (_, row) in enumerate(significant_uncovered.head(10).iterrows(), 1):
        market_name = row['metro_area_name']
        if len(market_name) > 47:
            market_name = market_name[:47] + "..."

        print(f"{i:2}.  {market_name:<50} "
              f"{row['total_population']:>10,} "
              f"${row['median_household_income']:>12,}")

# Summary statistics for uncovered markets
if len(significant_uncovered) > 0:
    total_uncovered_pop = significant_uncovered['total_population'].sum()
    avg_income = significant_uncovered['median_household_income'].mean()

    print(f"\nUNCOVERED MARKET POTENTIAL:")
    print(f"   Combined population: {total_uncovered_pop:,}")
    print(f"   Average median income: ${avg_income:,.0f}")
    print(f"   Largest uncovered market: {significant_uncovered.iloc[0]['metro_area_name']}")

MARKET GAP ANALYSIS:
   Total census MSAs not served by any competitor: 782
   Significant uncovered markets (Pop > 500K or Income > $70K): 164

TOP EXPANSION OPPORTUNITIES:
Rank Market Name                                        Population   Median Income  
-------------------------------------------------------------------------------------
 1.  San Juan-Bayamón-Caguas, PR Metro Area              2,073,546 $      26,853
 2.  Grand Rapids-Kentwood, MI Metro Area                1,087,068 $      76,898
 3.  McAllen-Edinburg-Mission, TX Metro Area               873,167 $      49,371
 4.  Lakeland-Winter Haven, FL Metro Area                  736,229 $      60,901
 5.  Poughkeepsie-Newburgh-Middletown, NY Metro Area       697,704 $      92,870
 6.  Madison, WI Metro Area                                678,995 $      82,996
 7.  Deltona-Daytona Beach-Ormond Beach, FL Metro Ar...    676,035 $      64,197
 8.  Syracuse, NY Metro Area                               658,694 $      70,096
 9.  Au

## Comprehensive Dataset Creation
This section combines competitor-mapped markets with uncovered expansion opportunities to create the final analysis dataset.

In [19]:
# Display final matching statistics
final_match_quality = matching_df['match_type'].value_counts()
print("FINAL MATCHING QUALITY:")
for match_type, count in final_match_quality.items():
    print(f"   {match_type}: {count}")

total_matched = len(matching_df[matching_df['match_type'] != 'no_match'])
match_rate = (total_matched / len(matching_df)) * 100
print(f"   Final match rate: {match_rate:.1f}%")

# Create comprehensive dataset combining competitor markets and opportunities
comprehensive_data = []

# Add successfully matched competitor MSAs
for _, row in matching_df.iterrows():
    if row['match_type'] != 'no_match':
        msa = row['competitor_msa']
        comprehensive_data.append({
            'metro_area_name': row['census_msa'],
            'competitor_msa': msa,
            'match_type': row['match_type'],
            'population': row['population'],
            'income': row['income'],
            'cbsa_code': row['cbsa_code'],
            'taskrabbit_active': 1 if msa in taskrabbit_msas else 0,
            'dolly_active': 1 if msa in dolly_msas else 0,
            'goshare_active': 1 if msa in goshare_msas else 0,
            'lugg_active': 1 if msa in lugg_msas else 0,
            'data_source': 'competitor_mapped'
        })

# Add significant uncovered markets as expansion opportunities
for _, row in significant_uncovered.iterrows():
    comprehensive_data.append({
        'metro_area_name': row['metro_area_name'],
        'competitor_msa': None,
        'match_type': 'census_only',
        'population': row['total_population'],
        'income': row['median_household_income'],
        'cbsa_code': row['cbsa_code'],
        'taskrabbit_active': 0,
        'dolly_active': 0,
        'goshare_active': 0,
        'lugg_active': 0,
        'data_source': 'uncovered_opportunity'
    })

# Convert to DataFrame and calculate competitor metrics
comprehensive_df = pd.DataFrame(comprehensive_data)
comprehensive_df['total_competitors'] = (
    comprehensive_df['taskrabbit_active'] + comprehensive_df['dolly_active'] +
    comprehensive_df['goshare_active'] + comprehensive_df['lugg_active']
)

# Display comprehensive dataset summary
print("\nCOMPREHENSIVE DATASET SUMMARY:")
competitor_mapped = len(comprehensive_df[comprehensive_df['data_source'] == 'competitor_mapped'])
uncovered_opps = len(comprehensive_df[comprehensive_df['data_source'] == 'uncovered_opportunity'])

print(f"   Competitor-mapped markets: {competitor_mapped}")
print(f"   Uncovered opportunities: {uncovered_opps}")
print(f"   Total markets for analysis: {len(comprehensive_df)}")

# Market potential summary
total_population = comprehensive_df['population'].sum()
avg_income = comprehensive_df['income'].mean()

print(f"\nMARKET POTENTIAL:")
print(f"   Combined population: {total_population:,.0f}")
print(f"   Average median income: ${avg_income:,.0f}")
print(f"   Markets with zero competition: {len(comprehensive_df[comprehensive_df['total_competitors'] == 0])}")

FINAL MATCHING QUALITY:
   exact: 157
   no_match: 4
   Final match rate: 97.5%

COMPREHENSIVE DATASET SUMMARY:
   Competitor-mapped markets: 157
   Uncovered opportunities: 164
   Total markets for analysis: 321

MARKET POTENTIAL:
   Combined population: 264,676,273
   Average median income: $76,453
   Markets with zero competition: 164


## Market Scoring and Expansion Prioritization
This section calculates market attractiveness and competitive opportunity scores to prioritize expansion targets for Lugg.

In [20]:
# Filter to markets with complete census data for scoring
markets_for_analysis = comprehensive_df[comprehensive_df['population'].notna()].copy()
print(f"Markets with complete data for analysis: {len(markets_for_analysis)}")

print("\nCalculating market attractiveness scores...")

# Population score (0-50 points, normalized by percentile ranking)
markets_for_analysis['pop_percentile'] = markets_for_analysis['population'].rank(pct=True) * 100
markets_for_analysis['pop_score'] = (markets_for_analysis['pop_percentile'] / 100) * 50

# Income score (0-30 points, normalized by percentile ranking)
markets_for_analysis['income_percentile'] = markets_for_analysis['income'].rank(pct=True) * 100
markets_for_analysis['income_score'] = (markets_for_analysis['income_percentile'] / 100) * 30

# Market size category bonus (0-20 points based on population tiers)
def categorize_market_size(population):
    if population >= 5000000:
        return 20  # Mega markets (5M+)
    elif population >= 2000000:
        return 15  # Major markets (2-5M)
    elif population >= 1000000:
        return 12  # Large markets (1-2M)
    elif population >= 500000:
        return 8   # Medium markets (500K-1M)
    elif population >= 250000:
        return 4   # Small markets (250K-500K)
    else:
        return 0   # Micro markets (<250K)

markets_for_analysis['size_bonus'] = markets_for_analysis['population'].apply(categorize_market_size)

# Calculate total market attractiveness score (0-100 scale)
markets_for_analysis['market_attractiveness'] = (
    markets_for_analysis['pop_score'] +
    markets_for_analysis['income_score'] +
    markets_for_analysis['size_bonus']
)

print("Market attractiveness calculated (0-100 scale)")

# Calculate competitive opportunity scores
print("\nCalculating competitive opportunity scores...")

# Competition opportunity score (fewer competitors = higher opportunity)
max_competitors = markets_for_analysis['total_competitors'].max()
markets_for_analysis['competition_opportunity'] = (
    (max_competitors - markets_for_analysis['total_competitors']) / max_competitors * 100
)

# Classify markets by Lugg presence
markets_for_analysis['lugg_status'] = markets_for_analysis['lugg_active'].apply(
    lambda x: 'Current Market' if x == 1 else 'Expansion Target'
)

# Calculate expansion priority scores for target markets
print("\nCalculating expansion priority scores...")

expansion_targets = markets_for_analysis[markets_for_analysis['lugg_active'] == 0].copy()

if len(expansion_targets) > 0:
    # Expansion score: weighted combination of attractiveness and opportunity
    expansion_targets['expansion_score'] = (
        expansion_targets['market_attractiveness'] * 0.6 +
        expansion_targets['competition_opportunity'] * 0.4
    )

    # Revenue potential estimate (simplified model)
    # Assumptions: $50 AOV, 0.1% market penetration, 12 orders/year per customer
    expansion_targets['estimated_annual_revenue'] = (
        expansion_targets['population'] * 0.001 * 50 * 12
    ).round(-3)  # Round to nearest thousand

    print(f"Expansion scoring complete for {len(expansion_targets)} target markets")

    # Display top expansion opportunities
    print("\nTOP 10 EXPANSION OPPORTUNITIES:")
    print(f"{'Rank':<4} {'Market Name':<45} {'Score':<6} {'Pop':<10} {'Revenue Est':<12}")
    print("-" * 85)

    top_targets = expansion_targets.nlargest(10, 'expansion_score')
    for i, (_, row) in enumerate(top_targets.iterrows(), 1):
        market_name = row['metro_area_name']
        if len(market_name) > 42:
            market_name = market_name[:42] + "..."

        print(f"{i:2}.  {market_name:<45} "
              f"{row['expansion_score']:5.1f} "
              f"{row['population']/1000:7.0f}K "
              f"${row['estimated_annual_revenue']/1000:8.0f}K")
else:
    print("No expansion targets found (Lugg present in all analyzed markets)")

Markets with complete data for analysis: 321

Calculating market attractiveness scores...
Market attractiveness calculated (0-100 scale)

Calculating competitive opportunity scores...

Calculating expansion priority scores...
Expansion scoring complete for 281 target markets

TOP 10 EXPANSION OPPORTUNITIES:
Rank Market Name                                   Score  Pop        Revenue Est 
-------------------------------------------------------------------------------------
 1.  Poughkeepsie-Newburgh-Middletown, NY Metro...  83.2     698K $     419K
 2.  Grand Rapids-Kentwood, MI Metro Area           82.9    1087K $     652K
 3.  Madison, WI Metro Area                         80.6     679K $     407K
 4.  Riverside-San Bernardino-Ontario, CA Metro...  80.6    4610K $    2766K
 5.  Portland-South Portland, ME Metro Area         79.4     553K $     332K
 6.  Urban Honolulu, HI Metro Area                  79.3    1010K $     606K
 7.  Lancaster, PA Metro Area                       78.2     

## Final Market Analysis Report
This section provides the complete strategic analysis with professional formatting for executive presentation.

In [21]:
# Market summary statistics
print("\nMARKET SUMMARY STATISTICS:")
print(f"Total markets analyzed: {len(markets_for_analysis)}")
print(f"Current Lugg markets: {markets_for_analysis['lugg_active'].sum()}")
print(f"Expansion targets: {len(expansion_targets) if len(expansion_targets) > 0 else 0}")
print(f"Average market population: {markets_for_analysis['population'].mean():,.0f}")
print(f"Average market income: ${markets_for_analysis['income'].mean():,.0f}")

# Competition landscape analysis
print("\nCOMPETITION LANDSCAPE:")
competitor_presence = {
    'TaskRabbit': markets_for_analysis['taskrabbit_active'].sum(),
    'Dolly': markets_for_analysis['dolly_active'].sum(),
    'GoShare': markets_for_analysis['goshare_active'].sum(),
    'Lugg': markets_for_analysis['lugg_active'].sum()
}

for competitor, count in competitor_presence.items():
    percentage = (count / len(markets_for_analysis)) * 100
    print(f"  {competitor}: {count} markets ({percentage:.1f}%)")

# Competition intensity breakdown
print("\nCOMPETITION INTENSITY:")
competition_breakdown = markets_for_analysis['total_competitors'].value_counts().sort_index()
for competitors, count in competition_breakdown.items():
    percentage = (count / len(markets_for_analysis)) * 100
    print(f"  {competitors} competitors: {count} markets ({percentage:.1f}%)")

# TOP EXPANSION OPPORTUNITIES
if len(expansion_targets) > 0:
    top_opportunities = expansion_targets.nlargest(15, 'expansion_score')

    print("\nTOP 15 EXPANSION OPPORTUNITIES:")
    print(f"{'Rank':<4} {'Metro Area':<45} {'Pop (M)':<8} {'Income':<8} {'Comp':<4} {'Score':<6} {'Est Revenue':<12}")
    print("-" * 95)

    for rank, (_, row) in enumerate(top_opportunities.iterrows(), 1):
        metro_short = (row['metro_area_name'][:42] + "...") if len(row['metro_area_name']) > 45 else row['metro_area_name']
        pop_display = f"{row['population']/1000000:.1f}M"
        income_display = f"${row['income']/1000:.0f}K"
        competitors = row['total_competitors']
        score = f"{row['expansion_score']:.1f}"
        revenue = f"${row['estimated_annual_revenue']/1000:.0f}K"

        print(f"{rank:<4} {metro_short:<45} {pop_display:<8} {income_display:<8} {competitors:<4} {score:<6} {revenue:<12}")

# MARKET TIERS ANALYSIS
if len(expansion_targets) > 0:
    print("\nMARKET TIER ANALYSIS:")

    # Categorize opportunities
    tier_1 = expansion_targets[
        (expansion_targets['population'] >= 2000000) &
        (expansion_targets['total_competitors'] <= 2)
    ]

    tier_2 = expansion_targets[
        (expansion_targets['population'] >= 1000000) &
        (expansion_targets['population'] < 2000000) &
        (expansion_targets['total_competitors'] <= 2)
    ]

    tier_3 = expansion_targets[
        (expansion_targets['population'] >= 500000) &
        (expansion_targets['population'] < 1000000) &
        (expansion_targets['total_competitors'] <= 1)
    ]

    print(f"  TIER 1 (Major metros, ≤2 competitors): {len(tier_1)} markets")
    if len(tier_1) > 0:
        for _, row in tier_1.nlargest(5, 'expansion_score').iterrows():
            print(f"    - {row['metro_area_name'][:50]} (Score: {row['expansion_score']:.1f})")

    print(f"  TIER 2 (Large metros, ≤2 competitors): {len(tier_2)} markets")
    if len(tier_2) > 0:
        for _, row in tier_2.nlargest(3, 'expansion_score').iterrows():
            print(f"    - {row['metro_area_name'][:50]} (Score: {row['expansion_score']:.1f})")

    print(f"  TIER 3 (Medium metros, ≤1 competitor): {len(tier_3)} markets")
    if len(tier_3) > 0:
        for _, row in tier_3.nlargest(3, 'expansion_score').iterrows():
            print(f"    - {row['metro_area_name'][:50]} (Score: {row['expansion_score']:.1f})")

# STRATEGIC RECOMMENDATIONS
print("\nSTRATEGIC RECOMMENDATIONS:")
if len(expansion_targets) > 0:
    # Immediate priorities (top 3)
    immediate_priorities = top_opportunities.head(3)
    total_immediate_revenue = immediate_priorities['estimated_annual_revenue'].sum()

    print("  IMMEDIATE PRIORITIES (Top 3 markets):")
    print(f"    Combined revenue potential: ${total_immediate_revenue/1000:.0f}K annually")
    for _, row in immediate_priorities.iterrows():
        print(f"    - {row['metro_area_name']}")
        print(f"      Population: {row['population']:,}, Income: ${row['income']:,}")
        print(f"      Competitors: {row['total_competitors']}, Score: {row['expansion_score']:.1f}")

    # Medium-term targets (next 5)
    medium_term = top_opportunities.iloc[3:8]
    if len(medium_term) > 0:
        print("\n  MEDIUM-TERM TARGETS (Next 5 markets):")
        for _, row in medium_term.iterrows():
            print(f"    - {row['metro_area_name'][:50]} (Score: {row['expansion_score']:.1f})")

    # Market entry strategy insights
    zero_competition = expansion_targets[expansion_targets['total_competitors'] == 0]
    light_competition = expansion_targets[expansion_targets['total_competitors'] == 1]

    print("\n  MARKET ENTRY INSIGHTS:")
    print(f"    Zero competition markets: {len(zero_competition)} (easy entry)")
    print(f"    Light competition markets: {len(light_competition)} (moderate entry)")
    print("    Avoid markets with 3+ competitors initially")


MARKET SUMMARY STATISTICS:
Total markets analyzed: 321
Current Lugg markets: 40
Expansion targets: 281
Average market population: 824,537
Average market income: $76,453

COMPETITION LANDSCAPE:
  TaskRabbit: 52 markets (16.2%)
  Dolly: 42 markets (13.1%)
  GoShare: 146 markets (45.5%)
  Lugg: 40 markets (12.5%)

COMPETITION INTENSITY:
  0 competitors: 164 markets (51.1%)
  1 competitors: 96 markets (29.9%)
  2 competitors: 20 markets (6.2%)
  3 competitors: 20 markets (6.2%)
  4 competitors: 21 markets (6.5%)

TOP 15 EXPANSION OPPORTUNITIES:
Rank Metro Area                                    Pop (M)  Income   Comp Score  Est Revenue 
-----------------------------------------------------------------------------------------------
1    Poughkeepsie-Newburgh-Middletown, NY Metro... 0.7M     $93K     0    83.2   $419K       
2    Grand Rapids-Kentwood, MI Metro Area          1.1M     $77K     0    82.9   $652K       
3    Madison, WI Metro Area                        0.7M     $83K     0    

## Data Export and File Generation
This section merges expansion scores back into the main dataset, exports three different CSV files for various use cases, and downloads them from the Colab environment for further analysis.

In [22]:
# Export comprehensive dataset with all scores
if len(expansion_targets) > 0:
   # Add expansion targets back to main dataset
   markets_for_analysis = markets_for_analysis.merge(
       expansion_targets[['metro_area_name', 'expansion_score', 'estimated_annual_revenue']],
       on='metro_area_name',
       how='left'
   )

# Fill NaN expansion scores for current Lugg markets
markets_for_analysis['expansion_score'] = markets_for_analysis['expansion_score'].fillna(0)
markets_for_analysis['estimated_annual_revenue'] = markets_for_analysis['estimated_annual_revenue'].fillna(0)

# Export files
export_columns = [
   'metro_area_name', 'population', 'income', 'cbsa_code',
   'taskrabbit_active', 'dolly_active', 'goshare_active', 'lugg_active',
   'total_competitors', 'lugg_status', 'market_attractiveness',
   'competition_opportunity', 'expansion_score', 'estimated_annual_revenue',
   'data_source'
]

# 1. Complete strategic analysis
strategic_df = markets_for_analysis[export_columns].sort_values('expansion_score', ascending=False)
strategic_df.to_csv('lugg_strategic_market_analysis.csv', index=False)
print("Complete analysis: lugg_strategic_market_analysis.csv")

# 2. Top expansion opportunities only
if len(expansion_targets) > 0:
   top_15_opportunities = expansion_targets.nlargest(15, 'expansion_score')[export_columns]
   top_15_opportunities.to_csv('lugg_top_15_expansion_opportunities.csv', index=False)
   print("Top opportunities: lugg_top_15_expansion_opportunities.csv")

# 3. Executive summary
if len(expansion_targets) > 0:
   executive_summary = top_opportunities.head(10)[[
       'metro_area_name', 'population', 'income', 'total_competitors',
       'expansion_score', 'estimated_annual_revenue'
   ]].copy()

   executive_summary.columns = [
       'Metro Area', 'Population', 'Median Income', 'Competitors',
       'Expansion Score', 'Est Annual Revenue'
   ]

   executive_summary.to_csv('lugg_executive_summary.csv', index=False)
   print("Executive summary: lugg_executive_summary.csv")

# Download files (for Google Colab environment)
from google.colab import files

file_list = [
  'lugg_strategic_market_analysis.csv',
  'lugg_top_15_expansion_opportunities.csv',
  'lugg_executive_summary.csv',
]

print("\nDownloading files...")
for filename in file_list:
  try:
      files.download(filename)
      print(f"Downloaded: {filename}")
  except Exception as e:
      print(f"Error downloading {filename}: {str(e)}")

print("Download complete!")

Complete analysis: lugg_strategic_market_analysis.csv
Top opportunities: lugg_top_15_expansion_opportunities.csv
Executive summary: lugg_executive_summary.csv



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Downloaded: lugg_strategic_market_analysis.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Downloaded: lugg_top_15_expansion_opportunities.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Downloaded: lugg_executive_summary.csv
Download complete!
